In \[114\]:

    import re
    import boto3
    import sagemaker
    from sagemaker import get_execution_role

    sess = sagemaker.Session()

    role = get_execution_role()

In \[41\]:

    #Downloading the csv data file from the S3 bucket
    s3 = boto3.client("s3")
    bucket_name = 'linear-regression-xxxxxxxxxx-bucket'
    prefix = 'housing_cleaned_data.csv'
    s3.download_file(bucket_name,prefix,'housing_data.csv')

In \[42\]:

    import pandas as pd

    #Creating a pandas data frome from the data in the csv file
    housing_data_df = pd.read_csv('housing_data.csv')
    #Viewing the sample first 5 rows of data
    print(housing_data_df.head(5))

       housing_median_age  total_rooms  total_bedrooms  population  \
    0                  41          880             129         322   
    1                  21         7099            1106        2401   
    2                  52         1467             190         496   
    3                  52         1274             235         558   
    4                  52         1627             280         565   

       median_house_value  
    0              452600  
    1              358500  
    2              352100  
    3              341300  
    4              342200  

In \[67\]:

    #The X dataframe will contain the columns that will be used to predict the house value i.e. the predictors
    X = housing_data_df[['housing_median_age','total_rooms','total_bedrooms','population']]
    #The Y dataframe will contain the housing price data i.e. the predicted value 
    Y = housing_data_df[['median_house_value']]

    #The below code will split the data randomly into training data and testing data; the test data will comprise 33% of the total data
    #and the training data will comprise 67% of the total data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0)

In \[97\]:

    import io
    import numpy as np
    import sagemaker.amazon.common as smac

    #The below code converts the dataframes into Numpy arrays with datatype float32
    x_train = np.array(X_train).astype('float32')
    y_train = np.array(y_train).astype('float32')

    #The below reshapes the data for training purposes
    label = np.reshape(y_train,-1)
    vector = x_train
    print(label.shape)
    print(vector.shape)
    buf = io.BytesIO()
    smac.write_numpy_to_dense_tensor(buf, vector, label)
    buf.seek(0)

    (13690,)
    (13690, 4)

Out\[97\]:

    0

In \[98\]:

    #The converted training data is uploaded to the S3 bucket
    boto3.resource("s3").Bucket(bucket_name).Object('train_data').upload_fileobj(buf)

In \[99\]:

    from sagemaker.amazon.amazon_estimator import get_image_uri
    #Downloading the linear learner image 
    container = get_image_uri(boto3.Session().region_name, 'linear-learner')

    sess = sagemaker.Session()

    #The below code initializes the ML object that will be the model which will predict the house value
    linear = sagemaker.estimator.Estimator(container,
              role, 
              train_instance_count = 1, 
              train_instance_type = 'ml.c4.xlarge',
              output_path = "s3://linear-regression-xxxxxxxxxxxxx-bucket/linear_regression_model",
              sagemaker_session = sagemaker_session)


    linear.set_hyperparameters(feature_dim = 4,
                               predictor_type = 'regressor',
                               mini_batch_size = 5,
                               epochs = 5,
                               num_models = 32,
                               loss = 'absolute_loss')

    s3_train_data = 's3://linear-regression-xxxxxxxx-bucket/train_data'
    #The below code trains the ML model that was initialized before 
    #and will upload this model into the S3 bucket under the train_data prefix 
    linear.fit({"train": s3_train_data})

    'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
    Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
    's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.

    2021-06-23 13:51:11 Starting - Starting the training job...
    2021-06-23 13:51:13 Starting - Launching requested ML instances......
    2021-06-23 13:52:16 Starting - Preparing the instances for training......
    2021-06-23 13:53:34 Downloading - Downloading input data...
    2021-06-23 13:54:00 Training - Downloading the training image..Docker entrypoint called with argument(s): train
    Running default environment configuration script
    [06/23/2021 13:54:26 INFO 139646453495616] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta': '1.0', 'num_classes': '1', 'accuracy_top_k': '3', 'wd': 'auto', 'l1': 'auto', 'momentum': 'auto', 'learning_rate': 'auto', 'beta_1': 'auto', 'beta_2': 'auto', 'bias_lr_mult': 'auto', 'bias_wd_mult': 'auto', 'use_lr_scheduler': 'true', 'lr_scheduler_step': 'auto', 'lr_scheduler_factor': 'auto', 'lr_scheduler_minimum_lr': 'auto', 'positive_example_weight_mult': '1.0', 'balance_multiclass_weights': 'false', 'normalize_data': 'true', 'normalize_label': 'auto', 'unbias_data': 'auto', 'unbias_label': 'auto', 'num_point_for_scaler': '10000', '_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_log_level': 'info', '_tuning_objective_metric': '', 'early_stopping_patience': '3', 'early_stopping_tolerance': '0.001', '_enable_profiler': 'false'}
    [06/23/2021 13:54:26 INFO 139646453495616] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {'loss': 'absolute_loss', 'feature_dim': '4', 'num_models': '32', 'predictor_type': 'regressor', 'epochs': '5', 'mini_batch_size': '5'}
    [06/23/2021 13:54:26 INFO 139646453495616] Final configuration: {'mini_batch_size': '5', 'epochs': '5', 'feature_dim': '4', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': '32', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'absolute_loss', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta': '1.0', 'num_classes': '1', 'accuracy_top_k': '3', 'wd': 'auto', 'l1': 'auto', 'momentum': 'auto', 'learning_rate': 'auto', 'beta_1': 'auto', 'beta_2': 'auto', 'bias_lr_mult': 'auto', 'bias_wd_mult': 'auto', 'use_lr_scheduler': 'true', 'lr_scheduler_step': 'auto', 'lr_scheduler_factor': 'auto', 'lr_scheduler_minimum_lr': 'auto', 'positive_example_weight_mult': '1.0', 'balance_multiclass_weights': 'false', 'normalize_data': 'true', 'normalize_label': 'auto', 'unbias_data': 'auto', 'unbias_label': 'auto', 'num_point_for_scaler': '10000', '_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_log_level': 'info', '_tuning_objective_metric': '', 'early_stopping_patience': '3', 'early_stopping_tolerance': '0.001', '_enable_profiler': 'false', 'predictor_type': 'regressor'}
    [06/23/2021 13:54:26 WARNING 139646453495616] Loggers have already been setup.
    Process 1 is a worker.
    [06/23/2021 13:54:26 INFO 139646453495616] Using default worker.
    [06/23/2021 13:54:26 INFO 139646453495616] Checkpoint loading and saving are disabled.
    [2021-06-23 13:54:26.496] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 0, "duration": 24, "num_examples": 1, "num_bytes": 300}
    [06/23/2021 13:54:26 INFO 139646453495616] Create Store: local

    2021-06-23 13:54:21 Training - Training image download completed. Training in progress.[2021-06-23 13:54:39.340] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 1, "duration": 12843, "num_examples": 2001, "num_bytes": 600300}
    [06/23/2021 13:54:39 INFO 139646453495616] Scaler algorithm parameters
     <algorithm.scaler.ScalerAlgorithmStable object at 0x7f0198a1c890>
    [06/23/2021 13:54:39 INFO 139646453495616] Scaling model computed with parameters:
     {'stdev_label': 
    [115319.11]
    <NDArray 1 @cpu(0)>, 'stdev_weight': 
    [  12.556476 2141.4397    415.49002  1096.5479  ]
    <NDArray 4 @cpu(0)>, 'mean_label': 
    [206232.38]
    <NDArray 1 @cpu(0)>, 'mean_weight': 
    [  28.732147 2618.113     534.69     1412.4558  ]
    <NDArray 4 @cpu(0)>}
    [06/23/2021 13:54:39 INFO 139646453495616] nvidia-smi: took 0.030 seconds to run.
    [06/23/2021 13:54:39 INFO 139646453495616] nvidia-smi identified 0 GPUs.
    [06/23/2021 13:54:39 INFO 139646453495616] Number of GPUs being used: 0
    #metrics {"StartTime": 1624456479.4409235, "EndTime": 1624456479.4409554, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "Meta": "init_train_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 10010.0, "count": 1, "min": 10010, "max": 10010}, "Total Batches Seen": {"sum": 2002.0, "count": 1, "min": 2002, "max": 2002}, "Max Records Seen Between Resets": {"sum": 10005.0, "count": 1, "min": 10005, "max": 10005}, "Max Batches Seen Between Resets": {"sum": 2001.0, "count": 1, "min": 2001, "max": 2001}, "Reset Count": {"sum": 2.0, "count": 1, "min": 2, "max": 2}, "Number of Records Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}, "Number of Batches Since Last Reset": {"sum": 0.0, "count": 1, "min": 0, "max": 0}}}

    [2021-06-23 13:55:35.039] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 3, "duration": 55598, "num_examples": 2738, "num_bytes": 821400}
    #metrics {"StartTime": 1624456535.0395424, "EndTime": 1624456535.039616, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 0}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.730554318079729, "count": 1, "min": 0.730554318079729, "max": 0.730554318079729}}}

    #metrics {"StartTime": 1624456535.039707, "EndTime": 1624456535.039728, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 1}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.730916357715257, "count": 1, "min": 0.730916357715257, "max": 0.730916357715257}}}

    #metrics {"StartTime": 1624456535.0397832, "EndTime": 1624456535.0398002, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 2}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7293616642170008, "count": 1, "min": 0.7293616642170008, "max": 0.7293616642170008}}}

    #metrics {"StartTime": 1624456535.039853, "EndTime": 1624456535.03987, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 3}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7318380376211129, "count": 1, "min": 0.7318380376211129, "max": 0.7318380376211129}}}

    #metrics {"StartTime": 1624456535.0399241, "EndTime": 1624456535.0399404, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 4}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7597235145709913, "count": 1, "min": 0.7597235145709913, "max": 0.7597235145709913}}}

    #metrics {"StartTime": 1624456535.039991, "EndTime": 1624456535.040006, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 5}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8266414466698384, "count": 1, "min": 0.8266414466698384, "max": 0.8266414466698384}}}

    #metrics {"StartTime": 1624456535.0400596, "EndTime": 1624456535.0400755, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 6}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7630255407317004, "count": 1, "min": 0.7630255407317004, "max": 0.7630255407317004}}}

    #metrics {"StartTime": 1624456535.0401256, "EndTime": 1624456535.0401416, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 7}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8308322956259012, "count": 1, "min": 0.8308322956259012, "max": 0.8308322956259012}}}

    #metrics {"StartTime": 1624456535.040191, "EndTime": 1624456535.0402071, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 8}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7311996138992756, "count": 1, "min": 0.7311996138992756, "max": 0.7311996138992756}}}

    #metrics {"StartTime": 1624456535.0402555, "EndTime": 1624456535.0402713, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 9}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7314104817181803, "count": 1, "min": 0.7314104817181803, "max": 0.7314104817181803}}}

    #metrics {"StartTime": 1624456535.0403218, "EndTime": 1624456535.0403361, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 10}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7299905270272751, "count": 1, "min": 0.7299905270272751, "max": 0.7299905270272751}}}

    #metrics {"StartTime": 1624456535.040385, "EndTime": 1624456535.0404003, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 11}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.730867522062395, "count": 1, "min": 0.730867522062395, "max": 0.730867522062395}}}

    #metrics {"StartTime": 1624456535.0404515, "EndTime": 1624456535.0404673, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 12}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7612727142908523, "count": 1, "min": 0.7612727142908523, "max": 0.7612727142908523}}}

    #metrics {"StartTime": 1624456535.0405197, "EndTime": 1624456535.0405347, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 13}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.84221240419206, "count": 1, "min": 0.84221240419206, "max": 0.84221240419206}}}

    #metrics {"StartTime": 1624456535.0405846, "EndTime": 1624456535.0405998, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 14}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7583267031750425, "count": 1, "min": 0.7583267031750425, "max": 0.7583267031750425}}}

    #metrics {"StartTime": 1624456535.0406494, "EndTime": 1624456535.0406635, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 15}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8300492260907147, "count": 1, "min": 0.8300492260907147, "max": 0.8300492260907147}}}

    #metrics {"StartTime": 1624456535.0407143, "EndTime": 1624456535.0407298, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 16}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.753043388675999, "count": 1, "min": 0.753043388675999, "max": 0.753043388675999}}}

    #metrics {"StartTime": 1624456535.0407786, "EndTime": 1624456535.040794, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 17}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7567276853074811, "count": 1, "min": 0.7567276853074811, "max": 0.7567276853074811}}}

    #metrics {"StartTime": 1624456535.0408435, "EndTime": 1624456535.0408592, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 18}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7530434611550729, "count": 1, "min": 0.7530434611550729, "max": 0.7530434611550729}}}

    #metrics {"StartTime": 1624456535.0409062, "EndTime": 1624456535.0409207, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 19}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7568693716653165, "count": 1, "min": 0.7568693716653165, "max": 0.7568693716653165}}}

    #metrics {"StartTime": 1624456535.0409756, "EndTime": 1624456535.0409913, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 20}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7907412110257096, "count": 1, "min": 0.7907412110257096, "max": 0.7907412110257096}}}

    #metrics {"StartTime": 1624456535.0410302, "EndTime": 1624456535.0410435, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 21}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8462306116687894, "count": 1, "min": 0.8462306116687894, "max": 0.8462306116687894}}}

    #metrics {"StartTime": 1624456535.0410962, "EndTime": 1624456535.041112, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 22}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7885078035561685, "count": 1, "min": 0.7885078035561685, "max": 0.7885078035561685}}}

    #metrics {"StartTime": 1624456535.0411751, "EndTime": 1624456535.0411904, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 23}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8509926724033342, "count": 1, "min": 0.8509926724033342, "max": 0.8509926724033342}}}

    #metrics {"StartTime": 1624456535.0412383, "EndTime": 1624456535.0412533, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 24}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7650802788941681, "count": 1, "min": 0.7650802788941681, "max": 0.7650802788941681}}}

    #metrics {"StartTime": 1624456535.041291, "EndTime": 1624456535.0413046, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 25}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.767797939354177, "count": 1, "min": 0.767797939354177, "max": 0.767797939354177}}}

    #metrics {"StartTime": 1624456535.0413606, "EndTime": 1624456535.0413773, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 26}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7650129593489725, "count": 1, "min": 0.7650129593489725, "max": 0.7650129593489725}}}

    #metrics {"StartTime": 1624456535.041434, "EndTime": 1624456535.0414503, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 27}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7675427508188739, "count": 1, "min": 0.7675427508188739, "max": 0.7675427508188739}}}

    #metrics {"StartTime": 1624456535.0415027, "EndTime": 1624456535.0415187, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 28}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7907709040155891, "count": 1, "min": 0.7907709040155891, "max": 0.7907709040155891}}}

    #metrics {"StartTime": 1624456535.0415714, "EndTime": 1624456535.0415866, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 29}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8407626407698526, "count": 1, "min": 0.8407626407698526, "max": 0.8407626407698526}}}

    #metrics {"StartTime": 1624456535.041629, "EndTime": 1624456535.0416443, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 30}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7921950258679247, "count": 1, "min": 0.7921950258679247, "max": 0.7921950258679247}}}

    #metrics {"StartTime": 1624456535.0416908, "EndTime": 1624456535.0417051, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "model": 31}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8389875671783444, "count": 1, "min": 0.8389875671783444, "max": 0.8389875671783444}}}

    [06/23/2021 13:55:35 INFO 139646453495616] #quality_metric: host=algo-1, epoch=0, train absolute_loss_objective <loss>=0.730554318079729
    [06/23/2021 13:55:35 INFO 139646453495616] #early_stopping_criteria_metric: host=algo-1, epoch=0, criteria=absolute_loss_objective, value=0.7293616642170008
    [06/23/2021 13:55:35 INFO 139646453495616] Epoch 0: Loss improved. Updating best model
    [06/23/2021 13:55:35 INFO 139646453495616] Saving model for epoch: 0
    [06/23/2021 13:55:35 INFO 139646453495616] Saved checkpoint to "/tmp/tmpe9d88nxt/mx-mod-0000.params"
    [06/23/2021 13:55:35 INFO 139646453495616] #progress_metric: host=algo-1, completed 20.0 % of epochs
    #metrics {"StartTime": 1624456479.4412065, "EndTime": 1624456535.0566134, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 0, "Meta": "training_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 23700.0, "count": 1, "min": 23700, "max": 23700}, "Total Batches Seen": {"sum": 4740.0, "count": 1, "min": 4740, "max": 4740}, "Max Records Seen Between Resets": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Max Batches Seen Between Resets": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}, "Reset Count": {"sum": 3.0, "count": 1, "min": 3, "max": 3}, "Number of Records Since Last Reset": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Number of Batches Since Last Reset": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}}}

    [06/23/2021 13:55:35 INFO 139646453495616] #throughput_metric: host=algo-1, train throughput=246.1542201303113 records/second
    [2021-06-23 13:56:30.897] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 55840, "num_examples": 2738, "num_bytes": 821400}
    #metrics {"StartTime": 1624456590.8977492, "EndTime": 1624456590.8978224, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7213270940207669, "count": 1, "min": 0.7213270940207669, "max": 0.7213270940207669}}}

    #metrics {"StartTime": 1624456590.8979158, "EndTime": 1624456590.8979356, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7221783018530156, "count": 1, "min": 0.7221783018530156, "max": 0.7221783018530156}}}

    #metrics {"StartTime": 1624456590.897991, "EndTime": 1624456590.898008, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 2}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7211717115126012, "count": 1, "min": 0.7211717115126012, "max": 0.7211717115126012}}}

    #metrics {"StartTime": 1624456590.8980703, "EndTime": 1624456590.898087, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 3}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7220391356788298, "count": 1, "min": 0.7220391356788298, "max": 0.7220391356788298}}}

    #metrics {"StartTime": 1624456590.8981478, "EndTime": 1624456590.898164, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 4}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7198562159992814, "count": 1, "min": 0.7198562159992814, "max": 0.7198562159992814}}}

    #metrics {"StartTime": 1624456590.8982146, "EndTime": 1624456590.8982294, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 5}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7975306172454505, "count": 1, "min": 0.7975306172454505, "max": 0.7975306172454505}}}

    #metrics {"StartTime": 1624456590.8982785, "EndTime": 1624456590.8982933, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 6}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7199292913852076, "count": 1, "min": 0.7199292913852076, "max": 0.7199292913852076}}}

    #metrics {"StartTime": 1624456590.8983421, "EndTime": 1624456590.8983574, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 7}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8045426907854589, "count": 1, "min": 0.8045426907854589, "max": 0.8045426907854589}}}

    #metrics {"StartTime": 1624456590.898408, "EndTime": 1624456590.8984244, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 8}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7219043255592454, "count": 1, "min": 0.7219043255592454, "max": 0.7219043255592454}}}

    #metrics {"StartTime": 1624456590.8984842, "EndTime": 1624456590.898501, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 9}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7226418907142539, "count": 1, "min": 0.7226418907142539, "max": 0.7226418907142539}}}

    #metrics {"StartTime": 1624456590.8985605, "EndTime": 1624456590.8985765, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 10}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.721451066539965, "count": 1, "min": 0.721451066539965, "max": 0.721451066539965}}}

    #metrics {"StartTime": 1624456590.8986337, "EndTime": 1624456590.8986497, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 11}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7226802080499595, "count": 1, "min": 0.7226802080499595, "max": 0.7226802080499595}}}

    #metrics {"StartTime": 1624456590.8986976, "EndTime": 1624456590.898712, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 12}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7201854246171223, "count": 1, "min": 0.7201854246171223, "max": 0.7201854246171223}}}

    #metrics {"StartTime": 1624456590.8987582, "EndTime": 1624456590.8987713, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 13}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8063275096275407, "count": 1, "min": 0.8063275096275407, "max": 0.8063275096275407}}}

    #metrics {"StartTime": 1624456590.898816, "EndTime": 1624456590.898831, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 14}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7203309837312398, "count": 1, "min": 0.7203309837312398, "max": 0.7203309837312398}}}

    #metrics {"StartTime": 1624456590.8988779, "EndTime": 1624456590.8988924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 15}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8000542680981213, "count": 1, "min": 0.8000542680981213, "max": 0.8000542680981213}}}

    #metrics {"StartTime": 1624456590.898937, "EndTime": 1624456590.898951, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 16}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7504015528607142, "count": 1, "min": 0.7504015528607142, "max": 0.7504015528607142}}}

    #metrics {"StartTime": 1624456590.8990252, "EndTime": 1624456590.899041, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 17}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7550290710434938, "count": 1, "min": 0.7550290710434938, "max": 0.7550290710434938}}}

    #metrics {"StartTime": 1624456590.8990893, "EndTime": 1624456590.8991032, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 18}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7504009553252388, "count": 1, "min": 0.7504009553252388, "max": 0.7504009553252388}}}

    #metrics {"StartTime": 1624456590.8991575, "EndTime": 1624456590.8991723, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 19}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7548886100487434, "count": 1, "min": 0.7548886100487434, "max": 0.7548886100487434}}}

    #metrics {"StartTime": 1624456590.8992217, "EndTime": 1624456590.8992367, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 20}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7532113005289464, "count": 1, "min": 0.7532113005289464, "max": 0.7532113005289464}}}

    #metrics {"StartTime": 1624456590.8992825, "EndTime": 1624456590.8992968, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 21}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8214720037207279, "count": 1, "min": 0.8214720037207279, "max": 0.8214720037207279}}}

    #metrics {"StartTime": 1624456590.8993495, "EndTime": 1624456590.899365, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 22}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7531819211905503, "count": 1, "min": 0.7531819211905503, "max": 0.7531819211905503}}}

    #metrics {"StartTime": 1624456590.8994126, "EndTime": 1624456590.8994277, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 23}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8292674729474014, "count": 1, "min": 0.8292674729474014, "max": 0.8292674729474014}}}

    #metrics {"StartTime": 1624456590.8994732, "EndTime": 1624456590.8994875, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 24}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635649959434986, "count": 1, "min": 0.7635649959434986, "max": 0.7635649959434986}}}

    #metrics {"StartTime": 1624456590.899533, "EndTime": 1624456590.8995478, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 25}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7665098110326805, "count": 1, "min": 0.7665098110326805, "max": 0.7665098110326805}}}

    #metrics {"StartTime": 1624456590.899588, "EndTime": 1624456590.8996022, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 26}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635877762422499, "count": 1, "min": 0.7635877762422499, "max": 0.7635877762422499}}}

    #metrics {"StartTime": 1624456590.899645, "EndTime": 1624456590.8996592, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 27}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7665649855894446, "count": 1, "min": 0.7665649855894446, "max": 0.7665649855894446}}}

    #metrics {"StartTime": 1624456590.8997018, "EndTime": 1624456590.8997161, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 28}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7654449481481701, "count": 1, "min": 0.7654449481481701, "max": 0.7654449481481701}}}

    #metrics {"StartTime": 1624456590.8997529, "EndTime": 1624456590.899767, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 29}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.818387601014643, "count": 1, "min": 0.818387601014643, "max": 0.818387601014643}}}

    #metrics {"StartTime": 1624456590.899813, "EndTime": 1624456590.8998284, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 30}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7654356881769319, "count": 1, "min": 0.7654356881769319, "max": 0.7654356881769319}}}

    #metrics {"StartTime": 1624456590.8998754, "EndTime": 1624456590.89989, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 31}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8159986126131172, "count": 1, "min": 0.8159986126131172, "max": 0.8159986126131172}}}

    [06/23/2021 13:56:30 INFO 139646453495616] #quality_metric: host=algo-1, epoch=1, train absolute_loss_objective <loss>=0.7213270940207669
    [06/23/2021 13:56:30 INFO 139646453495616] #early_stopping_criteria_metric: host=algo-1, epoch=1, criteria=absolute_loss_objective, value=0.7198562159992814
    [06/23/2021 13:56:30 INFO 139646453495616] Epoch 1: Loss improved. Updating best model
    [06/23/2021 13:56:30 INFO 139646453495616] Saving model for epoch: 1
    [06/23/2021 13:56:30 INFO 139646453495616] Saved checkpoint to "/tmp/tmpmbod02s1/mx-mod-0000.params"
    [06/23/2021 13:56:30 INFO 139646453495616] #progress_metric: host=algo-1, completed 40.0 % of epochs
    #metrics {"StartTime": 1624456535.0569103, "EndTime": 1624456590.908974, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "Meta": "training_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 37390.0, "count": 1, "min": 37390, "max": 37390}, "Total Batches Seen": {"sum": 7478.0, "count": 1, "min": 7478, "max": 7478}, "Max Records Seen Between Resets": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Max Batches Seen Between Resets": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}, "Reset Count": {"sum": 4.0, "count": 1, "min": 4, "max": 4}, "Number of Records Since Last Reset": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Number of Batches Since Last Reset": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}}}

    [06/23/2021 13:56:30 INFO 139646453495616] #throughput_metric: host=algo-1, train throughput=245.1112166324981 records/second
    [2021-06-23 13:57:26.299] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 55390, "num_examples": 2738, "num_bytes": 821400}
    #metrics {"StartTime": 1624456646.2994998, "EndTime": 1624456646.2995713, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.72080655755833, "count": 1, "min": 0.72080655755833, "max": 0.72080655755833}}}

    #metrics {"StartTime": 1624456646.2996507, "EndTime": 1624456646.2996662, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7203933912920552, "count": 1, "min": 0.7203933912920552, "max": 0.7203933912920552}}}

    #metrics {"StartTime": 1624456646.2997112, "EndTime": 1624456646.2997222, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 2}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7205431216080055, "count": 1, "min": 0.7205431216080055, "max": 0.7205431216080055}}}

    #metrics {"StartTime": 1624456646.2997596, "EndTime": 1624456646.2997692, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 3}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7204069429152721, "count": 1, "min": 0.7204069429152721, "max": 0.7204069429152721}}}

    #metrics {"StartTime": 1624456646.299802, "EndTime": 1624456646.2998114, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 4}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7159620899345163, "count": 1, "min": 0.7159620899345163, "max": 0.7159620899345163}}}

    #metrics {"StartTime": 1624456646.2998433, "EndTime": 1624456646.2998521, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 5}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7788643907943374, "count": 1, "min": 0.7788643907943374, "max": 0.7788643907943374}}}

    #metrics {"StartTime": 1624456646.2998836, "EndTime": 1624456646.299892, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 6}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7160098188806399, "count": 1, "min": 0.7160098188806399, "max": 0.7160098188806399}}}

    #metrics {"StartTime": 1624456646.2999265, "EndTime": 1624456646.2999356, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 7}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7804630302529513, "count": 1, "min": 0.7804630302529513, "max": 0.7804630302529513}}}

    #metrics {"StartTime": 1624456646.2999735, "EndTime": 1624456646.2999828, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 8}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7214041368686169, "count": 1, "min": 0.7214041368686169, "max": 0.7214041368686169}}}

    #metrics {"StartTime": 1624456646.3000174, "EndTime": 1624456646.3000264, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 9}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.72106727763107, "count": 1, "min": 0.72106727763107, "max": 0.72106727763107}}}

    #metrics {"StartTime": 1624456646.300058, "EndTime": 1624456646.3000665, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 10}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7208715244180709, "count": 1, "min": 0.7208715244180709, "max": 0.7208715244180709}}}

    #metrics {"StartTime": 1624456646.3000977, "EndTime": 1624456646.3001063, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 11}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7210596068311208, "count": 1, "min": 0.7210596068311208, "max": 0.7210596068311208}}}

    #metrics {"StartTime": 1624456646.3001392, "EndTime": 1624456646.3001482, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 12}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.716323120651496, "count": 1, "min": 0.716323120651496, "max": 0.716323120651496}}}

    #metrics {"StartTime": 1624456646.3001795, "EndTime": 1624456646.3001878, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 13}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7843453623283685, "count": 1, "min": 0.7843453623283685, "max": 0.7843453623283685}}}

    #metrics {"StartTime": 1624456646.300219, "EndTime": 1624456646.3002276, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 14}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7163474639721209, "count": 1, "min": 0.7163474639721209, "max": 0.7163474639721209}}}

    #metrics {"StartTime": 1624456646.3002598, "EndTime": 1624456646.300269, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 15}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7778972536347224, "count": 1, "min": 0.7778972536347224, "max": 0.7778972536347224}}}

    #metrics {"StartTime": 1624456646.3003001, "EndTime": 1624456646.300309, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 16}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7501474957730841, "count": 1, "min": 0.7501474957730841, "max": 0.7501474957730841}}}

    #metrics {"StartTime": 1624456646.300346, "EndTime": 1624456646.3003554, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 17}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7539053692053146, "count": 1, "min": 0.7539053692053146, "max": 0.7539053692053146}}}

    #metrics {"StartTime": 1624456646.3003936, "EndTime": 1624456646.300403, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 18}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7502575501288002, "count": 1, "min": 0.7502575501288002, "max": 0.7502575501288002}}}

    #metrics {"StartTime": 1624456646.300441, "EndTime": 1624456646.3004506, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 19}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7537081745482431, "count": 1, "min": 0.7537081745482431, "max": 0.7537081745482431}}}

    #metrics {"StartTime": 1624456646.300485, "EndTime": 1624456646.3004935, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 20}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7503967388147329, "count": 1, "min": 0.7503967388147329, "max": 0.7503967388147329}}}

    #metrics {"StartTime": 1624456646.3005435, "EndTime": 1624456646.3005538, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 21}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8018828172331922, "count": 1, "min": 0.8018828172331922, "max": 0.8018828172331922}}}

    #metrics {"StartTime": 1624456646.3005934, "EndTime": 1624456646.3006032, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 22}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7504594249260347, "count": 1, "min": 0.7504594249260347, "max": 0.7504594249260347}}}

    #metrics {"StartTime": 1624456646.3006377, "EndTime": 1624456646.3006465, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 23}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8037709734250892, "count": 1, "min": 0.8037709734250892, "max": 0.8037709734250892}}}

    #metrics {"StartTime": 1624456646.3006837, "EndTime": 1624456646.3006935, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 24}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7634601257115581, "count": 1, "min": 0.7634601257115581, "max": 0.7634601257115581}}}

    #metrics {"StartTime": 1624456646.3007262, "EndTime": 1624456646.3007352, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 25}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7657388490120628, "count": 1, "min": 0.7657388490120628, "max": 0.7657388490120628}}}

    #metrics {"StartTime": 1624456646.3007722, "EndTime": 1624456646.3007815, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 26}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635246429328417, "count": 1, "min": 0.7635246429328417, "max": 0.7635246429328417}}}

    #metrics {"StartTime": 1624456646.3008144, "EndTime": 1624456646.300823, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 27}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7658425805863017, "count": 1, "min": 0.7658425805863017, "max": 0.7658425805863017}}}

    #metrics {"StartTime": 1624456646.3008597, "EndTime": 1624456646.3008687, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 28}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7636084122323398, "count": 1, "min": 0.7636084122323398, "max": 0.7636084122323398}}}

    #metrics {"StartTime": 1624456646.300903, "EndTime": 1624456646.300912, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 29}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8062849323034634, "count": 1, "min": 0.8062849323034634, "max": 0.8062849323034634}}}

    #metrics {"StartTime": 1624456646.3009484, "EndTime": 1624456646.300958, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 30}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7636149114940705, "count": 1, "min": 0.7636149114940705, "max": 0.7636149114940705}}}

    #metrics {"StartTime": 1624456646.300999, "EndTime": 1624456646.3010118, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 31}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.8076051508143838, "count": 1, "min": 0.8076051508143838, "max": 0.8076051508143838}}}

    [06/23/2021 13:57:26 INFO 139646453495616] #quality_metric: host=algo-1, epoch=2, train absolute_loss_objective <loss>=0.72080655755833
    [06/23/2021 13:57:26 INFO 139646453495616] #early_stopping_criteria_metric: host=algo-1, epoch=2, criteria=absolute_loss_objective, value=0.7159620899345163
    [06/23/2021 13:57:26 INFO 139646453495616] Epoch 2: Loss improved. Updating best model
    [06/23/2021 13:57:26 INFO 139646453495616] Saving model for epoch: 2
    [06/23/2021 13:57:26 INFO 139646453495616] Saved checkpoint to "/tmp/tmpzia72ksx/mx-mod-0000.params"
    [06/23/2021 13:57:26 INFO 139646453495616] #progress_metric: host=algo-1, completed 60.0 % of epochs
    #metrics {"StartTime": 1624456590.9092472, "EndTime": 1624456646.3102028, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "Meta": "training_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 51080.0, "count": 1, "min": 51080, "max": 51080}, "Total Batches Seen": {"sum": 10216.0, "count": 1, "min": 10216, "max": 10216}, "Max Records Seen Between Resets": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Max Batches Seen Between Resets": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}, "Reset Count": {"sum": 5.0, "count": 1, "min": 5, "max": 5}, "Number of Records Since Last Reset": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Number of Batches Since Last Reset": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}}}

    [06/23/2021 13:57:26 INFO 139646453495616] #throughput_metric: host=algo-1, train throughput=247.10703809551623 records/second
    [2021-06-23 13:58:22.022] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 55712, "num_examples": 2738, "num_bytes": 821400}
    #metrics {"StartTime": 1624456702.0230129, "EndTime": 1624456702.0230849, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7207475037362299, "count": 1, "min": 0.7207475037362299, "max": 0.7207475037362299}}}

    #metrics {"StartTime": 1624456702.0231595, "EndTime": 1624456702.0231743, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7193818130660353, "count": 1, "min": 0.7193818130660353, "max": 0.7193818130660353}}}

    #metrics {"StartTime": 1624456702.0232131, "EndTime": 1624456702.0232234, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 2}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7201795843763887, "count": 1, "min": 0.7201795843763887, "max": 0.7201795843763887}}}

    #metrics {"StartTime": 1624456702.0232573, "EndTime": 1624456702.0232666, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 3}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7193539180966343, "count": 1, "min": 0.7193539180966343, "max": 0.7193539180966343}}}

    #metrics {"StartTime": 1624456702.0232992, "EndTime": 1624456702.0233085, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 4}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7156125876396742, "count": 1, "min": 0.7156125876396742, "max": 0.7156125876396742}}}

    #metrics {"StartTime": 1624456702.0233421, "EndTime": 1624456702.023351, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 5}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7620669244544361, "count": 1, "min": 0.7620669244544361, "max": 0.7620669244544361}}}

    #metrics {"StartTime": 1624456702.023385, "EndTime": 1624456702.0233943, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 6}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.715816328658772, "count": 1, "min": 0.715816328658772, "max": 0.715816328658772}}}

    #metrics {"StartTime": 1624456702.0234337, "EndTime": 1624456702.0234435, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 7}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.762786272666679, "count": 1, "min": 0.762786272666679, "max": 0.762786272666679}}}

    #metrics {"StartTime": 1624456702.023477, "EndTime": 1624456702.023486, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 8}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.721345216401569, "count": 1, "min": 0.721345216401569, "max": 0.721345216401569}}}

    #metrics {"StartTime": 1624456702.0235171, "EndTime": 1624456702.023526, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 9}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7197571934662743, "count": 1, "min": 0.7197571934662743, "max": 0.7197571934662743}}}

    #metrics {"StartTime": 1624456702.023563, "EndTime": 1624456702.0235727, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 10}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7205236863445591, "count": 1, "min": 0.7205236863445591, "max": 0.7205236863445591}}}

    #metrics {"StartTime": 1624456702.0236113, "EndTime": 1624456702.0236208, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 11}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7196777728589632, "count": 1, "min": 0.7196777728589632, "max": 0.7196777728589632}}}

    #metrics {"StartTime": 1624456702.0236595, "EndTime": 1624456702.023669, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 12}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7159739104215902, "count": 1, "min": 0.7159739104215902, "max": 0.7159739104215902}}}

    #metrics {"StartTime": 1624456702.0237076, "EndTime": 1624456702.0237172, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 13}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7642411828345849, "count": 1, "min": 0.7642411828345849, "max": 0.7642411828345849}}}

    #metrics {"StartTime": 1624456702.0237556, "EndTime": 1624456702.0237648, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 14}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7161481828520645, "count": 1, "min": 0.7161481828520645, "max": 0.7161481828520645}}}

    #metrics {"StartTime": 1624456702.0238032, "EndTime": 1624456702.0238125, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 15}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7637419329933044, "count": 1, "min": 0.7637419329933044, "max": 0.7637419329933044}}}

    #metrics {"StartTime": 1624456702.0238507, "EndTime": 1624456702.0238605, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 16}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7501297894318318, "count": 1, "min": 0.7501297894318318, "max": 0.7501297894318318}}}

    #metrics {"StartTime": 1624456702.0238986, "EndTime": 1624456702.0239084, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 17}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7528897814980556, "count": 1, "min": 0.7528897814980556, "max": 0.7528897814980556}}}

    #metrics {"StartTime": 1624456702.023941, "EndTime": 1624456702.0239499, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 18}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7502276699590021, "count": 1, "min": 0.7502276699590021, "max": 0.7502276699590021}}}

    #metrics {"StartTime": 1624456702.023981, "EndTime": 1624456702.0239897, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 19}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.75291598513668, "count": 1, "min": 0.75291598513668, "max": 0.75291598513668}}}

    #metrics {"StartTime": 1624456702.024022, "EndTime": 1624456702.0240312, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 20}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7501068040804761, "count": 1, "min": 0.7501068040804761, "max": 0.7501068040804761}}}

    #metrics {"StartTime": 1624456702.0240629, "EndTime": 1624456702.0240717, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 21}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7951125231426593, "count": 1, "min": 0.7951125231426593, "max": 0.7951125231426593}}}

    #metrics {"StartTime": 1624456702.0241032, "EndTime": 1624456702.0241115, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 22}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7502548156601395, "count": 1, "min": 0.7502548156601395, "max": 0.7502548156601395}}}

    #metrics {"StartTime": 1624456702.0241432, "EndTime": 1624456702.0241518, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 23}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7916524350882621, "count": 1, "min": 0.7916524350882621, "max": 0.7916524350882621}}}

    #metrics {"StartTime": 1624456702.024183, "EndTime": 1624456702.0241919, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 24}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7634452330404341, "count": 1, "min": 0.7634452330404341, "max": 0.7634452330404341}}}

    #metrics {"StartTime": 1624456702.024223, "EndTime": 1624456702.0242317, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 25}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7652878690540921, "count": 1, "min": 0.7652878690540921, "max": 0.7652878690540921}}}

    #metrics {"StartTime": 1624456702.0242631, "EndTime": 1624456702.024272, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 26}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635248489672464, "count": 1, "min": 0.7635248489672464, "max": 0.7635248489672464}}}

    #metrics {"StartTime": 1624456702.0243099, "EndTime": 1624456702.0243192, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 27}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7652424768908691, "count": 1, "min": 0.7652424768908691, "max": 0.7652424768908691}}}

    #metrics {"StartTime": 1624456702.0243535, "EndTime": 1624456702.0243628, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 28}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7634689580009661, "count": 1, "min": 0.7634689580009661, "max": 0.7634689580009661}}}

    #metrics {"StartTime": 1624456702.0243948, "EndTime": 1624456702.0244036, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 29}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.795677721968439, "count": 1, "min": 0.795677721968439, "max": 0.795677721968439}}}

    #metrics {"StartTime": 1624456702.0244343, "EndTime": 1624456702.0244431, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 30}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635246180286366, "count": 1, "min": 0.7635246180286366, "max": 0.7635246180286366}}}

    #metrics {"StartTime": 1624456702.0244744, "EndTime": 1624456702.024483, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 31}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.796668367955352, "count": 1, "min": 0.796668367955352, "max": 0.796668367955352}}}

    [06/23/2021 13:58:22 INFO 139646453495616] #quality_metric: host=algo-1, epoch=3, train absolute_loss_objective <loss>=0.7207475037362299
    [06/23/2021 13:58:22 INFO 139646453495616] #early_stopping_criteria_metric: host=algo-1, epoch=3, criteria=absolute_loss_objective, value=0.7156125876396742
    [06/23/2021 13:58:22 INFO 139646453495616] Saving model for epoch: 3
    [06/23/2021 13:58:22 INFO 139646453495616] Saved checkpoint to "/tmp/tmp_0m8n1fh/mx-mod-0000.params"
    [06/23/2021 13:58:22 INFO 139646453495616] #progress_metric: host=algo-1, completed 80.0 % of epochs
    #metrics {"StartTime": 1624456646.3104901, "EndTime": 1624456702.0326757, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "Meta": "training_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 64770.0, "count": 1, "min": 64770, "max": 64770}, "Total Batches Seen": {"sum": 12954.0, "count": 1, "min": 12954, "max": 12954}, "Max Records Seen Between Resets": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Max Batches Seen Between Resets": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}, "Reset Count": {"sum": 6.0, "count": 1, "min": 6, "max": 6}, "Number of Records Since Last Reset": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Number of Batches Since Last Reset": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}}}

    [06/23/2021 13:58:22 INFO 139646453495616] #throughput_metric: host=algo-1, train throughput=245.6825911683808 records/second
    [2021-06-23 13:59:16.272] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 54239, "num_examples": 2738, "num_bytes": 821400}
    #metrics {"StartTime": 1624456756.2723885, "EndTime": 1624456756.2724562, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7206968672872895, "count": 1, "min": 0.7206968672872895, "max": 0.7206968672872895}}}

    #metrics {"StartTime": 1624456756.272547, "EndTime": 1624456756.2725673, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7184065194614095, "count": 1, "min": 0.7184065194614095, "max": 0.7184065194614095}}}

    #metrics {"StartTime": 1624456756.2726126, "EndTime": 1624456756.2726278, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 2}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7198496622138862, "count": 1, "min": 0.7198496622138862, "max": 0.7198496622138862}}}

    #metrics {"StartTime": 1624456756.2726784, "EndTime": 1624456756.2726934, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 3}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7184252402179863, "count": 1, "min": 0.7184252402179863, "max": 0.7184252402179863}}}

    #metrics {"StartTime": 1624456756.2727377, "EndTime": 1624456756.2727525, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 4}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7155968107255903, "count": 1, "min": 0.7155968107255903, "max": 0.7155968107255903}}}

    #metrics {"StartTime": 1624456756.2728016, "EndTime": 1624456756.2728128, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 5}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7549695908762052, "count": 1, "min": 0.7549695908762052, "max": 0.7549695908762052}}}

    #metrics {"StartTime": 1624456756.2728505, "EndTime": 1624456756.2728589, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 6}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7158069722011544, "count": 1, "min": 0.7158069722011544, "max": 0.7158069722011544}}}

    #metrics {"StartTime": 1624456756.272886, "EndTime": 1624456756.272898, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 7}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7540595930932298, "count": 1, "min": 0.7540595930932298, "max": 0.7540595930932298}}}

    #metrics {"StartTime": 1624456756.272943, "EndTime": 1624456756.2729566, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 8}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7212961840838564, "count": 1, "min": 0.7212961840838564, "max": 0.7212961840838564}}}

    #metrics {"StartTime": 1624456756.2729847, "EndTime": 1624456756.2729924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 9}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7189581230794365, "count": 1, "min": 0.7189581230794365, "max": 0.7189581230794365}}}

    #metrics {"StartTime": 1624456756.2730148, "EndTime": 1624456756.2730258, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 10}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7202293191407177, "count": 1, "min": 0.7202293191407177, "max": 0.7202293191407177}}}

    #metrics {"StartTime": 1624456756.2730772, "EndTime": 1624456756.273092, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 11}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7188834479684112, "count": 1, "min": 0.7188834479684112, "max": 0.7188834479684112}}}

    #metrics {"StartTime": 1624456756.2731407, "EndTime": 1624456756.2731545, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 12}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7159591787700849, "count": 1, "min": 0.7159591787700849, "max": 0.7159591787700849}}}

    #metrics {"StartTime": 1624456756.2732, "EndTime": 1624456756.2732446, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 13}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.754834063417551, "count": 1, "min": 0.754834063417551, "max": 0.754834063417551}}}

    #metrics {"StartTime": 1624456756.2732952, "EndTime": 1624456756.273311, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 14}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7161469223945136, "count": 1, "min": 0.7161469223945136, "max": 0.7161469223945136}}}

    #metrics {"StartTime": 1624456756.273349, "EndTime": 1624456756.2733622, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 15}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.754531049308244, "count": 1, "min": 0.754531049308244, "max": 0.754531049308244}}}

    #metrics {"StartTime": 1624456756.2734048, "EndTime": 1624456756.273419, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 16}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7501195484569597, "count": 1, "min": 0.7501195484569597, "max": 0.7501195484569597}}}

    #metrics {"StartTime": 1624456756.2734625, "EndTime": 1624456756.2734745, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 17}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7521299566815072, "count": 1, "min": 0.7521299566815072, "max": 0.7521299566815072}}}

    #metrics {"StartTime": 1624456756.2735183, "EndTime": 1624456756.2735307, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 18}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7502181946360517, "count": 1, "min": 0.7502181946360517, "max": 0.7502181946360517}}}

    #metrics {"StartTime": 1624456756.273575, "EndTime": 1624456756.2735887, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 19}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7521096946419896, "count": 1, "min": 0.7521096946419896, "max": 0.7521096946419896}}}

    #metrics {"StartTime": 1624456756.2736187, "EndTime": 1624456756.2736266, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 20}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7500856872128256, "count": 1, "min": 0.7500856872128256, "max": 0.7500856872128256}}}

    #metrics {"StartTime": 1624456756.2736504, "EndTime": 1624456756.273662, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 21}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7851035638522544, "count": 1, "min": 0.7851035638522544, "max": 0.7851035638522544}}}

    #metrics {"StartTime": 1624456756.273706, "EndTime": 1624456756.2737188, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 22}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7502258369224275, "count": 1, "min": 0.7502258369224275, "max": 0.7502258369224275}}}

    #metrics {"StartTime": 1624456756.2737458, "EndTime": 1624456756.2737532, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 23}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7831681265763073, "count": 1, "min": 0.7831681265763073, "max": 0.7831681265763073}}}

    #metrics {"StartTime": 1624456756.2737753, "EndTime": 1624456756.2737823, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 24}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7634430727747952, "count": 1, "min": 0.7634430727747952, "max": 0.7634430727747952}}}

    #metrics {"StartTime": 1624456756.2738066, "EndTime": 1624456756.273818, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 25}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7649273845163725, "count": 1, "min": 0.7649273845163725, "max": 0.7649273845163725}}}

    #metrics {"StartTime": 1624456756.2738621, "EndTime": 1624456756.2738757, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 26}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635251177106137, "count": 1, "min": 0.7635251177106137, "max": 0.7635251177106137}}}

    #metrics {"StartTime": 1624456756.2739108, "EndTime": 1624456756.273924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 27}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7649125738114314, "count": 1, "min": 0.7649125738114314, "max": 0.7649125738114314}}}

    #metrics {"StartTime": 1624456756.2739704, "EndTime": 1624456756.2739835, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 28}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7634451914303837, "count": 1, "min": 0.7634451914303837, "max": 0.7634451914303837}}}

    #metrics {"StartTime": 1624456756.2740397, "EndTime": 1624456756.27405, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 29}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7889505361339487, "count": 1, "min": 0.7889505361339487, "max": 0.7889505361339487}}}

    #metrics {"StartTime": 1624456756.2740824, "EndTime": 1624456756.2740943, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 30}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7635248469557528, "count": 1, "min": 0.7635248469557528, "max": 0.7635248469557528}}}

    #metrics {"StartTime": 1624456756.2741446, "EndTime": 1624456756.2741578, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 31}, "Metrics": {"train_absolute_loss_objective": {"sum": 0.7918228639726486, "count": 1, "min": 0.7918228639726486, "max": 0.7918228639726486}}}

    [06/23/2021 13:59:16 INFO 139646453495616] #quality_metric: host=algo-1, epoch=4, train absolute_loss_objective <loss>=0.7206968672872895
    [06/23/2021 13:59:16 INFO 139646453495616] #early_stopping_criteria_metric: host=algo-1, epoch=4, criteria=absolute_loss_objective, value=0.7155968107255903
    [06/23/2021 13:59:16 INFO 139646453495616] Saving model for epoch: 4
    [06/23/2021 13:59:16 INFO 139646453495616] Saved checkpoint to "/tmp/tmpv0c1wo_h/mx-mod-0000.params"
    [06/23/2021 13:59:16 INFO 139646453495616] #progress_metric: host=algo-1, completed 100.0 % of epochs
    #metrics {"StartTime": 1624456702.0329416, "EndTime": 1624456756.2824292, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "Meta": "training_data_iter"}, "Metrics": {"Total Records Seen": {"sum": 78460.0, "count": 1, "min": 78460, "max": 78460}, "Total Batches Seen": {"sum": 15692.0, "count": 1, "min": 15692, "max": 15692}, "Max Records Seen Between Resets": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Max Batches Seen Between Resets": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}, "Reset Count": {"sum": 7.0, "count": 1, "min": 7, "max": 7}, "Number of Records Since Last Reset": {"sum": 13690.0, "count": 1, "min": 13690, "max": 13690}, "Number of Batches Since Last Reset": {"sum": 2738.0, "count": 1, "min": 2738, "max": 2738}}}

    [06/23/2021 13:59:16 INFO 139646453495616] #throughput_metric: host=algo-1, train throughput=252.3520525674001 records/second
    [06/23/2021 13:59:16 WARNING 139646453495616] wait_for_all_workers will not sync workers since the kv store is not running distributed
    [06/23/2021 13:59:16 WARNING 139646453495616] wait_for_all_workers will not sync workers since the kv store is not running distributed
    [2021-06-23 13:59:16.283] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 0, "num_examples": 1, "num_bytes": 300}
    [2021-06-23 13:59:19.636] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 15, "duration": 3350, "num_examples": 2738, "num_bytes": 821400}
    [06/23/2021 13:59:19 INFO 139646453495616] #train_score (algo-1) : ('absolute_loss_objective', 82521.37994201972)
    [06/23/2021 13:59:19 INFO 139646453495616] #train_score (algo-1) : ('mse', 11785870430.209496)
    [06/23/2021 13:59:19 INFO 139646453495616] #train_score (algo-1) : ('absolute_loss', 82521.37994201972)
    [06/23/2021 13:59:19 INFO 139646453495616] #quality_metric: host=algo-1, train absolute_loss_objective <loss>=82521.37994201972
    [06/23/2021 13:59:19 INFO 139646453495616] #quality_metric: host=algo-1, train mse <loss>=11785870430.209496
    [06/23/2021 13:59:19 INFO 139646453495616] #quality_metric: host=algo-1, train absolute_loss <loss>=82521.37994201972
    [06/23/2021 13:59:19 INFO 139646453495616] Best model found for hyperparameters: {"optimizer": "adam", "learning_rate": 0.1, "wd": 0.0001, "l1": 0.0, "lr_scheduler_step": 10, "lr_scheduler_factor": 0.99, "lr_scheduler_minimum_lr": 1e-05}
    [06/23/2021 13:59:19 INFO 139646453495616] Saved checkpoint to "/tmp/tmpepcl3g16/mx-mod-0000.params"
    [06/23/2021 13:59:19 INFO 139646453495616] Test data is not provided.
    #metrics {"StartTime": 1624456466.4713798, "EndTime": 1624456759.6423705, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training"}, "Metrics": {"initialize.time": {"sum": 12967.874526977539, "count": 1, "min": 12967.874526977539, "max": 12967.874526977539}, "epochs": {"sum": 5.0, "count": 1, "min": 5, "max": 5}, "check_early_stopping.time": {"sum": 3.597736358642578, "count": 5, "min": 0.17380714416503906, "max": 1.1141300201416016}, "update.time": {"sum": 276822.09157943726, "count": 5, "min": 54246.933460235596, "max": 55849.44987297058}, "finalize.time": {"sum": 3354.7611236572266, "count": 1, "min": 3354.7611236572266, "max": 3354.7611236572266}, "setuptime": {"sum": 24.288654327392578, "count": 1, "min": 24.288654327392578, "max": 24.288654327392578}, "totaltime": {"sum": 293432.03258514404, "count": 1, "min": 293432.03258514404, "max": 293432.03258514404}}}


    2021-06-23 13:59:33 Uploading - Uploading generated training model
    2021-06-23 13:59:33 Completed - Training job completed
    Training seconds: 359
    Billable seconds: 359

In \[103\]:

    #Converting the test sets to Numpy array
    X_test = np.array(X_test).astype('float32')
    y_test = np.array(y_test).astype('float32')
    y_test = np.reshape(y_test,-1)

    print(X_test)
    print(y_test)

    [[  37. 2176.  418. 1301.]
     [  20. 3158.  684. 2396.]
     [  11. 2393.  726. 1905.]
     ...
     [  32. 1375.  448. 1698.]
     [  36. 1293.  312. 1128.]
     [  15. 1703.  320.  587.]]
    [ 98900. 153000.  91300. ... 130700. 253900. 209800.]

In \[104\]:

    #Creating the Amazon Sagemaker endpoint to make predictions from the model dynamically
    linear_regressor = linear.deploy(initial_instance_count = 1,
                                     instance_type = 'ml.m4.xlarge')

    Parameter image will be renamed to image_uri in SageMaker Python SDK v2.

    ----------------!

In \[107\]:

    #Specifying the serializer and deserializer
    from sagemaker.predictor import csv_serializer, json_deserializer
    linear_regressor.content_type = 'text/csv'
    linear_regressor.serializer = csv_serializer
    linear_regressor.deserializer = json_deserializer

In \[108\]:

    #Getting the predicted result
    predicted_result = linear_regressor.predict(X_test)
    print(predicted_result)

    {'predictions': [{'score': 199673.375}, {'score': 151846.03125}, {'score': 116469.078125}, {'score': 229660.75}, {'score': 219234.46875}, {'score': 163002.03125}, {'score': 201281.03125}, {'score': 139292.34375}, {'score': 162170.375}, {'score': 142415.34375}, {'score': 196029.6875}, {'score': 180802.828125}, {'score': 195086.90625}, {'score': 151057.53125}, {'score': 205343.1875}, {'score': 188815.6875}, {'score': 216887.53125}, {'score': 205138.4375}, {'score': 196845.84375}, {'score': 298724.6875}, {'score': 200045.0}, {'score': 165296.65625}, {'score': 162874.4375}, {'score': 230759.171875}, {'score': 198780.8125}, {'score': 191768.03125}, {'score': 174580.1875}, {'score': 235222.46875}, {'score': 104789.8046875}, {'score': 232117.5}, {'score': 206638.28125}, {'score': 235434.09375}, {'score': 200375.15625}, {'score': 203124.6875}, {'score': 274812.34375}, {'score': 159800.765625}, {'score': 166001.890625}, {'score': 248007.59375}, {'score': 194395.3125}, {'score': 186866.4375}, {'score': 208414.9375}, {'score': 169298.109375}, {'score': 104464.3828125}, {'score': 211958.796875}, {'score': 215062.71875}, {'score': 201191.9375}, {'score': 178360.28125}, {'score': 220676.0}, {'score': 190828.0}, {'score': 227489.125}, {'score': 175021.640625}, {'score': 205528.265625}, {'score': 173992.125}, {'score': 204193.34375}, {'score': 151168.28125}, {'score': 157897.46875}, {'score': 150563.140625}, {'score': 121737.5546875}, {'score': 198024.28125}, {'score': 183287.46875}, {'score': 144719.78125}, {'score': 101717.3984375}, {'score': 176093.15625}, {'score': 157102.265625}, {'score': 216231.921875}, {'score': 249125.09375}, {'score': 181083.0625}, {'score': 181033.1875}, {'score': 233472.703125}, {'score': 198757.3125}, {'score': 149255.65625}, {'score': 258209.96875}, {'score': 171056.25}, {'score': 207395.71875}, {'score': 170162.5}, {'score': 179342.875}, {'score': 165383.09375}, {'score': 236813.03125}, {'score': 184260.0}, {'score': 144356.40625}, {'score': 371691.125}, {'score': 92987.2421875}, {'score': 210150.9375}, {'score': 199291.4375}, {'score': 173336.9375}, {'score': 134786.875}, {'score': 231696.125}, {'score': 156883.125}, {'score': 199327.4375}, {'score': 174470.296875}, {'score': 151323.296875}, {'score': 110433.0546875}, {'score': 160931.96875}, {'score': 217617.078125}, {'score': 193262.65625}, {'score': 352567.9375}, {'score': 214593.875}, {'score': 211813.09375}, {'score': 198996.65625}, {'score': 226747.96875}, {'score': 237809.90625}, {'score': 170423.25}, {'score': 196613.96875}, {'score': 236440.875}, {'score': 166418.265625}, {'score': 168276.875}, {'score': 235962.75}, {'score': 260955.90625}, {'score': 219934.0}, {'score': 230440.9375}, {'score': 189588.21875}, {'score': 417008.1875}, {'score': 160372.859375}, {'score': 230071.5}, {'score': 198630.875}, {'score': 180352.71875}, {'score': 133307.5}, {'score': 157998.40625}, {'score': 163737.15625}, {'score': 220681.15625}, {'score': 202019.5625}, {'score': 175497.40625}, {'score': 198353.09375}, {'score': 171532.28125}, {'score': 178936.5}, {'score': 174801.578125}, {'score': 149138.09375}, {'score': 186921.28125}, {'score': 181103.78125}, {'score': 192144.96875}, {'score': 279834.78125}, {'score': 194539.0}, {'score': 182437.125}, {'score': 167826.03125}, {'score': 170488.375}, {'score': 262706.40625}, {'score': 253936.84375}, {'score': 203251.90625}, {'score': 176345.9375}, {'score': 184624.65625}, {'score': 203873.609375}, {'score': 196062.84375}, {'score': 135751.21875}, {'score': 262340.5625}, {'score': 174466.75}, {'score': 245053.625}, {'score': 179699.359375}, {'score': 167191.5625}, {'score': 182789.84375}, {'score': 181404.25}, {'score': 193262.8125}, {'score': 223405.46875}, {'score': 193917.265625}, {'score': 181820.15625}, {'score': 240755.59375}, {'score': 175635.125}, {'score': 331224.75}, {'score': 220962.75}, {'score': 202477.84375}, {'score': 211042.4375}, {'score': 173348.0625}, {'score': 181263.875}, {'score': 154508.03125}, {'score': 227017.5625}, {'score': 170120.09375}, {'score': 195923.125}, {'score': 241531.8125}, {'score': 161345.21875}, {'score': 174354.9375}, {'score': 160190.0625}, {'score': 158113.46875}, {'score': 183496.1875}, {'score': 175938.78125}, {'score': 192762.640625}, {'score': 212091.4375}, {'score': 227457.21875}, {'score': 195929.9375}, {'score': 148826.140625}, {'score': 160559.640625}, {'score': 104719.671875}, {'score': 204402.84375}, {'score': 163452.46875}, {'score': 142616.59375}, {'score': -8497.8828125}, {'score': 252010.0625}, {'score': 267538.375}, {'score': 195321.3125}, {'score': 200409.40625}, {'score': 202504.09375}, {'score': 202481.78125}, {'score': 163785.359375}, {'score': 180530.8125}, {'score': 179420.9375}, {'score': 179312.28125}, {'score': 169341.6875}, {'score': 97414.6484375}, {'score': 157391.875}, {'score': 151582.5625}, {'score': 159474.375}, {'score': 184066.15625}, {'score': 228794.71875}, {'score': 90382.0859375}, {'score': 228816.46875}, {'score': 212488.65625}, {'score': 141780.96875}, {'score': 219230.15625}, {'score': 104902.2109375}, {'score': 172169.25}, {'score': 175899.46875}, {'score': 213609.65625}, {'score': 158223.3125}, {'score': 174237.65625}, {'score': 180743.4375}, {'score': 169107.859375}, {'score': 244773.625}, {'score': 192478.15625}, {'score': 182992.09375}, {'score': 151789.71875}, {'score': 218989.21875}, {'score': 153090.734375}, {'score': 199493.03125}, {'score': 194748.03125}, {'score': 182416.8125}, {'score': 208988.5625}, {'score': 190491.9375}, {'score': 173069.515625}, {'score': 153624.1875}, {'score': 185009.59375}, {'score': 225923.875}, {'score': 200279.625}, {'score': 370978.0}, {'score': 149209.875}, {'score': 180440.703125}, {'score': 217367.59375}, {'score': 233880.5}, {'score': 248937.28125}, {'score': 150843.5}, {'score': 178648.875}, {'score': 258436.65625}, {'score': 157305.34375}, {'score': 201995.3125}, {'score': 188941.65625}, {'score': 172740.375}, {'score': 190006.40625}, {'score': 118496.8046875}, {'score': 224907.53125}, {'score': 167578.25}, {'score': 188306.59375}, {'score': 251897.28125}, {'score': 191566.375}, {'score': 215656.3125}, {'score': 173077.359375}, {'score': 211193.75}, {'score': 151654.0}, {'score': 142553.90625}, {'score': 202360.21875}, {'score': 180486.140625}, {'score': 199844.390625}, {'score': 190213.5}, {'score': 200701.09375}, {'score': 174145.78125}, {'score': 141194.71875}, {'score': 209504.125}, {'score': 210739.953125}, {'score': 221694.09375}, {'score': 207120.53125}, {'score': 214151.90625}, {'score': 221560.21875}, {'score': 144030.4375}, {'score': 187653.3125}, {'score': 228278.75}, {'score': 218371.640625}, {'score': 199426.28125}, {'score': 190753.46875}, {'score': 151658.015625}, {'score': 214152.625}, {'score': 140136.59375}, {'score': 193155.3125}, {'score': 122981.8828125}, {'score': 157743.3125}, {'score': 184963.625}, {'score': 402320.125}, {'score': 200925.125}, {'score': 215278.5625}, {'score': 192302.90625}, {'score': 208025.125}, {'score': 271527.25}, {'score': 154435.96875}, {'score': 148762.40625}, {'score': 274494.5}, {'score': 188853.71875}, {'score': 150273.65625}, {'score': 209623.640625}, {'score': 236517.71875}, {'score': 197954.25}, {'score': 196716.84375}, {'score': 207083.0}, {'score': 136333.53125}, {'score': 158395.0}, {'score': 160957.046875}, {'score': 216062.65625}, {'score': 158162.875}, {'score': 216760.40625}, {'score': 135981.40625}, {'score': 162567.28125}, {'score': 149603.3125}, {'score': 150695.484375}, {'score': 249170.15625}, {'score': 144072.75}, {'score': 179820.125}, {'score': 170326.71875}, {'score': 186422.453125}, {'score': 152330.875}, {'score': 186315.3125}, {'score': 37288.890625}, {'score': 169698.125}, {'score': 186591.28125}, {'score': 183689.25}, {'score': 217433.03125}, {'score': 180131.53125}, {'score': 228430.21875}, {'score': 185301.546875}, {'score': 135128.34375}, {'score': 140046.953125}, {'score': 213320.625}, {'score': 161391.03125}, {'score': 185344.28125}, {'score': 195117.953125}, {'score': 174212.953125}, {'score': 158168.234375}, {'score': 136996.328125}, {'score': 123487.3828125}, {'score': 215711.34375}, {'score': 185566.4375}, {'score': 191178.375}, {'score': 149437.875}, {'score': 180724.84375}, {'score': 174883.671875}, {'score': 207108.484375}, {'score': 160543.1875}, {'score': 133863.25}, {'score': 221693.890625}, {'score': 131080.28125}, {'score': 206897.1875}, {'score': 204759.84375}, {'score': 236165.375}, {'score': 172747.734375}, {'score': 210785.015625}, {'score': 225211.671875}, {'score': 235568.375}, {'score': 198289.96875}, {'score': 208671.484375}, {'score': 279548.9375}, {'score': 206761.125}, {'score': 196957.71875}, {'score': 204347.125}, {'score': 170062.59375}, {'score': 161127.9375}, {'score': 270438.65625}, {'score': 148490.9375}, {'score': 193654.9375}, {'score': 190538.84375}, {'score': 230450.125}, {'score': 153703.9375}, {'score': 218345.03125}, {'score': 196050.046875}, {'score': 206502.90625}, {'score': 184549.9375}, {'score': 105790.0546875}, {'score': 180194.671875}, {'score': 195842.09375}, {'score': 209437.6875}, {'score': 222422.625}, {'score': 145712.0}, {'score': 161298.46875}, {'score': 193248.65625}, {'score': 122101.7265625}, {'score': 226274.0}, {'score': 249786.15625}, {'score': 180742.5}, {'score': 191153.78125}, {'score': 220206.0}, {'score': 365820.5}, {'score': 211969.21875}, {'score': 246350.6875}, {'score': 165470.65625}, {'score': 195312.46875}, {'score': 135080.59375}, {'score': 168420.0}, {'score': 177139.671875}, {'score': 153446.84375}, {'score': 170042.0625}, {'score': 214970.25}, {'score': 165725.1875}, {'score': 159587.546875}, {'score': 190119.1875}, {'score': 193356.28125}, {'score': 208312.6875}, {'score': 192418.375}, {'score': 159467.859375}, {'score': 163491.125}, {'score': 193073.65625}, {'score': 164380.46875}, {'score': 229894.28125}, {'score': 193531.484375}, {'score': 179550.625}, {'score': 175958.46875}, {'score': 177017.96875}, {'score': 221916.75}, {'score': 182280.71875}, {'score': 254298.65625}, {'score': 195111.65625}, {'score': 211804.21875}, {'score': 308923.1875}, {'score': 177260.15625}, {'score': 145945.90625}, {'score': 237808.875}, {'score': 139891.5}, {'score': 151389.375}, {'score': 197290.9375}, {'score': 143353.140625}, {'score': 160884.71875}, {'score': 222071.5625}, {'score': 225448.75}, {'score': 157990.84375}, {'score': 180905.96875}, {'score': 177956.328125}, {'score': 155576.90625}, {'score': 135493.6875}, {'score': 185520.15625}, {'score': 212282.125}, {'score': 158187.8125}, {'score': 185002.0}, {'score': 243937.34375}, {'score': 200115.96875}, {'score': 186917.875}, {'score': 165668.703125}, {'score': 181247.0625}, {'score': 221672.59375}, {'score': 209636.90625}, {'score': 156040.28125}, {'score': 202923.375}, {'score': 198808.6875}, {'score': 179559.765625}, {'score': 188840.125}, {'score': 164345.453125}, {'score': 209491.03125}, {'score': 178302.90625}, {'score': 202786.3125}, {'score': 205465.1875}, {'score': 188440.5}, {'score': 223326.125}, {'score': 211266.46875}, {'score': 145166.3125}, {'score': 207531.875}, {'score': 233465.234375}, {'score': 206221.375}, {'score': 213973.46875}, {'score': 207040.75}, {'score': 208414.375}, {'score': 193544.59375}, {'score': 198416.84375}, {'score': 192628.03125}, {'score': 128279.9765625}, {'score': 177149.8125}, {'score': 283836.4375}, {'score': 172372.75}, {'score': 141339.125}, {'score': 220586.0}, {'score': 230902.1875}, {'score': 167970.71875}, {'score': 225630.765625}, {'score': 166032.546875}, {'score': 153502.78125}, {'score': 192484.53125}, {'score': 149581.875}, {'score': 167410.203125}, {'score': 187428.15625}, {'score': 212579.84375}, {'score': 238418.46875}, {'score': 244857.84375}, {'score': 211237.65625}, {'score': 273150.59375}, {'score': 198819.34375}, {'score': 194483.5625}, {'score': 200732.84375}, {'score': 175913.375}, {'score': 153339.3125}, {'score': 233240.75}, {'score': 185646.84375}, {'score': 183931.671875}, {'score': 182467.78125}, {'score': 199642.453125}, {'score': 207338.875}, {'score': 148775.125}, {'score': 167515.875}, {'score': 175938.484375}, {'score': 204075.65625}, {'score': 174681.15625}, {'score': 160698.375}, {'score': 146142.125}, {'score': 218908.75}, {'score': 180996.28125}, {'score': 199193.0}, {'score': 145294.3125}, {'score': 193260.40625}, {'score': 205636.25}, {'score': 219029.25}, {'score': 176445.15625}, {'score': 215868.65625}, {'score': 137308.59375}, {'score': 162575.125}, {'score': 197574.375}, {'score': 210119.9375}, {'score': 149799.84375}, {'score': 252234.28125}, {'score': 111649.1875}, {'score': 186028.09375}, {'score': 177993.9375}, {'score': 175808.53125}, {'score': 158617.0625}, {'score': 163781.71875}, {'score': 203000.5}, {'score': 205230.5625}, {'score': 133817.40625}, {'score': 236189.625}, {'score': 205464.5625}, {'score': 187745.25}, {'score': 217232.5}, {'score': 168354.421875}, {'score': 190957.515625}, {'score': 184987.90625}, {'score': 221714.984375}, {'score': 178506.375}, {'score': 342950.28125}, {'score': 207501.71875}, {'score': 159309.375}, {'score': 221949.65625}, {'score': 187368.59375}, {'score': 203056.25}, {'score': 154949.53125}, {'score': 199687.09375}, {'score': 224788.03125}, {'score': 196422.1875}, {'score': 160242.984375}, {'score': 193791.1875}, {'score': 163667.8125}, {'score': 129588.8046875}, {'score': 208387.5625}, {'score': 188987.34375}, {'score': 161865.84375}, {'score': 197792.71875}, {'score': 223185.546875}, {'score': 210553.75}, {'score': 174375.875}, {'score': 238414.625}, {'score': 185585.125}, {'score': 215264.0625}, {'score': 202332.96875}, {'score': 208266.09375}, {'score': 178748.625}, {'score': 176224.65625}, {'score': 331699.5625}, {'score': 193115.140625}, {'score': 179166.328125}, {'score': 192551.71875}, {'score': 217406.84375}, {'score': 255600.125}, {'score': 138505.75}, {'score': 199165.84375}, {'score': 158444.140625}, {'score': 213358.03125}, {'score': 139819.03125}, {'score': 188607.125}, {'score': 140045.75}, {'score': 176907.8125}, {'score': 218049.59375}, {'score': 220978.46875}, {'score': 179367.015625}, {'score': 235479.78125}, {'score': 250415.34375}, {'score': 176003.78125}, {'score': 158681.03125}, {'score': 170677.0}, {'score': 147212.671875}, {'score': 193677.0}, {'score': 225619.59375}, {'score': 179988.8125}, {'score': 218755.171875}, {'score': 211813.125}, {'score': 173494.375}, {'score': 193595.4375}, {'score': 172407.59375}, {'score': 251440.90625}, {'score': 178393.96875}, {'score': 124251.3984375}, {'score': 197078.40625}, {'score': 185141.15625}, {'score': 192128.703125}, {'score': 216904.375}, {'score': 172057.125}, {'score': 189289.3125}, {'score': 199027.625}, {'score': 217033.5625}, {'score': 306082.03125}, {'score': 172000.625}, {'score': 175830.125}, {'score': 236648.6875}, {'score': 206851.75}, {'score': 168765.40625}, {'score': 172617.71875}, {'score': 189564.53125}, {'score': 143348.3125}, {'score': 192173.8125}, {'score': 150945.78125}, {'score': 302815.875}, {'score': 224947.21875}, {'score': 151993.84375}, {'score': 196905.40625}, {'score': 168881.90625}, {'score': 140870.515625}, {'score': 228356.15625}, {'score': 156678.96875}, {'score': 182637.40625}, {'score': 231590.84375}, {'score': 180164.359375}, {'score': 185462.125}, {'score': 247882.0625}, {'score': 174636.3125}, {'score': 169610.59375}, {'score': 186789.375}, {'score': 206276.21875}, {'score': 137138.09375}, {'score': 104965.7734375}, {'score': 162966.75}, {'score': 97223.0234375}, {'score': 282138.28125}, {'score': 151117.671875}, {'score': 206412.5625}, {'score': 206592.46875}, {'score': 201521.28125}, {'score': 192371.21875}, {'score': 165432.03125}, {'score': 235048.0}, {'score': 204956.15625}, {'score': 186261.09375}, {'score': 168777.390625}, {'score': 214957.21875}, {'score': 147653.53125}, {'score': 179582.09375}, {'score': 217549.0625}, {'score': 170756.6875}, {'score': 213890.46875}, {'score': 186857.59375}, {'score': 166283.71875}, {'score': 138839.875}, {'score': 244612.0}, {'score': 96966.6484375}, {'score': 225903.78125}, {'score': 184670.4375}, {'score': 213494.96875}, {'score': 155277.6875}, {'score': 225192.125}, {'score': 248223.03125}, {'score': 162907.375}, {'score': 183635.796875}, {'score': 181162.46875}, {'score': 235495.6875}, {'score': 196467.84375}, {'score': 172061.21875}, {'score': 187518.125}, {'score': 199506.5625}, {'score': 146146.6875}, {'score': 241528.375}, {'score': 150591.5625}, {'score': 194732.0}, {'score': 127705.015625}, {'score': 213731.9375}, {'score': 212534.6875}, {'score': 172376.84375}, {'score': 125717.859375}, {'score': 215757.15625}, {'score': 152748.875}, {'score': 169192.421875}, {'score': 183684.21875}, {'score': 195427.0}, {'score': 202231.6875}, {'score': 224765.03125}, {'score': 178730.25}, {'score': 156137.078125}, {'score': 222425.453125}, {'score': 97939.6328125}, {'score': 186028.65625}, {'score': 192874.9375}, {'score': 173705.609375}, {'score': 185814.140625}, {'score': 200532.0625}, {'score': 153106.546875}, {'score': 143108.1875}, {'score': 216423.4375}, {'score': 143885.84375}, {'score': 134180.578125}, {'score': 200638.546875}, {'score': 173510.40625}, {'score': 207958.6875}, {'score': 132696.15625}, {'score': 184334.46875}, {'score': 121956.7734375}, {'score': 133994.53125}, {'score': 233356.71875}, {'score': 168721.84375}, {'score': 236090.671875}, {'score': 111451.4296875}, {'score': 218981.859375}, {'score': 318320.46875}, {'score': 181391.875}, {'score': 176363.734375}, {'score': 229808.34375}, {'score': 152289.8125}, {'score': 182497.8125}, {'score': 180809.609375}, {'score': 146615.109375}, {'score': 197692.78125}, {'score': 193133.4375}, {'score': 171398.015625}, {'score': 158124.75}, {'score': 170514.0625}, {'score': 167454.390625}, {'score': 178208.09375}, {'score': 187258.53125}, {'score': 185553.28125}, {'score': 162194.6875}, {'score': 220693.5}, {'score': 168635.96875}, {'score': 130137.359375}, {'score': 219530.34375}, {'score': 187530.3125}, {'score': 203557.75}, {'score': 237408.03125}, {'score': 154158.546875}, {'score': 224323.09375}, {'score': 214660.4375}, {'score': 237013.5625}, {'score': 129820.265625}, {'score': 159934.65625}, {'score': 163863.28125}, {'score': 189347.3125}, {'score': 303730.71875}, {'score': 111985.0234375}, {'score': 157598.25}, {'score': 234609.09375}, {'score': 120302.484375}, {'score': 179557.90625}, {'score': 136528.375}, {'score': 151039.84375}, {'score': 160769.84375}, {'score': 160810.53125}, {'score': 240943.875}, {'score': 156226.9375}, {'score': 191622.65625}, {'score': 196613.34375}, {'score': 157776.8125}, {'score': 167865.65625}, {'score': 200806.0625}, {'score': 269629.875}, {'score': 179303.484375}, {'score': 173714.265625}, {'score': 188968.65625}, {'score': 92680.140625}, {'score': 155070.46875}, {'score': 140806.75}, {'score': 135985.375}, {'score': 199262.96875}, {'score': 178763.609375}, {'score': 158593.78125}, {'score': 98373.5546875}, {'score': 145630.9375}, {'score': 92830.71875}, {'score': 167917.8125}, {'score': 150571.21875}, {'score': 198082.1875}, {'score': 176936.5}, {'score': 115281.515625}, {'score': 225751.0625}, {'score': 173294.75}, {'score': 191453.34375}, {'score': 246694.0625}, {'score': 228744.03125}, {'score': 198364.34375}, {'score': 201123.90625}, {'score': 236433.9375}, {'score': 212670.6875}, {'score': 169902.578125}, {'score': 183023.6875}, {'score': 253076.125}, {'score': 145781.78125}, {'score': 322770.84375}, {'score': 196751.8125}, {'score': 215088.65625}, {'score': 179454.125}, {'score': 238987.90625}, {'score': 160239.703125}, {'score': 190101.0625}, {'score': 140104.5625}, {'score': 192991.609375}, {'score': 235847.09375}, {'score': 184208.40625}, {'score': 200963.71875}, {'score': 188089.375}, {'score': 212017.21875}, {'score': 168867.40625}, {'score': 208713.046875}, {'score': 291738.4375}, {'score': 184798.40625}, {'score': 169682.71875}, {'score': 190385.59375}, {'score': 210728.828125}, {'score': 284812.84375}, {'score': 154871.4375}, {'score': 215965.171875}, {'score': 195289.84375}, {'score': 209232.828125}, {'score': 169229.1875}, {'score': 183824.03125}, {'score': 272403.5}, {'score': 200892.59375}, {'score': 137403.59375}, {'score': 131381.75}, {'score': 186314.953125}, {'score': 246498.21875}, {'score': 98128.2890625}, {'score': 130847.4140625}, {'score': 288707.96875}, {'score': 92385.8203125}, {'score': 209578.1875}, {'score': 184310.4375}, {'score': 180240.4375}, {'score': 196026.34375}, {'score': 255620.03125}, {'score': 168491.75}, {'score': 122468.421875}, {'score': 246145.3125}, {'score': 203598.8125}, {'score': 146966.578125}, {'score': 197888.15625}, {'score': 195637.90625}, {'score': 190618.8125}, {'score': 207537.96875}, {'score': 136554.09375}, {'score': 156734.453125}, {'score': 192163.03125}, {'score': 152002.1875}, {'score': 179210.96875}, {'score': 172536.75}, {'score': 187948.78125}, {'score': 179584.96875}, {'score': 174374.671875}, {'score': 218733.625}, {'score': 281111.6875}, {'score': 255508.6875}, {'score': 165578.953125}, {'score': 197873.25}, {'score': 146719.28125}, {'score': 162715.6875}, {'score': 255490.84375}, {'score': 193405.28125}, {'score': 208489.84375}, {'score': 175718.125}, {'score': 226389.125}, {'score': 198961.375}, {'score': 182216.96875}, {'score': 154703.328125}, {'score': 217409.640625}, {'score': 183690.40625}, {'score': 198888.375}, {'score': 194859.0625}, {'score': 179579.046875}, {'score': 190238.96875}, {'score': 222004.46875}, {'score': 150699.78125}, {'score': 181582.75}, {'score': 227129.15625}, {'score': 213280.625}, {'score': 187657.21875}, {'score': 152574.6875}, {'score': 217429.046875}, {'score': 149874.15625}, {'score': 310260.53125}, {'score': 204966.4375}, {'score': 151693.609375}, {'score': 232261.625}, {'score': 136927.890625}, {'score': 185056.03125}, {'score': 203819.53125}, {'score': 153345.40625}, {'score': 192867.4375}, {'score': 214636.0625}, {'score': 161099.1875}, {'score': 181617.5}, {'score': 202561.03125}, {'score': 158813.75}, {'score': 179803.21875}, {'score': 168047.734375}, {'score': 166146.03125}, {'score': 122831.2578125}, {'score': 167132.875}, {'score': 203385.265625}, {'score': 206297.484375}, {'score': 169049.4375}, {'score': 161556.8125}, {'score': 150777.828125}, {'score': 224010.6875}, {'score': 198233.859375}, {'score': 212780.90625}, {'score': 188809.875}, {'score': 241222.75}, {'score': 194436.828125}, {'score': 138727.96875}, {'score': 190045.09375}, {'score': 483078.5625}, {'score': 132627.703125}, {'score': 160155.8125}, {'score': 165131.515625}, {'score': 206461.484375}, {'score': 153184.03125}, {'score': 186729.75}, {'score': 181308.09375}, {'score': 215042.859375}, {'score': 213562.328125}, {'score': 215781.09375}, {'score': 209042.4375}, {'score': 266350.125}, {'score': 252999.25}, {'score': 206249.875}, {'score': 199307.84375}, {'score': 200368.59375}, {'score': 131439.890625}, {'score': 208395.34375}, {'score': 209915.28125}, {'score': 205229.25}, {'score': 202030.09375}, {'score': 187314.75}, {'score': 197389.78125}, {'score': 134364.1875}, {'score': 238823.15625}, {'score': 167990.734375}, {'score': 150310.875}, {'score': 241836.34375}, {'score': 219793.953125}, {'score': 243120.5}, {'score': 177530.828125}, {'score': 204495.5625}, {'score': 185123.53125}, {'score': 209026.765625}, {'score': 238449.4375}, {'score': 132533.46875}, {'score': 174859.78125}, {'score': 161733.34375}, {'score': 193619.8125}, {'score': 132159.875}, {'score': 224720.453125}, {'score': 200747.125}, {'score': 212102.90625}, {'score': 184847.0}, {'score': 199881.78125}, {'score': 208157.0625}, {'score': 171573.96875}, {'score': 193196.53125}, {'score': 174677.5625}, {'score': 194154.625}, {'score': 155876.75}, {'score': 203491.09375}, {'score': 285720.375}, {'score': 131491.9375}, {'score': 176359.8125}, {'score': 210098.9375}, {'score': 110825.0234375}, {'score': 178268.34375}, {'score': 254683.0625}, {'score': 215983.78125}, {'score': 72816.2421875}, {'score': 162165.40625}, {'score': 177387.4375}, {'score': 199609.09375}, {'score': 184995.484375}, {'score': 186251.9375}, {'score': 292473.40625}, {'score': 262380.65625}, {'score': 138209.6875}, {'score': 184939.71875}, {'score': 186119.53125}, {'score': 150378.421875}, {'score': 242273.90625}, {'score': 132726.78125}, {'score': 188965.5625}, {'score': 169838.140625}, {'score': 187183.1875}, {'score': 168667.125}, {'score': 148530.125}, {'score': 136936.125}, {'score': 239046.90625}, {'score': 159133.84375}, {'score': 187702.3125}, {'score': 207248.8125}, {'score': 167257.828125}, {'score': 198273.5625}, {'score': 234456.21875}, {'score': 174908.96875}, {'score': 187431.34375}, {'score': 279965.65625}, {'score': 175741.03125}, {'score': 151629.9375}, {'score': 182291.46875}, {'score': 159284.03125}, {'score': 198144.8125}, {'score': 133111.1875}, {'score': 248761.3125}, {'score': 212711.65625}, {'score': 160160.71875}, {'score': 181389.25}, {'score': 136848.78125}, {'score': 168313.34375}, {'score': 182213.484375}, {'score': 162028.046875}, {'score': 225661.71875}, {'score': 209813.921875}, {'score': 167489.71875}, {'score': 215636.8125}, {'score': 180232.625}, {'score': 183067.3125}, {'score': 193034.625}, {'score': 192448.53125}, {'score': 177312.875}, {'score': 143373.4375}, {'score': 192931.015625}, {'score': 269651.96875}, {'score': 205628.3125}, {'score': 208509.90625}, {'score': 205069.4375}, {'score': 201899.78125}, {'score': 166252.75}, {'score': 154332.0625}, {'score': 215573.34375}, {'score': 211261.03125}, {'score': 199825.59375}, {'score': 191725.296875}, {'score': 217452.59375}, {'score': 202442.09375}, {'score': 212656.65625}, {'score': 180802.09375}, {'score': 196645.8125}, {'score': 170590.9375}, {'score': 149877.1875}, {'score': 285755.59375}, {'score': 199386.125}, {'score': 192721.296875}, {'score': 175993.578125}, {'score': 224900.203125}, {'score': 119535.6171875}, {'score': 188232.1875}, {'score': 185147.3125}, {'score': 202043.875}, {'score': 135747.15625}, {'score': 181616.46875}, {'score': 217049.59375}, {'score': 228179.4375}, {'score': 184500.03125}, {'score': 179341.75}, {'score': 163067.6875}, {'score': 153778.109375}, {'score': 214007.90625}, {'score': 179622.15625}, {'score': 192971.34375}, {'score': 200589.96875}, {'score': 208277.25}, {'score': 160708.9375}, {'score': 186248.328125}, {'score': 202367.71875}, {'score': 190862.59375}, {'score': 179866.5}, {'score': 189794.375}, {'score': 218263.78125}, {'score': 210125.96875}, {'score': 215166.90625}, {'score': 165517.09375}, {'score': 357333.375}, {'score': 229769.71875}, {'score': 150663.015625}, {'score': 167034.21875}, {'score': 292696.53125}, {'score': 206247.78125}, {'score': 197944.34375}, {'score': 206700.5625}, {'score': 204555.5625}, {'score': 185070.875}, {'score': 227595.46875}, {'score': 169779.15625}, {'score': 218396.53125}, {'score': 208677.3125}, {'score': 186631.96875}, {'score': 174327.28125}, {'score': 191697.78125}, {'score': 169394.421875}, {'score': 194584.71875}, {'score': 160710.03125}, {'score': 192587.90625}, {'score': 212577.21875}, {'score': 222325.09375}, {'score': 228651.8125}, {'score': 185550.125}, {'score': 166574.171875}, {'score': 253813.0}, {'score': 167332.0}, {'score': 232383.65625}, {'score': 202058.59375}, {'score': 156207.5}, {'score': 137929.640625}, {'score': 171934.625}, {'score': 134441.09375}, {'score': 235311.0}, {'score': 106321.2421875}, {'score': 210413.03125}, {'score': 239709.390625}, {'score': 239356.28125}, {'score': 160224.1875}, {'score': 203857.4375}, {'score': 208926.8125}, {'score': 210008.171875}, {'score': 162166.546875}, {'score': 182023.3125}, {'score': 175530.8125}, {'score': 163015.65625}, {'score': 218198.34375}, {'score': 207839.625}, {'score': 131040.4296875}, {'score': 167627.96875}, {'score': 151372.4375}, {'score': 207996.75}, {'score': 147946.21875}, {'score': 197501.28125}, {'score': 173335.546875}, {'score': 194397.96875}, {'score': 225352.09375}, {'score': 221266.21875}, {'score': 161416.8125}, {'score': 235505.34375}, {'score': 181171.21875}, {'score': 175977.8125}, {'score': 173596.671875}, {'score': 276225.375}, {'score': 253743.28125}, {'score': 170683.875}, {'score': 157832.03125}, {'score': 180289.28125}, {'score': 171103.921875}, {'score': 166850.96875}, {'score': 181503.796875}, {'score': 194152.109375}, {'score': 192505.328125}, {'score': 218380.09375}, {'score': 114499.96875}, {'score': 162324.6875}, {'score': 179371.859375}, {'score': 200122.0625}, {'score': 190113.0}, {'score': 164157.171875}, {'score': 195115.4375}, {'score': 205564.828125}, {'score': 185528.25}, {'score': 140095.21875}, {'score': 179473.71875}, {'score': 168731.75}, {'score': 170339.125}, {'score': 187484.40625}, {'score': 192553.8125}, {'score': 165785.9375}, {'score': 197678.53125}, {'score': 215451.390625}, {'score': 184924.125}, {'score': 226263.5625}, {'score': 179430.09375}, {'score': 244078.78125}, {'score': 194992.96875}, {'score': 250736.21875}, {'score': 214001.28125}, {'score': 193169.28125}, {'score': 189652.9375}, {'score': 123438.2265625}, {'score': 217896.96875}, {'score': 169599.359375}, {'score': 220131.59375}, {'score': 180000.1875}, {'score': 151628.4375}, {'score': 171940.015625}, {'score': 250770.25}, {'score': 221091.25}, {'score': 192158.78125}, {'score': 157654.4375}, {'score': 175775.28125}, {'score': 167348.625}, {'score': 215505.84375}, {'score': 189220.1875}, {'score': 153129.8125}, {'score': 140701.78125}, {'score': 358861.625}, {'score': 186659.4375}, {'score': 196151.40625}, {'score': 232887.3125}, {'score': 207973.9375}, {'score': 186006.625}, {'score': 237080.75}, {'score': 207745.5625}, {'score': 49182.7890625}, {'score': 170034.125}, {'score': 244913.8125}, {'score': 126774.2421875}, {'score': 129339.0546875}, {'score': 194872.09375}, {'score': 178417.84375}, {'score': 153603.78125}, {'score': 166849.984375}, {'score': 239840.375}, {'score': 122062.8984375}, {'score': 185388.1875}, {'score': 161409.4375}, {'score': 191798.75}, {'score': 162923.65625}, {'score': 170786.59375}, {'score': 224459.78125}, {'score': 255709.53125}, {'score': 187018.421875}, {'score': 162170.4375}, {'score': 183653.140625}, {'score': 189399.4375}, {'score': 167447.421875}, {'score': 190686.90625}, {'score': 209967.78125}, {'score': 198161.1875}, {'score': 214386.0}, {'score': 250663.90625}, {'score': 238085.09375}, {'score': 205977.078125}, {'score': 157373.90625}, {'score': 241998.40625}, {'score': 245790.96875}, {'score': 314389.375}, {'score': 160896.625}, {'score': 153035.03125}, {'score': 174901.53125}, {'score': 246202.5625}, {'score': 169326.4375}, {'score': 192265.125}, {'score': 218323.375}, {'score': 157249.375}, {'score': 183795.28125}, {'score': 174559.1875}, {'score': 188580.0}, {'score': 193292.8125}, {'score': 180197.65625}, {'score': 167652.015625}, {'score': 181735.0}, {'score': 196164.625}, {'score': 169477.03125}, {'score': 205407.0625}, {'score': 180092.0}, {'score': 211568.875}, {'score': 221065.1875}, {'score': 168882.640625}, {'score': 151144.28125}, {'score': 196708.71875}, {'score': 152400.8125}, {'score': 228169.5}, {'score': 197443.75}, {'score': 175650.5625}, {'score': 154213.8125}, {'score': 173494.171875}, {'score': 249027.5}, {'score': 290727.625}, {'score': 202022.671875}, {'score': 220882.875}, {'score': 222596.03125}, {'score': 198226.5}, {'score': 132719.765625}, {'score': 221573.34375}, {'score': 232183.25}, {'score': 198466.0}, {'score': 158079.0}, {'score': 178504.8125}, {'score': 229368.84375}, {'score': 250383.875}, {'score': 408217.09375}, {'score': 127684.0859375}, {'score': 214384.4375}, {'score': 217550.125}, {'score': 201844.875}, {'score': 157702.9375}, {'score': 266739.53125}, {'score': 499849.28125}, {'score': 139351.234375}, {'score': 216165.765625}, {'score': 146191.078125}, {'score': 209972.78125}, {'score': 182573.6875}, {'score': 167813.1875}, {'score': 180383.5}, {'score': 188420.34375}, {'score': 170501.1875}, {'score': 197267.09375}, {'score': 187525.625}, {'score': 232044.359375}, {'score': 200162.515625}, {'score': 195322.375}, {'score': 180023.84375}, {'score': 197855.28125}, {'score': 154800.53125}, {'score': 175675.71875}, {'score': 248230.3125}, {'score': 149646.09375}, {'score': 165574.4375}, {'score': 178594.8125}, {'score': 171269.84375}, {'score': 152966.3125}, {'score': 135402.71875}, {'score': 120039.3125}, {'score': 272567.34375}, {'score': 169920.75}, {'score': 157758.40625}, {'score': 174045.5625}, {'score': 171496.53125}, {'score': 184864.484375}, {'score': 224142.125}, {'score': 187114.875}, {'score': 216242.75}, {'score': 220984.40625}, {'score': 182845.5625}, {'score': 120341.9296875}, {'score': 155394.625}, {'score': 171214.5}, {'score': 133405.671875}, {'score': 209117.5}, {'score': 188842.921875}, {'score': 175775.1875}, {'score': 174992.1875}, {'score': 186984.0}, {'score': 187259.984375}, {'score': 185494.03125}, {'score': 158876.09375}, {'score': 180389.96875}, {'score': 227153.625}, {'score': 237933.5}, {'score': 200889.65625}, {'score': 156845.90625}, {'score': 184962.21875}, {'score': 185475.1875}, {'score': 174127.875}, {'score': 217754.3125}, {'score': 190108.625}, {'score': 198551.203125}, {'score': 208753.625}, {'score': 214968.171875}, {'score': 267570.15625}, {'score': 151023.375}, {'score': 159672.21875}, {'score': 218891.53125}, {'score': 389379.875}, {'score': 199354.0625}, {'score': 308635.375}, {'score': 197735.03125}, {'score': 188260.265625}, {'score': 188240.1875}, {'score': 146532.234375}, {'score': 205738.0}, {'score': 233107.046875}, {'score': 189145.453125}, {'score': 158838.8125}, {'score': 161811.3125}, {'score': 208293.390625}, {'score': 165945.890625}, {'score': 165386.0625}, {'score': 158729.0}, {'score': 189766.3125}, {'score': 190149.421875}, {'score': 196835.84375}, {'score': 156216.46875}, {'score': 150421.953125}, {'score': 160412.5625}, {'score': 171846.46875}, {'score': 223315.65625}, {'score': 118496.4921875}, {'score': 169364.359375}, {'score': 183010.109375}, {'score': 238309.59375}, {'score': 171382.734375}, {'score': 48847.4921875}, {'score': 144894.28125}, {'score': 204070.875}, {'score': 171137.65625}, {'score': 196217.75}, {'score': 78858.0703125}, {'score': 215729.03125}, {'score': 216841.78125}, {'score': 223594.90625}, {'score': 166255.4375}, {'score': 151348.21875}, {'score': 233232.0}, {'score': 94271.0390625}, {'score': 228481.96875}, {'score': 144683.21875}, {'score': 182800.4375}, {'score': 78108.9921875}, {'score': 173436.28125}, {'score': 168896.65625}, {'score': 207189.09375}, {'score': 132204.6875}, {'score': 217224.265625}, {'score': 178075.4375}, {'score': 219397.0}, {'score': 149068.15625}, {'score': 183363.6875}, {'score': 202779.9375}, {'score': 198976.21875}, {'score': 195449.421875}, {'score': 162985.8125}, {'score': 174284.8125}, {'score': 223553.78125}, {'score': 135385.390625}, {'score': 210248.75}, {'score': 184842.875}, {'score': 209513.71875}, {'score': 151849.421875}, {'score': 228267.59375}, {'score': 165287.34375}, {'score': 234051.234375}, {'score': 220219.71875}, {'score': 203375.65625}, {'score': 169319.40625}, {'score': 185360.546875}, {'score': 168115.125}, {'score': 165113.0625}, {'score': 235560.96875}, {'score': 184645.09375}, {'score': 119989.046875}, {'score': 189884.34375}, {'score': 221839.8125}, {'score': 131928.65625}, {'score': 209465.46875}, {'score': 169671.75}, {'score': 181526.125}, {'score': 151601.5}, {'score': 187147.5}, {'score': 424006.5}, {'score': 223705.96875}, {'score': 155177.34375}, {'score': 198107.25}, {'score': 212762.359375}, {'score': 243737.125}, {'score': 249254.5}, {'score': 199893.375}, {'score': 252301.375}, {'score': 204349.15625}, {'score': 182564.578125}, {'score': 169530.5625}, {'score': 221797.875}, {'score': 217057.84375}, {'score': 194767.34375}, {'score': 206655.84375}, {'score': 199448.9375}, {'score': 210895.09375}, {'score': 194684.03125}, {'score': 167833.65625}, {'score': 208694.25}, {'score': 202424.3125}, {'score': 151414.6875}, {'score': 153442.78125}, {'score': 182072.40625}, {'score': 194522.78125}, {'score': 232816.21875}, {'score': 202142.1875}, {'score': 199231.46875}, {'score': 228610.4375}, {'score': 232074.1875}, {'score': 245242.75}, {'score': 180495.984375}, {'score': 232187.1875}, {'score': 183162.125}, {'score': 145575.0625}, {'score': 200519.3125}, {'score': 243644.09375}, {'score': 188621.234375}, {'score': 161953.09375}, {'score': 142304.0625}, {'score': 145502.1875}, {'score': 176358.125}, {'score': 303184.8125}, {'score': 93904.9609375}, {'score': 150269.171875}, {'score': 210547.8125}, {'score': 186386.046875}, {'score': 177131.1875}, {'score': 178452.796875}, {'score': 205594.78125}, {'score': 203853.9375}, {'score': 240814.84375}, {'score': 157901.25}, {'score': 183390.53125}, {'score': 209285.84375}, {'score': 211074.734375}, {'score': 184326.0625}, {'score': 220442.921875}, {'score': 211832.53125}, {'score': 243958.5625}, {'score': 170600.65625}, {'score': 216453.65625}, {'score': 180774.140625}, {'score': 201437.21875}, {'score': 252637.4375}, {'score': 209600.75}, {'score': 180162.0625}, {'score': 159777.125}, {'score': 162952.84375}, {'score': 179168.5625}, {'score': 285527.59375}, {'score': 273714.78125}, {'score': 256149.59375}, {'score': 167621.0625}, {'score': 229768.484375}, {'score': 165867.65625}, {'score': 145160.1875}, {'score': 167207.59375}, {'score': 160751.65625}, {'score': 211483.15625}, {'score': 186755.40625}, {'score': 227957.515625}, {'score': 200549.4375}, {'score': 206914.53125}, {'score': 200658.90625}, {'score': 223686.65625}, {'score': 165350.4375}, {'score': 203600.09375}, {'score': 140292.625}, {'score': 188109.625}, {'score': 234974.96875}, {'score': 144808.0625}, {'score': 176721.609375}, {'score': 149921.875}, {'score': 152360.46875}, {'score': 193385.1875}, {'score': 207749.5625}, {'score': 166196.765625}, {'score': 200043.5625}, {'score': 393993.6875}, {'score': 209590.328125}, {'score': 199442.390625}, {'score': 147255.15625}, {'score': 146835.53125}, {'score': 171950.375}, {'score': 283507.15625}, {'score': 184477.53125}, {'score': 155528.546875}, {'score': 197758.6875}, {'score': 252878.125}, {'score': 131822.8125}, {'score': 182131.6875}, {'score': 186199.71875}, {'score': 188622.96875}, {'score': 267105.9375}, {'score': 178309.1875}, {'score': 199421.8125}, {'score': 194788.625}, {'score': 167187.875}, {'score': 192592.75}, {'score': 222170.59375}, {'score': 225222.375}, {'score': 229410.71875}, {'score': 182528.875}, {'score': 174016.6875}, {'score': 181562.375}, {'score': 603827.1875}, {'score': 185575.65625}, {'score': 219238.5}, {'score': 181728.375}, {'score': 240796.875}, {'score': 150166.3125}, {'score': 198427.9375}, {'score': 222547.875}, {'score': 229001.03125}, {'score': 161988.90625}, {'score': 209375.84375}, {'score': 274505.71875}, {'score': 184583.046875}, {'score': 187090.53125}, {'score': 218232.03125}, {'score': 146475.953125}, {'score': 103136.0703125}, {'score': 246996.3125}, {'score': 216713.734375}, {'score': 261199.9375}, {'score': 187900.96875}, {'score': 197717.296875}, {'score': 192455.0625}, {'score': 191145.890625}, {'score': 192933.84375}, {'score': 200156.5625}, {'score': 155569.8125}, {'score': 111733.5859375}, {'score': 188272.84375}, {'score': 201755.40625}, {'score': 179580.6875}, {'score': 194371.9375}, {'score': 236854.1875}, {'score': 192826.71875}, {'score': 216191.453125}, {'score': 177750.84375}, {'score': 169804.6875}, {'score': 213977.484375}, {'score': 167915.0}, {'score': 179935.34375}, {'score': 196350.765625}, {'score': 212194.5625}, {'score': 184181.875}, {'score': 163781.125}, {'score': 198023.546875}, {'score': 141343.125}, {'score': 184066.875}, {'score': 119826.9921875}, {'score': 173272.15625}, {'score': 203404.171875}, {'score': 213290.1875}, {'score': 144064.15625}, {'score': 239381.78125}, {'score': 148556.609375}, {'score': 167363.125}, {'score': 184534.59375}, {'score': 206523.46875}, {'score': 223198.21875}, {'score': 148625.46875}, {'score': 137731.796875}, {'score': 155343.5625}, {'score': 248946.40625}, {'score': 214483.8125}, {'score': 172460.0}, {'score': 186819.46875}, {'score': 205551.34375}, {'score': 185669.5625}, {'score': 162517.875}, {'score': 122140.2421875}, {'score': 168117.375}, {'score': 196013.59375}, {'score': 181514.6875}, {'score': 201868.8125}, {'score': 195186.375}, {'score': -29324.6953125}, {'score': 201005.25}, {'score': 172844.03125}, {'score': 160474.90625}, {'score': 193383.8125}, {'score': 244248.9375}, {'score': 141473.0625}, {'score': 208371.75}, {'score': 188190.9375}, {'score': 195710.5625}, {'score': 353339.90625}, {'score': 244327.421875}, {'score': 186997.65625}, {'score': 235125.375}, {'score': 158435.828125}, {'score': 257966.6875}, {'score': 235930.8125}, {'score': 250388.65625}, {'score': 234893.96875}, {'score': 153787.375}, {'score': 173087.34375}, {'score': 222292.953125}, {'score': 217179.25}, {'score': 184164.6875}, {'score': 205866.875}, {'score': 135581.46875}, {'score': 212004.484375}, {'score': 247061.0}, {'score': 188965.34375}, {'score': 237525.96875}, {'score': 138593.09375}, {'score': 254665.21875}, {'score': 111836.8671875}, {'score': 204874.96875}, {'score': 194102.6875}, {'score': 187919.359375}, {'score': 241848.09375}, {'score': 257951.34375}, {'score': 169278.59375}, {'score': 174490.78125}, {'score': 180140.3125}, {'score': 207134.75}, {'score': 230614.375}, {'score': 197965.875}, {'score': 185639.265625}, {'score': 196051.3125}, {'score': 126670.6171875}, {'score': 177793.3125}, {'score': 183047.46875}, {'score': 205533.15625}, {'score': 157166.1875}, {'score': 217581.5625}, {'score': 168870.03125}, {'score': 161272.8125}, {'score': 180458.875}, {'score': 174860.09375}, {'score': 287754.65625}, {'score': 226038.4375}, {'score': 203117.28125}, {'score': 248820.6875}, {'score': 161661.53125}, {'score': 130996.2265625}, {'score': 209169.3125}, {'score': 182802.3125}, {'score': 202377.09375}, {'score': 217184.46875}, {'score': 153049.59375}, {'score': 170653.328125}, {'score': 183850.734375}, {'score': 280525.03125}, {'score': 179967.71875}, {'score': 211728.34375}, {'score': 190743.3125}, {'score': 178012.03125}, {'score': 129440.1484375}, {'score': 187029.375}, {'score': 162130.59375}, {'score': 159921.84375}, {'score': 162954.03125}, {'score': 281095.53125}, {'score': 187383.28125}, {'score': 218900.5}, {'score': 143706.375}, {'score': 190491.125}, {'score': 190513.21875}, {'score': 195303.0625}, {'score': 190481.4375}, {'score': 198863.28125}, {'score': 154134.3125}, {'score': 211625.6875}, {'score': 227351.5}, {'score': 238479.40625}, {'score': 178504.625}, {'score': 194580.75}, {'score': 169180.46875}, {'score': 161173.328125}, {'score': 198976.34375}, {'score': 197246.78125}, {'score': 242293.53125}, {'score': 163609.0}, {'score': 159478.9375}, {'score': 194419.9375}, {'score': 197097.875}, {'score': 174816.21875}, {'score': 213467.6875}, {'score': 191173.75}, {'score': 209065.203125}, {'score': 151851.09375}, {'score': 203410.09375}, {'score': 158202.875}, {'score': 149086.796875}, {'score': 222114.375}, {'score': 196472.78125}, {'score': 124222.1171875}, {'score': 194465.125}, {'score': 166162.96875}, {'score': 201413.8125}, {'score': 174609.0625}, {'score': 183084.46875}, {'score': 147137.75}, {'score': 206342.375}, {'score': 160607.78125}, {'score': 152682.15625}, {'score': 166691.09375}, {'score': 199892.96875}, {'score': 171941.671875}, {'score': 193446.6875}, {'score': 153234.6875}, {'score': 220707.984375}, {'score': 167474.828125}, {'score': 231890.203125}, {'score': 163063.90625}, {'score': 274330.9375}, {'score': 220214.15625}, {'score': 233280.34375}, {'score': 232453.9375}, {'score': 220929.03125}, {'score': 176363.46875}, {'score': 183988.390625}, {'score': 198709.6875}, {'score': 165101.6875}, {'score': 203510.375}, {'score': 152626.453125}, {'score': 175840.40625}, {'score': 295258.9375}, {'score': 207578.859375}, {'score': 200503.40625}, {'score': 175049.1875}, {'score': 254704.5}, {'score': 184390.96875}, {'score': 183812.84375}, {'score': 146293.3125}, {'score': 226334.3125}, {'score': 170956.59375}, {'score': 216149.40625}, {'score': 268497.25}, {'score': 216066.21875}, {'score': 190506.40625}, {'score': 248631.15625}, {'score': 185487.0}, {'score': 165250.625}, {'score': 188108.03125}, {'score': 184244.40625}, {'score': 302556.75}, {'score': 172031.03125}, {'score': 207955.109375}, {'score': 207778.9375}, {'score': 179479.40625}, {'score': 204173.125}, {'score': 172624.234375}, {'score': 424562.9375}, {'score': 217258.265625}, {'score': 147127.96875}, {'score': 200594.234375}, {'score': 206433.78125}, {'score': 149930.078125}, {'score': 209405.0625}, {'score': 176042.1875}, {'score': 222403.125}, {'score': 204718.109375}, {'score': 157847.875}, {'score': 89407.4921875}, {'score': 133951.25}, {'score': 184042.9375}, {'score': 185715.75}, {'score': 191634.828125}, {'score': 160640.125}, {'score': 170389.375}, {'score': 146847.09375}, {'score': 144785.40625}, {'score': 192329.78125}, {'score': 177221.53125}, {'score': 322741.75}, {'score': 90306.0}, {'score': 194728.046875}, {'score': 178453.5625}, {'score': 217883.53125}, {'score': 224507.25}, {'score': 147239.90625}, {'score': 152293.859375}, {'score': 166536.84375}, {'score': 186287.546875}, {'score': 193336.875}, {'score': 159079.59375}, {'score': 205213.6875}, {'score': 172377.15625}, {'score': 159373.28125}, {'score': 164634.375}, {'score': 236700.125}, {'score': 205090.28125}, {'score': 207555.21875}, {'score': 214214.875}, {'score': 177344.21875}, {'score': 227722.78125}, {'score': 165060.4375}, {'score': 215041.5625}, {'score': 201302.46875}, {'score': 245255.0625}, {'score': 220206.4375}, {'score': 181200.96875}, {'score': 226293.5625}, {'score': 222305.03125}, {'score': 199196.6875}, {'score': 163647.40625}, {'score': 175868.1875}, {'score': 156501.1875}, {'score': 205471.96875}, {'score': 157020.875}, {'score': 137365.65625}, {'score': 134858.8125}, {'score': 175841.90625}, {'score': 122879.6796875}, {'score': 186780.484375}, {'score': 238524.40625}, {'score': 196950.375}, {'score': 174016.09375}, {'score': 211551.0}, {'score': 220944.40625}, {'score': 139527.3125}, {'score': 194564.609375}, {'score': 175120.875}, {'score': 185851.421875}, {'score': 187089.09375}, {'score': 225862.0}, {'score': 146413.90625}, {'score': 157643.625}, {'score': 280237.65625}, {'score': 229083.90625}, {'score': 197398.0625}, {'score': 189930.59375}, {'score': 187211.375}, {'score': 123178.0859375}, {'score': 183579.46875}, {'score': 190756.4375}, {'score': 152826.34375}, {'score': 195344.4375}, {'score': 153882.46875}, {'score': 163780.125}, {'score': 166664.75}, {'score': 166803.53125}, {'score': 155206.75}, {'score': 201699.078125}, {'score': 167461.328125}, {'score': 211169.84375}, {'score': 172442.78125}, {'score': 218956.5}, {'score': 215932.15625}, {'score': 180171.65625}, {'score': 210706.25}, {'score': 137496.15625}, {'score': 210434.84375}, {'score': 194598.765625}, {'score': 227500.8125}, {'score': 208531.265625}, {'score': 248654.78125}, {'score': 326255.375}, {'score': 224430.640625}, {'score': 146537.375}, {'score': 183156.90625}, {'score': 146853.890625}, {'score': 163527.015625}, {'score': 205483.125}, {'score': 205231.9375}, {'score': 145715.4375}, {'score': 223226.28125}, {'score': 165000.59375}, {'score': 166003.4375}, {'score': 169569.40625}, {'score': 181664.0625}, {'score': 128928.2265625}, {'score': 171996.53125}, {'score': 156123.65625}, {'score': 150539.375}, {'score': 174817.03125}, {'score': 226675.03125}, {'score': 167125.875}, {'score': 188374.0625}, {'score': 132484.59375}, {'score': 186977.90625}, {'score': 200632.03125}, {'score': 207957.1875}, {'score': 168857.734375}, {'score': 195062.03125}, {'score': 196303.15625}, {'score': 148987.578125}, {'score': 178674.375}, {'score': 192177.25}, {'score': 180702.703125}, {'score': 194137.15625}, {'score': 202514.53125}, {'score': 124929.7734375}, {'score': 148527.40625}, {'score': 192965.609375}, {'score': 389456.0625}, {'score': 196360.84375}, {'score': 198683.34375}, {'score': 179110.921875}, {'score': 199358.875}, {'score': 212716.6875}, {'score': 176577.484375}, {'score': 194355.109375}, {'score': 175325.4375}, {'score': 209702.46875}, {'score': 176495.1875}, {'score': 186059.71875}, {'score': 167880.078125}, {'score': 214472.90625}, {'score': 185254.875}, {'score': 207116.78125}, {'score': 82215.3828125}, {'score': 177839.40625}, {'score': 199489.6875}, {'score': 185599.5625}, {'score': 195763.609375}, {'score': 155243.59375}, {'score': 199709.375}, {'score': 193199.875}, {'score': 254910.46875}, {'score': 210470.0625}, {'score': 123036.0078125}, {'score': 221141.6875}, {'score': 159774.03125}, {'score': 177569.6875}, {'score': 190662.125}, {'score': 201672.84375}, {'score': 179972.5625}, {'score': 151416.5}, {'score': 208536.0}, {'score': 130952.0234375}, {'score': 203753.78125}, {'score': 147361.34375}, {'score': 172572.5}, {'score': 174109.90625}, {'score': 218124.140625}, {'score': 212907.71875}, {'score': 212895.1875}, {'score': 201164.453125}, {'score': 227805.125}, {'score': 209232.671875}, {'score': 264534.0}, {'score': 189044.5625}, {'score': 317698.78125}, {'score': 212239.953125}, {'score': 172995.3125}, {'score': 192924.8125}, {'score': 208910.0625}, {'score': 132367.21875}, {'score': 190853.21875}, {'score': 242595.421875}, {'score': 192760.46875}, {'score': 129182.8828125}, {'score': 202035.28125}, {'score': 263025.0625}, {'score': 145937.84375}, {'score': 169746.078125}, {'score': 234687.25}, {'score': 162698.8125}, {'score': 182981.3125}, {'score': 157223.4375}, {'score': 278298.5625}, {'score': 231936.84375}, {'score': 149420.953125}, {'score': 189263.28125}, {'score': 164169.5625}, {'score': 147891.5}, {'score': 183711.78125}, {'score': 240407.21875}, {'score': 224025.34375}, {'score': 189509.140625}, {'score': 203870.71875}, {'score': 198682.28125}, {'score': 187825.21875}, {'score': 193526.453125}, {'score': 189687.46875}, {'score': 143568.9375}, {'score': 169257.40625}, {'score': 157853.0}, {'score': 182058.6875}, {'score': 150785.34375}, {'score': 165521.953125}, {'score': 179977.8125}, {'score': 158919.375}, {'score': 181303.59375}, {'score': 200666.4375}, {'score': 175202.09375}, {'score': 201620.234375}, {'score': 127745.9453125}, {'score': 178276.5}, {'score': 203055.59375}, {'score': 202376.25}, {'score': 178579.625}, {'score': 179466.0625}, {'score': 185473.984375}, {'score': 185741.1875}, {'score': 211893.59375}, {'score': 125692.3046875}, {'score': 230176.75}, {'score': 179251.375}, {'score': 151764.03125}, {'score': 109910.7734375}, {'score': 203413.75}, {'score': 93737.9765625}, {'score': 197694.28125}, {'score': 196659.34375}, {'score': 204776.0625}, {'score': 198158.546875}, {'score': 152830.46875}, {'score': 205078.15625}, {'score': 163146.265625}, {'score': 181952.375}, {'score': 185165.6875}, {'score': 161528.625}, {'score': 204423.25}, {'score': 243188.09375}, {'score': 179850.46875}, {'score': 194204.3125}, {'score': 186347.125}, {'score': 163488.59375}, {'score': 140566.59375}, {'score': 184278.71875}, {'score': 180564.875}, {'score': 164738.265625}, {'score': 206396.875}, {'score': 208563.15625}, {'score': 209850.78125}, {'score': 135121.03125}, {'score': 197089.875}, {'score': 164822.609375}, {'score': 205822.125}, {'score': 202107.40625}, {'score': 166537.8125}, {'score': 535420.75}, {'score': 133423.5625}, {'score': 181037.03125}, {'score': 174232.59375}, {'score': 153964.40625}, {'score': 198305.3125}, {'score': 163025.71875}, {'score': 181537.25}, {'score': 215717.0625}, {'score': 204046.4375}, {'score': 199219.0625}, {'score': 190153.34375}, {'score': 193246.0}, {'score': 219962.1875}, {'score': 195650.234375}, {'score': 224017.28125}, {'score': 180376.203125}, {'score': 142768.15625}, {'score': 129718.6640625}, {'score': 153920.65625}, {'score': 257260.6875}, {'score': 154374.15625}, {'score': 183325.0}, {'score': 216941.75}, {'score': 204924.90625}, {'score': 196312.53125}, {'score': 266524.3125}, {'score': 185178.9375}, {'score': 173958.84375}, {'score': 193687.375}, {'score': 241393.59375}, {'score': 159856.65625}, {'score': 173024.71875}, {'score': 126944.8515625}, {'score': 179387.78125}, {'score': 153680.0625}, {'score': 220943.5625}, {'score': 207924.6875}, {'score': 223027.765625}, {'score': 238815.5}, {'score': 205928.03125}, {'score': 189057.375}, {'score': 228799.25}, {'score': 264647.5}, {'score': 167691.53125}, {'score': 230124.546875}, {'score': 199216.65625}, {'score': 232340.6875}, {'score': 197128.171875}, {'score': 207572.28125}, {'score': 172157.59375}, {'score': 145883.96875}, {'score': 199308.09375}, {'score': 175947.03125}, {'score': 186371.875}, {'score': 202106.65625}, {'score': 153286.96875}, {'score': 163111.9375}, {'score': 168839.9375}, {'score': 170344.96875}, {'score': 162156.453125}, {'score': 201225.3125}, {'score': 168752.421875}, {'score': 192209.875}, {'score': 156478.5625}, {'score': 202147.25}, {'score': 733064.6875}, {'score': 188833.984375}, {'score': 112408.1328125}, {'score': 193340.015625}, {'score': 140818.65625}, {'score': 180700.9375}, {'score': 223833.40625}, {'score': 219601.984375}, {'score': 137857.03125}, {'score': 229448.9375}, {'score': 191616.984375}, {'score': 225491.8125}, {'score': 174523.484375}, {'score': 165120.65625}, {'score': 118957.3046875}, {'score': 170619.53125}, {'score': 125416.0078125}, {'score': 180501.75}, {'score': 208412.734375}, {'score': 240280.875}, {'score': 202955.9375}, {'score': 181868.46875}, {'score': 162688.15625}, {'score': 170341.34375}, {'score': 173659.5}, {'score': 182579.875}, {'score': 184897.96875}, {'score': 185294.875}, {'score': 200846.65625}, {'score': 201829.25}, {'score': 107402.1015625}, {'score': 208366.578125}, {'score': 146105.25}, {'score': 174250.8125}, {'score': 175065.15625}, {'score': 219709.71875}, {'score': 132519.625}, {'score': 193878.34375}, {'score': 171608.0}, {'score': 203679.3125}, {'score': 201838.03125}, {'score': 197184.03125}, {'score': 146290.0}, {'score': 234537.1875}, {'score': 176146.03125}, {'score': 183502.59375}, {'score': 133585.71875}, {'score': 173449.578125}, {'score': 154256.59375}, {'score': 255608.25}, {'score': 183435.5}, {'score': 190508.625}, {'score': 182693.53125}, {'score': 196153.75}, {'score': 224712.5625}, {'score': 186219.84375}, {'score': 252541.09375}, {'score': 174264.15625}, {'score': 227116.125}, {'score': 216669.125}, {'score': 177499.671875}, {'score': 165161.625}, {'score': 143552.65625}, {'score': 181370.34375}, {'score': 122559.6328125}, {'score': 183649.21875}, {'score': 166422.875}, {'score': 228993.40625}, {'score': 164585.15625}, {'score': 128262.3671875}, {'score': 156106.21875}, {'score': 247217.96875}, {'score': 172325.40625}, {'score': 143542.328125}, {'score': 251138.875}, {'score': 162178.0}, {'score': 177284.28125}, {'score': 152340.90625}, {'score': 169073.84375}, {'score': 165486.75}, {'score': 225287.1875}, {'score': 150155.5}, {'score': 187527.46875}, {'score': 192293.625}, {'score': 173280.4375}, {'score': 150592.96875}, {'score': 216146.09375}, {'score': 164954.109375}, {'score': 189856.59375}, {'score': 293353.0}, {'score': 254183.4375}, {'score': 203718.34375}, {'score': 227703.28125}, {'score': 169159.34375}, {'score': 187230.0625}, {'score': 190364.375}, {'score': 219107.3125}, {'score': 178659.90625}, {'score': 157672.46875}, {'score': 203887.0}, {'score': 190828.578125}, {'score': 192484.46875}, {'score': 164125.53125}, {'score': 140591.5}, {'score': 212995.625}, {'score': 189637.46875}, {'score': 204316.8125}, {'score': 258031.75}, {'score': 141313.21875}, {'score': 217155.59375}, {'score': 279401.40625}, {'score': 194219.078125}, {'score': 208920.40625}, {'score': 155904.265625}, {'score': 181934.5625}, {'score': 135970.46875}, {'score': 229161.140625}, {'score': 218854.234375}, {'score': 190940.453125}, {'score': 211039.28125}, {'score': 211938.4375}, {'score': 245979.90625}, {'score': 179392.0625}, {'score': 123316.4296875}, {'score': 161231.46875}, {'score': 177645.15625}, {'score': 77684.8359375}, {'score': 190904.09375}, {'score': 204242.28125}, {'score': 168254.546875}, {'score': 375248.90625}, {'score': 383924.53125}, {'score': 235971.28125}, {'score': 219112.75}, {'score': 214023.09375}, {'score': 207413.90625}, {'score': 200221.0}, {'score': 193073.65625}, {'score': 211757.03125}, {'score': 170047.015625}, {'score': 228317.125}, {'score': 147300.125}, {'score': 258492.71875}, {'score': 219807.3125}, {'score': 209016.21875}, {'score': 211279.875}, {'score': 185409.96875}, {'score': 157165.65625}, {'score': 208955.53125}, {'score': 123174.1015625}, {'score': 182591.0}, {'score': 176195.4375}, {'score': 163442.578125}, {'score': 193108.9375}, {'score': 218722.03125}, {'score': 158516.28125}, {'score': 175967.5625}, {'score': 205952.15625}, {'score': 210436.90625}, {'score': 234187.890625}, {'score': 202682.6875}, {'score': 151392.78125}, {'score': 149917.75}, {'score': 231660.28125}, {'score': 259404.09375}, {'score': 192972.28125}, {'score': 168987.8125}, {'score': 250829.3125}, {'score': 147111.859375}, {'score': 219727.265625}, {'score': 176710.6875}, {'score': 239940.46875}, {'score': 183924.25}, {'score': 201548.3125}, {'score': 212448.859375}, {'score': 222646.4375}, {'score': 220949.921875}, {'score': 245526.78125}, {'score': 92895.7421875}, {'score': 156021.71875}, {'score': 175504.90625}, {'score': 422891.25}, {'score': 202900.1875}, {'score': 202111.375}, {'score': 219905.234375}, {'score': 188407.390625}, {'score': 164851.21875}, {'score': 216722.484375}, {'score': 183841.5}, {'score': 220460.3125}, {'score': 156941.28125}, {'score': 173239.53125}, {'score': 217537.1875}, {'score': 206534.21875}, {'score': 182713.46875}, {'score': 151630.046875}, {'score': 227154.5625}, {'score': 209020.6875}, {'score': 165512.78125}, {'score': 152693.15625}, {'score': 165776.0}, {'score': 125229.3359375}, {'score': 176083.921875}, {'score': 218619.21875}, {'score': 174165.28125}, {'score': 182246.875}, {'score': 187601.234375}, {'score': 150473.9375}, {'score': 226348.09375}, {'score': 148735.40625}, {'score': 196355.28125}, {'score': 171011.28125}, {'score': 197736.40625}, {'score': 170359.78125}, {'score': 168239.71875}, {'score': 164052.046875}, {'score': 235879.390625}, {'score': 211511.78125}, {'score': 211421.75}, {'score': 201653.25}, {'score': 200648.78125}, {'score': 175273.03125}, {'score': 154272.21875}, {'score': 162364.265625}, {'score': 198019.75}, {'score': 163689.1875}, {'score': 174931.0}, {'score': 218156.875}, {'score': 177352.453125}, {'score': 240364.03125}, {'score': 181370.59375}, {'score': 231024.15625}, {'score': 186551.375}, {'score': 253629.21875}, {'score': 187349.15625}, {'score': 205457.6875}, {'score': 174203.34375}, {'score': 188800.4375}, {'score': 165376.1875}, {'score': 237599.75}, {'score': 170353.6875}, {'score': 157130.84375}, {'score': 222909.6875}, {'score': 150772.875}, {'score': 164961.296875}, {'score': 209808.71875}, {'score': 194580.78125}, {'score': 202897.5625}, {'score': 182411.9375}, {'score': 210308.34375}, {'score': 220292.1875}, {'score': 173475.359375}, {'score': 184156.8125}, {'score': 180882.78125}, {'score': 196982.9375}, {'score': 152281.53125}, {'score': 191863.25}, {'score': 157002.21875}, {'score': 204916.25}, {'score': 149809.390625}, {'score': 148737.0}, {'score': 154043.96875}, {'score': 189505.265625}, {'score': 196742.46875}, {'score': 172877.96875}, {'score': 194011.46875}, {'score': 205589.8125}, {'score': 106359.7109375}, {'score': 173067.71875}, {'score': 204778.78125}, {'score': 166142.34375}, {'score': 197202.15625}, {'score': 134089.5625}, {'score': 185562.0625}, {'score': 201474.765625}, {'score': 221611.8125}, {'score': 158418.5625}, {'score': 214747.546875}, {'score': 203785.625}, {'score': 188933.6875}, {'score': 186936.65625}, {'score': 148961.421875}, {'score': 178610.90625}, {'score': 191545.671875}, {'score': 199806.25}, {'score': 204763.515625}, {'score': 193708.171875}, {'score': 204118.921875}, {'score': 140083.90625}, {'score': 297426.5}, {'score': 242166.71875}, {'score': 170949.9375}, {'score': 180827.84375}, {'score': 178929.75}, {'score': 202322.59375}, {'score': 223232.5}, {'score': 210341.5}, {'score': 159702.28125}, {'score': 212469.84375}, {'score': 168623.125}, {'score': 199391.625}, {'score': 208409.15625}, {'score': 204192.125}, {'score': 186328.25}, {'score': 207209.15625}, {'score': 232625.375}, {'score': 212377.21875}, {'score': 196428.4375}, {'score': 187905.40625}, {'score': 157161.71875}, {'score': 257427.65625}, {'score': 191200.875}, {'score': 186449.5}, {'score': 160648.5625}, {'score': 223664.84375}, {'score': 126764.03125}, {'score': 244251.15625}, {'score': 214900.03125}, {'score': 200483.328125}, {'score': 177089.140625}, {'score': 270360.8125}, {'score': 294460.78125}, {'score': 201206.9375}, {'score': 191561.59375}, {'score': 170881.1875}, {'score': 174595.390625}, {'score': 175694.5}, {'score': 198031.34375}, {'score': 245835.1875}, {'score': 193703.0}, {'score': 157429.765625}, {'score': 218853.875}, {'score': 245954.6875}, {'score': 390355.75}, {'score': 216159.75}, {'score': 167915.46875}, {'score': 162115.84375}, {'score': 186967.25}, {'score': 185880.84375}, {'score': 167556.875}, {'score': 179626.15625}, {'score': 220123.328125}, {'score': 191100.71875}, {'score': 201374.6875}, {'score': 162604.5}, {'score': 99874.40625}, {'score': 177205.953125}, {'score': 179469.84375}, {'score': 192114.875}, {'score': 153167.0}, {'score': 203902.84375}, {'score': 167824.53125}, {'score': 196560.46875}, {'score': 198247.0625}, {'score': 204724.46875}, {'score': 190688.40625}, {'score': 182170.0}, {'score': 208886.390625}, {'score': 216744.46875}, {'score': 204756.0625}, {'score': 232593.875}, {'score': 256098.0}, {'score': 182906.25}, {'score': 208498.6875}, {'score': 212705.71875}, {'score': 191144.53125}, {'score': 193539.53125}, {'score': 200703.34375}, {'score': 185720.25}, {'score': 221952.6875}, {'score': 231489.0625}, {'score': 184025.46875}, {'score': 224529.0}, {'score': 161486.25}, {'score': 253813.6875}, {'score': 156372.0625}, {'score': 114019.3515625}, {'score': 153311.65625}, {'score': 175356.3125}, {'score': 168539.78125}, {'score': 143068.328125}, {'score': 205723.34375}, {'score': 183236.1875}, {'score': 162785.5625}, {'score': 354894.40625}, {'score': 190742.203125}, {'score': 253948.25}, {'score': 210088.5625}, {'score': 201054.140625}, {'score': 232385.84375}, {'score': 208255.625}, {'score': 296902.8125}, {'score': 222836.1875}, {'score': 146121.203125}, {'score': 197832.78125}, {'score': 191129.234375}, {'score': 173127.15625}, {'score': 201058.1875}, {'score': 173428.09375}, {'score': 201384.28125}, {'score': 204031.09375}, {'score': 152485.53125}, {'score': 151980.0625}, {'score': 171603.375}, {'score': 178118.4375}, {'score': 221157.828125}, {'score': 217059.484375}, {'score': 170212.265625}, {'score': 172692.015625}, {'score': 235691.1875}, {'score': 128948.9140625}, {'score': 214736.375}, {'score': 188686.65625}, {'score': 182001.34375}, {'score': 192982.453125}, {'score': 201336.3125}, {'score': 182571.09375}, {'score': 164041.9375}, {'score': 208757.90625}, {'score': 204830.03125}, {'score': 155096.84375}, {'score': 174808.0}, {'score': 182847.375}, {'score': 227143.71875}, {'score': 190924.796875}, {'score': 169237.90625}, {'score': 164317.25}, {'score': 153597.1875}, {'score': 206160.578125}, {'score': 201647.1875}, {'score': 203787.359375}, {'score': 211132.3125}, {'score': 236645.8125}, {'score': 225857.09375}, {'score': 197333.65625}, {'score': 190003.3125}, {'score': 161903.875}, {'score': 121313.8046875}, {'score': 140605.125}, {'score': 211134.703125}, {'score': 164670.796875}, {'score': 178031.53125}, {'score': 229788.296875}, {'score': 209376.546875}, {'score': 169227.71875}, {'score': 194447.625}, {'score': 180445.8125}, {'score': 195676.578125}, {'score': 167177.625}, {'score': 195068.78125}, {'score': 219671.6875}, {'score': 225375.8125}, {'score': 233341.484375}, {'score': 167987.28125}, {'score': 170988.078125}, {'score': 186286.875}, {'score': 367841.8125}, {'score': 167267.59375}, {'score': 209247.34375}, {'score': 174779.96875}, {'score': 167684.78125}, {'score': 130688.8515625}, {'score': 196565.21875}, {'score': 180775.1875}, {'score': 173629.34375}, {'score': 162952.5}, {'score': 131862.125}, {'score': 218319.3125}, {'score': 174363.84375}, {'score': 207185.84375}, {'score': 182917.640625}, {'score': 212633.375}, {'score': 233209.65625}, {'score': 177244.5}, {'score': 167403.328125}, {'score': 131174.5}, {'score': 231530.578125}, {'score': 185702.90625}, {'score': 214808.78125}, {'score': 198528.6875}, {'score': 194584.09375}, {'score': 188357.65625}, {'score': 225148.640625}, {'score': 213486.125}, {'score': 169190.375}, {'score': 140951.15625}, {'score': 210728.5625}, {'score': 181886.90625}, {'score': 135412.25}, {'score': 115138.6171875}, {'score': 216800.34375}, {'score': 175362.375}, {'score': 208421.65625}, {'score': 183420.59375}, {'score': 158701.6875}, {'score': 173362.8125}, {'score': 207282.21875}, {'score': 164958.75}, {'score': 248857.0}, {'score': 149386.96875}, {'score': 182533.4375}, {'score': 227702.09375}, {'score': 186851.09375}, {'score': 263168.28125}, {'score': 192906.078125}, {'score': 168308.9375}, {'score': 203467.21875}, {'score': 171486.328125}, {'score': 192628.84375}, {'score': 163868.40625}, {'score': 234393.59375}, {'score': 171397.984375}, {'score': 197381.796875}, {'score': 51033.8203125}, {'score': 192570.71875}, {'score': 214024.125}, {'score': 205928.578125}, {'score': 212575.15625}, {'score': 251146.125}, {'score': 208720.28125}, {'score': 214369.6875}, {'score': 210061.28125}, {'score': 192526.71875}, {'score': 204149.59375}, {'score': 216867.15625}, {'score': 192893.5625}, {'score': 193032.90625}, {'score': 171087.0625}, {'score': 210487.0}, {'score': 206062.4375}, {'score': 200130.78125}, {'score': 175603.09375}, {'score': 179246.6875}, {'score': 150750.0}, {'score': 247658.09375}, {'score': 178782.234375}, {'score': 190034.5}, {'score': 21048.9296875}, {'score': 186748.40625}, {'score': 154227.78125}, {'score': 138413.875}, {'score': 163320.65625}, {'score': 262155.9375}, {'score': 176734.234375}, {'score': 210356.25}, {'score': 168602.796875}, {'score': 187439.65625}, {'score': 170248.90625}, {'score': 161311.9375}, {'score': 173450.796875}, {'score': 174825.09375}, {'score': 181937.6875}, {'score': 185072.625}, {'score': 211500.375}, {'score': 198892.828125}, {'score': 195197.46875}, {'score': 208073.65625}, {'score': 203580.6875}, {'score': 166955.0625}, {'score': 257907.5}, {'score': 199401.09375}, {'score': 117060.7421875}, {'score': 184608.9375}, {'score': 172744.0}, {'score': 210378.40625}, {'score': 155305.40625}, {'score': 176558.9375}, {'score': 211014.21875}, {'score': 262469.78125}, {'score': 165182.546875}, {'score': 205333.46875}, {'score': 235493.0625}, {'score': 156777.5}, {'score': 149674.96875}, {'score': 206952.8125}, {'score': 222752.90625}, {'score': 182198.921875}, {'score': 182855.03125}, {'score': 168965.09375}, {'score': 169157.8125}, {'score': 193934.59375}, {'score': 168498.828125}, {'score': 80907.9296875}, {'score': 172057.53125}, {'score': 53402.6484375}, {'score': 195476.53125}, {'score': 184613.953125}, {'score': 185484.125}, {'score': 218012.5625}, {'score': 142288.96875}, {'score': 186640.0625}, {'score': 173029.796875}, {'score': 145311.25}, {'score': 211932.65625}, {'score': 212547.90625}, {'score': 202927.71875}, {'score': 165152.125}, {'score': 176899.0}, {'score': 211956.71875}, {'score': 221019.375}, {'score': 186769.15625}, {'score': 150814.4375}, {'score': 142531.4375}, {'score': 250815.5}, {'score': 211461.890625}, {'score': 155795.0}, {'score': 213119.5}, {'score': 193230.234375}, {'score': 163574.46875}, {'score': 216186.34375}, {'score': 194252.25}, {'score': 165847.453125}, {'score': 241593.09375}, {'score': 204766.8125}, {'score': 162875.96875}, {'score': 179190.578125}, {'score': 220724.5625}, {'score': 151271.4375}, {'score': 114209.4609375}, {'score': 197687.8125}, {'score': 225659.984375}, {'score': 191570.03125}, {'score': 319408.34375}, {'score': 190035.90625}, {'score': 188864.6875}, {'score': 201061.90625}, {'score': 196961.375}, {'score': 194117.53125}, {'score': 403851.75}, {'score': 188144.1875}, {'score': 227393.0625}, {'score': 198474.78125}, {'score': 163457.4375}, {'score': 151200.703125}, {'score': 227942.6875}, {'score': 229852.0625}, {'score': 164869.5625}, {'score': 170951.71875}, {'score': 196062.1875}, {'score': 218420.9375}, {'score': 186938.03125}, {'score': 152608.328125}, {'score': 207418.25}, {'score': 209777.203125}, {'score': 164514.046875}, {'score': 163654.953125}, {'score': 220836.9375}, {'score': 120147.2265625}, {'score': 236801.6875}, {'score': 213659.0}, {'score': 160965.625}, {'score': 239271.4375}, {'score': 192629.78125}, {'score': 238227.6875}, {'score': 146377.8125}, {'score': 173457.15625}, {'score': 214889.734375}, {'score': 222091.875}, {'score': 243145.8125}, {'score': 215037.09375}, {'score': 186963.703125}, {'score': 235821.0}, {'score': 226454.40625}, {'score': 171724.25}, {'score': 203319.265625}, {'score': 122500.1015625}, {'score': 235692.6875}, {'score': 80898.5703125}, {'score': 194852.03125}, {'score': 247268.28125}, {'score': 198018.25}, {'score': 192540.375}, {'score': 207392.03125}, {'score': 191732.09375}, {'score': 188145.90625}, {'score': 138070.25}, {'score': 215733.71875}, {'score': 172155.53125}, {'score': 196843.09375}, {'score': 193632.890625}, {'score': 159275.40625}, {'score': 223035.96875}, {'score': 269438.46875}, {'score': 171088.4375}, {'score': 141474.15625}, {'score': 201553.875}, {'score': 241035.78125}, {'score': 203758.625}, {'score': 224495.375}, {'score': 145050.4375}, {'score': 218754.125}, {'score': 139902.25}, {'score': 218221.34375}, {'score': 181057.78125}, {'score': 182640.359375}, {'score': 200284.3125}, {'score': 160487.46875}, {'score': 131169.90625}, {'score': 174611.71875}, {'score': 228886.53125}, {'score': 192308.1875}, {'score': 181718.65625}, {'score': 204008.90625}, {'score': 236623.0}, {'score': 196318.28125}, {'score': 162449.4375}, {'score': 138205.59375}, {'score': 187124.8125}, {'score': 241203.75}, {'score': 192076.0625}, {'score': 156952.03125}, {'score': 176146.046875}, {'score': 197452.375}, {'score': 606955.9375}, {'score': 183570.375}, {'score': 136573.625}, {'score': 215187.21875}, {'score': 168465.0}, {'score': 196851.375}, {'score': 189631.28125}, {'score': 248137.71875}, {'score': 157868.265625}, {'score': 158106.5}, {'score': 175667.96875}, {'score': 194851.21875}, {'score': 179612.46875}, {'score': 250694.25}, {'score': 177666.25}, {'score': 185544.5}, {'score': 217871.75}, {'score': 156846.5}, {'score': 167584.328125}, {'score': 210630.015625}, {'score': 185176.828125}, {'score': 171081.25}, {'score': 205512.125}, {'score': 150152.03125}, {'score': 171419.4375}, {'score': 177467.375}, {'score': 206500.0625}, {'score': 158908.625}, {'score': 206299.625}, {'score': 250223.46875}, {'score': 192205.875}, {'score': 195118.34375}, {'score': 153031.6875}, {'score': 201440.609375}, {'score': 217434.859375}, {'score': 182457.28125}, {'score': 158282.25}, {'score': 186549.265625}, {'score': 180123.09375}, {'score': 253978.09375}, {'score': 190379.28125}, {'score': 199059.9375}, {'score': 190664.8125}, {'score': 210231.84375}, {'score': 134739.90625}, {'score': 199677.984375}, {'score': 209812.625}, {'score': 142542.8125}, {'score': 243860.5625}, {'score': 172336.078125}, {'score': 235321.40625}, {'score': 170419.4375}, {'score': 193837.65625}, {'score': 194609.84375}, {'score': 252052.59375}, {'score': 178491.15625}, {'score': 159656.90625}, {'score': 130274.796875}, {'score': 161879.03125}, {'score': 186230.21875}, {'score': 38760.4921875}, {'score': 201354.34375}, {'score': 176418.484375}, {'score': 222924.359375}, {'score': 186734.84375}, {'score': 182922.1875}, {'score': 190392.234375}, {'score': 248602.03125}, {'score': 194673.90625}, {'score': 199947.53125}, {'score': 171727.15625}, {'score': 177872.0625}, {'score': 230118.53125}, {'score': 383516.09375}, {'score': 249770.71875}, {'score': 218212.96875}, {'score': 175412.34375}, {'score': 181781.34375}, {'score': 192796.90625}, {'score': 149233.328125}, {'score': 164148.5625}, {'score': 155140.96875}, {'score': 197119.90625}, {'score': 213785.625}, {'score': 161781.875}, {'score': 141732.5}, {'score': 180051.90625}, {'score': 184948.1875}, {'score': 211268.40625}, {'score': 205918.890625}, {'score': 187490.59375}, {'score': 199394.09375}, {'score': 116849.421875}, {'score': 156536.46875}, {'score': 216345.4375}, {'score': 166782.796875}, {'score': 184572.9375}, {'score': 208338.9375}, {'score': 190078.21875}, {'score': 194584.359375}, {'score': 211336.84375}, {'score': 167002.125}, {'score': 139459.0625}, {'score': 162413.4375}, {'score': 202750.65625}, {'score': 34570.5546875}, {'score': 159785.125}, {'score': 269692.4375}, {'score': 188152.5}, {'score': 198115.03125}, {'score': 156391.234375}, {'score': 166666.296875}, {'score': 150128.90625}, {'score': 134877.578125}, {'score': 235437.8125}, {'score': 194754.6875}, {'score': 183562.9375}, {'score': 194146.703125}, {'score': 222826.40625}, {'score': 154441.25}, {'score': 216504.828125}, {'score': 168210.1875}, {'score': 266937.5}, {'score': 162157.3125}, {'score': 224319.5}, {'score': 159647.75}, {'score': 190588.390625}, {'score': 239103.84375}, {'score': 149124.21875}, {'score': 191835.6875}, {'score': 192484.875}, {'score': 296892.96875}, {'score': 233221.46875}, {'score': 315309.9375}, {'score': 285609.53125}, {'score': 179345.046875}, {'score': 139374.28125}, {'score': 166386.203125}, {'score': 151661.40625}, {'score': 259254.21875}, {'score': 216932.3125}, {'score': 182250.3125}, {'score': 221966.5625}, {'score': 221662.90625}, {'score': 203371.59375}, {'score': 172258.09375}, {'score': 182257.65625}, {'score': 191533.265625}, {'score': 147021.5625}, {'score': 144870.6875}, {'score': 155133.15625}, {'score': 124900.765625}, {'score': 72930.453125}, {'score': 224093.1875}, {'score': 172846.5625}, {'score': 206716.703125}, {'score': 161224.15625}, {'score': 176935.046875}, {'score': 184136.71875}, {'score': 211350.6875}, {'score': 207397.375}, {'score': 139757.25}, {'score': 180821.5}, {'score': 169418.71875}, {'score': 157678.9375}, {'score': 226840.65625}, {'score': 221475.1875}, {'score': 173612.34375}, {'score': 182916.109375}, {'score': 238454.90625}, {'score': 177869.671875}, {'score': 211773.4375}, {'score': 194222.46875}, {'score': 205436.046875}, {'score': 177528.9375}, {'score': 171804.59375}, {'score': 204186.03125}, {'score': 216795.359375}, {'score': 215068.5625}, {'score': 215427.375}, {'score': 202742.3125}, {'score': 265831.4375}, {'score': 198117.375}, {'score': 133204.71875}, {'score': 197161.53125}, {'score': 190098.28125}, {'score': 209170.46875}, {'score': 280291.0625}, {'score': 157595.078125}, {'score': 191783.46875}, {'score': 157022.46875}, {'score': 233131.65625}, {'score': 130039.0078125}, {'score': 201700.40625}, {'score': 141531.328125}, {'score': 274122.6875}, {'score': 178311.625}, {'score': 225895.5625}, {'score': 168136.84375}, {'score': 166972.53125}, {'score': 172473.15625}, {'score': 216914.8125}, {'score': 206563.921875}, {'score': 189185.21875}, {'score': 235239.5}, {'score': 165860.046875}, {'score': 251562.96875}, {'score': 172508.5}, {'score': 147719.96875}, {'score': 217419.078125}, {'score': 273155.53125}, {'score': 153088.59375}, {'score': 186163.375}, {'score': 180032.640625}, {'score': 147332.8125}, {'score': 207499.03125}, {'score': 192715.53125}, {'score': 164179.375}, {'score': 171781.90625}, {'score': 158262.9375}, {'score': 219299.28125}, {'score': 252416.59375}, {'score': 147685.125}, {'score': 240894.4375}, {'score': 231690.5}, {'score': 117834.4140625}, {'score': 210277.59375}, {'score': 205874.90625}, {'score': 251657.875}, {'score': 147099.578125}, {'score': 161891.59375}, {'score': 187512.6875}, {'score': 173779.40625}, {'score': 191777.578125}, {'score': 152500.4375}, {'score': 194212.5625}, {'score': 183953.5625}, {'score': 173466.203125}, {'score': 195164.3125}, {'score': 213864.03125}, {'score': 211852.03125}, {'score': 163031.1875}, {'score': 200118.9375}, {'score': 193848.0625}, {'score': 185447.9375}, {'score': 215997.375}, {'score': 202430.4375}, {'score': 205499.6875}, {'score': 144922.703125}, {'score': 233201.53125}, {'score': 121199.3125}, {'score': 228762.09375}, {'score': 415570.6875}, {'score': 212713.6875}, {'score': 181204.8125}, {'score': 207999.0}, {'score': 176931.5625}, {'score': 223057.96875}, {'score': 202339.5}, {'score': 249698.09375}, {'score': 195578.40625}, {'score': 197867.28125}, {'score': 174769.515625}, {'score': 132547.71875}, {'score': 174822.3125}, {'score': 190997.46875}, {'score': 195190.0625}, {'score': 218270.96875}, {'score': 247151.8125}, {'score': 225565.828125}, {'score': 203885.21875}, {'score': 169897.46875}, {'score': 295310.40625}, {'score': 247430.15625}, {'score': 188512.46875}, {'score': 196867.78125}, {'score': 213190.765625}, {'score': 184317.875}, {'score': 204472.90625}, {'score': 212481.34375}, {'score': 207306.84375}, {'score': 228951.625}, {'score': 199531.046875}, {'score': 239188.515625}, {'score': 239668.5}, {'score': 253838.0}, {'score': 207158.546875}, {'score': 159277.90625}, {'score': 354855.9375}, {'score': 252447.1875}, {'score': 191995.546875}, {'score': 153847.4375}, {'score': 182467.859375}, {'score': 224992.03125}, {'score': 171897.453125}, {'score': 238111.40625}, {'score': 125899.9140625}, {'score': 196307.6875}, {'score': 175189.984375}, {'score': 165104.46875}, {'score': 244109.4375}, {'score': 213882.3125}, {'score': 196400.40625}, {'score': 178494.84375}, {'score': 206418.9375}, {'score': 211645.78125}, {'score': 213575.6875}, {'score': 172917.125}, {'score': 157643.875}, {'score': 221295.78125}, {'score': 147274.453125}, {'score': 190320.84375}, {'score': 186455.90625}, {'score': 175837.28125}, {'score': 221537.734375}, {'score': 227743.234375}, {'score': 171512.5}, {'score': 221493.796875}, {'score': 138312.125}, {'score': 163936.5625}, {'score': 169512.96875}, {'score': 219777.25}, {'score': 174239.03125}, {'score': 192529.875}, {'score': 164662.328125}, {'score': 228739.4375}, {'score': 156709.578125}, {'score': 209380.8125}, {'score': 180732.25}, {'score': 138950.0625}, {'score': 161491.53125}, {'score': 165443.015625}, {'score': 201601.609375}, {'score': 207247.5}, {'score': 202709.28125}, {'score': 212104.296875}, {'score': 150035.296875}, {'score': 163886.21875}, {'score': 154235.90625}, {'score': 169939.65625}, {'score': 183444.375}, {'score': 152977.5}, {'score': 176922.28125}, {'score': 196100.625}, {'score': 225454.796875}, {'score': 245091.03125}, {'score': 195960.234375}, {'score': 199885.40625}, {'score': 258949.625}, {'score': 211249.125}, {'score': 204967.71875}, {'score': 161931.90625}, {'score': 218961.09375}, {'score': 209592.78125}, {'score': 323103.75}, {'score': 218078.71875}, {'score': 232099.515625}, {'score': 241393.125}, {'score': 199746.21875}, {'score': 216948.890625}, {'score': 163098.375}, {'score': 179835.0625}, {'score': 174071.96875}, {'score': 210742.25}, {'score': 147845.8125}, {'score': 204948.21875}, {'score': 191663.5}, {'score': 216346.84375}, {'score': 263434.46875}, {'score': 230442.53125}, {'score': 168666.140625}, {'score': 106792.0390625}, {'score': 173038.875}, {'score': 198026.25}, {'score': 182973.984375}, {'score': 249721.625}, {'score': 226654.875}, {'score': 204290.1875}, {'score': 156464.375}, {'score': 182090.59375}, {'score': 185401.40625}, {'score': 216697.515625}, {'score': 116640.0078125}, {'score': 215583.5625}, {'score': 182529.375}, {'score': 167058.140625}, {'score': 173892.78125}, {'score': 178775.09375}, {'score': 218899.90625}, {'score': 207905.4375}, {'score': 148432.75}, {'score': 206099.09375}, {'score': 186430.890625}, {'score': 186075.0625}, {'score': 126073.7578125}, {'score': 176249.25}, {'score': 223365.53125}, {'score': 168631.796875}, {'score': 219083.625}, {'score': 274617.1875}, {'score': 90139.0546875}, {'score': 177205.4375}, {'score': 175607.46875}, {'score': 209888.28125}, {'score': 132145.0625}, {'score': 193606.5}, {'score': 187700.875}, {'score': 206052.625}, {'score': 151840.390625}, {'score': 233477.15625}, {'score': 280286.78125}, {'score': 181361.515625}, {'score': 168520.34375}, {'score': 486144.5625}, {'score': 167176.1875}, {'score': 198516.21875}, {'score': 223407.53125}, {'score': 156483.78125}, {'score': 187441.59375}, {'score': 260119.375}, {'score': 220031.859375}, {'score': 243728.9375}, {'score': 185005.53125}, {'score': 213778.8125}, {'score': 179206.96875}, {'score': 145164.0}, {'score': 222917.6875}, {'score': 217212.828125}, {'score': 215671.21875}, {'score': 183316.65625}, {'score': 237034.0625}, {'score': 196353.34375}, {'score': 167945.03125}, {'score': 166704.75}, {'score': 215061.75}, {'score': 218504.75}, {'score': 183309.4375}, {'score': 138965.875}, {'score': 201375.5}, {'score': 217744.5625}, {'score': 186505.75}, {'score': 213019.78125}, {'score': 233384.875}, {'score': 261849.875}, {'score': 153803.96875}, {'score': 201717.40625}, {'score': 187273.09375}, {'score': 135250.6875}, {'score': 140673.5625}, {'score': 189158.625}, {'score': 174253.65625}, {'score': 163249.5625}, {'score': 179472.71875}, {'score': 146194.125}, {'score': 189164.28125}, {'score': 156115.5}, {'score': 208848.046875}, {'score': 192365.34375}, {'score': 189943.3125}, {'score': 242536.75}, {'score': 283950.1875}, {'score': 213723.46875}, {'score': 182577.375}, {'score': 170056.375}, {'score': 221687.890625}, {'score': 157308.1875}, {'score': 215232.6875}, {'score': 222891.078125}, {'score': 287576.75}, {'score': 170254.8125}, {'score': 165679.9375}, {'score': 179210.59375}, {'score': 139959.0625}, {'score': 183867.734375}, {'score': 196073.3125}, {'score': 178811.40625}, {'score': 212220.8125}, {'score': 165038.9375}, {'score': 116567.09375}, {'score': 185923.109375}, {'score': 204906.46875}, {'score': 162860.8125}, {'score': 141229.03125}, {'score': 171040.1875}, {'score': 150822.21875}, {'score': 160605.46875}, {'score': 189573.65625}, {'score': 162568.1875}, {'score': 178648.171875}, {'score': 166715.5}, {'score': 234716.546875}, {'score': 223352.40625}, {'score': 142302.25}, {'score': 220257.0}, {'score': 266370.4375}, {'score': 168712.328125}, {'score': 193943.75}, {'score': 160788.125}, {'score': 183439.15625}, {'score': 180551.0}, {'score': 211181.921875}, {'score': 267998.0}, {'score': 173406.640625}, {'score': 205568.125}, {'score': 246787.21875}, {'score': 228979.59375}, {'score': 211194.3125}, {'score': 191684.0625}, {'score': 174976.3125}, {'score': 130608.1953125}, {'score': 201629.71875}, {'score': 205542.4375}, {'score': 215760.9375}, {'score': 179304.1875}, {'score': 205806.21875}, {'score': 177925.703125}, {'score': 175502.875}, {'score': 185533.0625}, {'score': 145902.59375}, {'score': 245876.5625}, {'score': 215839.34375}, {'score': 258960.96875}, {'score': 218355.5625}, {'score': 204600.375}, {'score': 271529.5625}, {'score': 194764.90625}, {'score': 208702.125}, {'score': 259521.5}, {'score': 205096.5}, {'score': 194310.3125}, {'score': 220317.75}, {'score': 127535.6953125}, {'score': 172838.125}, {'score': 168724.125}, {'score': 442866.0}, {'score': 199546.796875}, {'score': 114136.5078125}, {'score': 207083.78125}, {'score': 196867.6875}, {'score': 161616.46875}, {'score': 163791.90625}, {'score': 217089.09375}, {'score': 168433.71875}, {'score': 147834.984375}, {'score': 154205.59375}, {'score': 202124.8125}, {'score': 146601.59375}, {'score': 141595.328125}, {'score': 214711.59375}, {'score': 153135.3125}, {'score': 174803.765625}, {'score': 190717.59375}, {'score': 218287.21875}, {'score': 201136.796875}, {'score': 122453.7734375}, {'score': 187406.921875}, {'score': 196894.125}, {'score': 219086.28125}, {'score': 193441.53125}, {'score': 214047.71875}, {'score': 246342.03125}, {'score': 162937.125}, {'score': 210100.59375}, {'score': 228263.5625}, {'score': 172804.71875}, {'score': 196080.515625}, {'score': 215901.5}, {'score': 255424.71875}, {'score': 192374.65625}, {'score': 217019.375}, {'score': 223471.015625}, {'score': 228190.625}, {'score': 232498.53125}, {'score': 128266.0859375}, {'score': 181240.71875}, {'score': 199368.09375}, {'score': 205556.75}, {'score': 96456.2265625}, {'score': 148460.28125}, {'score': 197177.21875}, {'score': 193980.1875}, {'score': 168664.6875}, {'score': 179053.0625}, {'score': 161491.28125}, {'score': 195419.0}, {'score': 209991.6875}, {'score': 188225.96875}, {'score': 210104.5}, {'score': 185255.8125}, {'score': 178812.1875}, {'score': 183565.71875}, {'score': 157154.34375}, {'score': 181990.484375}, {'score': 177657.46875}, {'score': 201467.21875}, {'score': 186787.53125}, {'score': 186409.0625}, {'score': 213294.28125}, {'score': 191727.09375}, {'score': 150791.21875}, {'score': 203782.328125}, {'score': 179429.96875}, {'score': 135113.875}, {'score': 130935.9765625}, {'score': 139131.71875}, {'score': 162609.15625}, {'score': 149724.96875}, {'score': 170157.625}, {'score': 157315.6875}, {'score': 239361.875}, {'score': 196142.15625}, {'score': 185339.125}, {'score': 237275.5}, {'score': 210310.625}, {'score': 230633.65625}, {'score': 174834.5625}, {'score': 178942.046875}, {'score': 158269.625}, {'score': 187144.5625}, {'score': 143896.34375}, {'score': 196302.078125}, {'score': 179088.328125}, {'score': 218130.75}, {'score': 188440.15625}, {'score': 178705.96875}, {'score': 182555.609375}, {'score': 169447.71875}, {'score': 180890.40625}, {'score': 208289.625}, {'score': 163651.15625}, {'score': 207457.21875}, {'score': 230993.53125}, {'score': 157764.375}, {'score': 178083.484375}, {'score': 126096.5234375}, {'score': 195022.984375}, {'score': 194195.46875}, {'score': 156019.703125}, {'score': 200411.375}, {'score': 202786.40625}, {'score': 209582.84375}, {'score': 259404.65625}, {'score': 218822.375}, {'score': 162472.34375}, {'score': 151331.9375}, {'score': 228760.90625}, {'score': 191987.625}, {'score': 197088.03125}, {'score': 240631.21875}, {'score': 206094.796875}, {'score': 228549.71875}, {'score': 238501.0625}, {'score': 184489.5}, {'score': 222922.21875}, {'score': 112488.9921875}, {'score': 254855.9375}, {'score': 168869.15625}, {'score': 186052.59375}, {'score': 207645.0}, {'score': 262412.75}, {'score': 199893.28125}, {'score': 232974.21875}, {'score': 144285.8125}, {'score': 225841.34375}, {'score': 225731.375}, {'score': 228159.0}, {'score': 170916.578125}, {'score': 222742.03125}, {'score': 201306.78125}, {'score': 174816.09375}, {'score': 235742.3125}, {'score': 199438.15625}, {'score': 242300.90625}, {'score': 204122.65625}, {'score': 180236.140625}, {'score': 191758.40625}, {'score': 272889.03125}, {'score': 226123.5}, {'score': 179093.46875}, {'score': 225853.59375}, {'score': 217476.0625}, {'score': 187221.921875}, {'score': 212982.21875}, {'score': 246576.375}, {'score': 219202.40625}, {'score': 149682.78125}, {'score': 176518.8125}, {'score': 175494.84375}, {'score': 187580.625}, {'score': 228402.59375}, {'score': 235891.28125}, {'score': 169146.28125}, {'score': 256767.625}, {'score': 207062.4375}, {'score': 176921.375}, {'score': 184537.125}, {'score': 206786.5625}, {'score': 190976.5}, {'score': 161364.28125}, {'score': 226610.59375}, {'score': 190348.375}, {'score': 217799.734375}, {'score': 207521.6875}, {'score': 160131.96875}, {'score': 290557.59375}, {'score': 157076.84375}, {'score': 180603.109375}, {'score': 169334.96875}, {'score': 227847.6875}, {'score': 184451.84375}, {'score': 186199.9375}, {'score': 162852.375}, {'score': 206446.375}, {'score': 192622.34375}, {'score': 189848.5625}, {'score': 191923.53125}, {'score': 216144.90625}, {'score': 226486.796875}, {'score': 200515.65625}, {'score': 163252.96875}, {'score': 134918.9375}, {'score': 163479.5625}, {'score': 212693.34375}, {'score': 183718.4375}, {'score': 198996.96875}, {'score': 172865.5625}, {'score': 169160.0}, {'score': 222359.3125}, {'score': 141244.9375}, {'score': 188180.46875}, {'score': 177837.53125}, {'score': 233585.9375}, {'score': 223009.8125}, {'score': 230175.9375}, {'score': 268682.4375}, {'score': 220768.6875}, {'score': 181289.46875}, {'score': 149166.234375}, {'score': 237994.9375}, {'score': 189124.21875}, {'score': 215722.25}, {'score': 181875.578125}, {'score': 173428.234375}, {'score': 196536.765625}, {'score': 200677.75}, {'score': 179378.96875}, {'score': 192655.46875}, {'score': 363412.21875}, {'score': 190926.171875}, {'score': 131790.6875}, {'score': 160802.28125}, {'score': 159151.28125}, {'score': 178373.828125}, {'score': 226657.03125}, {'score': 93337.5234375}, {'score': 203543.125}, {'score': 247155.84375}, {'score': 194619.4375}, {'score': 167553.8125}, {'score': 181341.25}, {'score': 112538.4921875}, {'score': 227807.34375}, {'score': 177514.328125}, {'score': 183992.15625}, {'score': 159692.984375}, {'score': 228709.4375}, {'score': 189108.5}, {'score': 209001.90625}, {'score': 235840.046875}, {'score': 185433.65625}, {'score': 152909.09375}, {'score': 378663.5}, {'score': 99092.1953125}, {'score': 199714.84375}, {'score': 126518.5703125}, {'score': 123580.0859375}, {'score': 280637.125}, {'score': 165697.46875}, {'score': 274582.90625}, {'score': 166507.265625}, {'score': 232237.5625}, {'score': 208938.734375}, {'score': 137981.5625}, {'score': 209681.03125}, {'score': 211913.765625}, {'score': 205026.96875}, {'score': 216592.703125}, {'score': 193777.375}, {'score': 199999.09375}, {'score': 210016.5}, {'score': 183884.875}, {'score': 172683.78125}, {'score': 168250.0625}, {'score': 185978.0625}, {'score': 178591.578125}, {'score': 149439.921875}, {'score': 226064.5625}, {'score': 136692.75}, {'score': 166603.59375}, {'score': 164825.09375}, {'score': 165901.53125}, {'score': 169385.4375}, {'score': 176329.46875}, {'score': 169742.046875}, {'score': 185219.78125}, {'score': 277812.84375}, {'score': 158127.9375}, {'score': 195305.03125}, {'score': 168301.71875}, {'score': 227552.65625}, {'score': 221356.84375}, {'score': 171662.25}, {'score': 207919.5625}, {'score': 149734.671875}, {'score': 137041.84375}, {'score': 215304.8125}, {'score': 173039.921875}, {'score': 158302.625}, {'score': 192307.5625}, {'score': 212422.5625}, {'score': 196313.96875}, {'score': 166610.71875}, {'score': 211025.9375}, {'score': 242988.25}, {'score': 264124.03125}, {'score': 176410.375}, {'score': 147554.28125}, {'score': 197927.15625}, {'score': 172234.0}, {'score': 155766.9375}, {'score': 257241.09375}, {'score': 193139.59375}, {'score': 231916.25}, {'score': 161400.125}, {'score': 179271.03125}, {'score': 196037.5}, {'score': 150350.90625}, {'score': 210109.34375}, {'score': 174034.78125}, {'score': 174522.03125}, {'score': 224078.25}, {'score': 199330.40625}, {'score': 222929.484375}, {'score': 195651.046875}, {'score': 199111.625}, {'score': 159080.65625}, {'score': 211647.3125}, {'score': 181162.484375}, {'score': 198956.125}, {'score': 165901.4375}, {'score': 365656.375}, {'score': 181200.40625}, {'score': 176742.015625}, {'score': 208906.96875}, {'score': 188246.3125}, {'score': 178447.28125}, {'score': 207346.59375}, {'score': 244514.34375}, {'score': 184368.8125}, {'score': 144094.078125}, {'score': 150407.875}, {'score': 151585.4375}, {'score': 219606.3125}, {'score': 156875.71875}, {'score': 211622.8125}, {'score': 165501.8125}, {'score': 182550.90625}, {'score': 222479.65625}, {'score': 210193.15625}, {'score': 179367.640625}, {'score': 210112.5625}, {'score': 171277.96875}, {'score': 161438.78125}, {'score': 141165.3125}, {'score': 159580.8125}, {'score': 162850.59375}, {'score': 207489.96875}, {'score': 179063.6875}, {'score': 152358.6875}, {'score': 217014.34375}, {'score': 160683.65625}, {'score': 230604.875}, {'score': 191290.21875}, {'score': 195897.734375}, {'score': 181817.71875}, {'score': 181555.875}, {'score': 222214.578125}, {'score': 175805.0625}, {'score': 175119.0625}, {'score': 224463.84375}, {'score': 186325.03125}, {'score': 170730.03125}, {'score': 670210.4375}, {'score': 183040.78125}, {'score': 217603.25}, {'score': 212283.625}, {'score': 174484.90625}, {'score': 162953.03125}, {'score': 188661.53125}, {'score': 196726.03125}, {'score': 234893.46875}, {'score': 166870.796875}, {'score': 223539.4375}, {'score': 197106.0}, {'score': 253104.96875}, {'score': 240423.15625}, {'score': 140264.578125}, {'score': 175290.875}, {'score': 138352.125}, {'score': 185946.09375}, {'score': 191412.4375}, {'score': 170895.640625}, {'score': 226493.59375}, {'score': 164943.6875}, {'score': 236294.59375}, {'score': 243254.59375}, {'score': 178632.25}, {'score': 167233.8125}, {'score': 165595.9375}, {'score': 221647.40625}, {'score': 182585.609375}, {'score': 209166.6875}, {'score': 126270.4609375}, {'score': 190302.8125}, {'score': 163518.3125}, {'score': 196224.375}, {'score': 205043.84375}, {'score': 207895.953125}, {'score': 195653.625}, {'score': 140357.984375}, {'score': 144545.65625}, {'score': 204062.59375}, {'score': 167320.828125}, {'score': 36592.2421875}, {'score': 229370.828125}, {'score': 156757.1875}, {'score': 126729.3515625}, {'score': 129739.1015625}, {'score': 220260.46875}, {'score': 190033.34375}, {'score': 223471.671875}, {'score': 199609.625}, {'score': 180026.078125}, {'score': 135030.34375}, {'score': 222494.40625}, {'score': 162297.5}, {'score': 181488.90625}, {'score': 175067.53125}, {'score': 191740.34375}, {'score': 180795.59375}, {'score': 211594.46875}, {'score': 212097.421875}, {'score': 195165.75}, {'score': 246489.3125}, {'score': 211506.1875}, {'score': 133953.390625}, {'score': 171079.90625}, {'score': 172117.25}, {'score': 167305.53125}, {'score': 174318.5}, {'score': 201376.125}, {'score': 163583.53125}, {'score': 225240.84375}, {'score': 217982.65625}, {'score': 147803.9375}, {'score': 252330.0625}, {'score': 137174.21875}, {'score': 203294.125}, {'score': 252552.6875}, {'score': 168411.25}, {'score': 200101.03125}, {'score': 172737.53125}, {'score': 186548.203125}, {'score': 177350.96875}, {'score': 202897.28125}, {'score': 161383.796875}, {'score': 263109.125}, {'score': 179417.484375}, {'score': 156961.9375}, {'score': 251646.96875}, {'score': 226655.5}, {'score': 415657.96875}, {'score': 158661.375}, {'score': 187981.125}, {'score': 170261.90625}, {'score': 141875.03125}, {'score': 197594.3125}, {'score': 339664.625}, {'score': 212477.625}, {'score': 209501.5}, {'score': 165691.90625}, {'score': 130597.4296875}, {'score': 268392.375}, {'score': 169935.671875}, {'score': 189416.15625}, {'score': 225041.71875}, {'score': 187343.625}, {'score': 171317.46875}, {'score': 155881.4375}, {'score': 247275.09375}, {'score': 181355.5625}, {'score': 161941.34375}, {'score': 135124.59375}, {'score': 242117.65625}, {'score': 243269.03125}, {'score': 226842.90625}, {'score': 192052.40625}, {'score': 264946.15625}, {'score': 98938.6171875}, {'score': 177271.296875}, {'score': 200393.875}, {'score': 202517.734375}, {'score': 154106.875}, {'score': 220725.90625}, {'score': 180994.796875}, {'score': 196958.34375}, {'score': 158790.71875}, {'score': 223005.5}, {'score': 162837.265625}, {'score': 189711.21875}, {'score': 164855.875}, {'score': 213365.375}, {'score': 264561.78125}, {'score': 154828.28125}, {'score': 232242.5}, {'score': 182903.671875}, {'score': 183219.65625}, {'score': 173898.96875}, {'score': 204461.296875}, {'score': 176240.25}, {'score': 168522.65625}, {'score': 194897.03125}, {'score': 167238.53125}, {'score': 180110.28125}, {'score': 206099.84375}, {'score': 181157.6875}, {'score': 152664.25}, {'score': 179278.625}, {'score': 260596.21875}, {'score': 146201.46875}, {'score': 193881.015625}, {'score': 210049.375}, {'score': 197990.125}, {'score': 206016.875}, {'score': 202434.25}, {'score': 153725.90625}, {'score': 215637.84375}, {'score': 202333.78125}, {'score': 227961.421875}, {'score': 237141.0}, {'score': 160229.421875}, {'score': 199028.90625}, {'score': 178200.1875}, {'score': 158265.78125}, {'score': 198410.65625}, {'score': 242414.71875}, {'score': 165719.25}, {'score': 202501.875}, {'score': 173605.125}, {'score': 173996.625}, {'score': 165342.84375}, {'score': 169324.390625}, {'score': 234816.875}, {'score': 207725.65625}, {'score': 180988.921875}, {'score': 259557.25}, {'score': 204546.25}, {'score': 117474.0546875}, {'score': 194659.96875}, {'score': 129024.2578125}, {'score': 128355.7578125}, {'score': 231623.40625}, {'score': 179249.625}, {'score': 181186.15625}, {'score': 192519.0}, {'score': 158305.9375}, {'score': 195606.53125}, {'score': 194954.28125}, {'score': 110754.5859375}, {'score': 194398.0625}, {'score': 138801.765625}, {'score': 176657.78125}, {'score': 177393.671875}, {'score': 213888.6875}, {'score': 243511.25}, {'score': 162540.03125}, {'score': 179096.171875}, {'score': 217759.921875}, {'score': 208987.671875}, {'score': 172049.84375}, {'score': 298027.90625}, {'score': 153365.90625}, {'score': 160897.34375}, {'score': 182855.140625}, {'score': 176049.5625}, {'score': 156820.15625}, {'score': 220628.03125}, {'score': 163666.890625}, {'score': 172437.46875}, {'score': 206097.96875}, {'score': 197840.875}, {'score': 204854.71875}, {'score': 189176.21875}, {'score': 187337.0625}, {'score': 292487.09375}, {'score': 183735.40625}, {'score': 328919.65625}, {'score': 230822.53125}, {'score': 167690.375}, {'score': 203254.3125}, {'score': 185964.375}, {'score': 199392.828125}, {'score': 174579.90625}, {'score': 220201.6875}, {'score': 196935.65625}, {'score': 144541.0625}, {'score': 215799.9375}, {'score': 172299.0625}, {'score': 177030.09375}, {'score': 217066.71875}, {'score': 159198.3125}, {'score': 190030.15625}, {'score': 207154.1875}, {'score': 278630.15625}, {'score': 147275.34375}, {'score': 157792.8125}, {'score': 182973.0625}, {'score': 148857.03125}, {'score': 164905.1875}, {'score': 136781.25}, {'score': 197173.03125}, {'score': 155514.1875}, {'score': 162794.1875}, {'score': 135697.53125}, {'score': 166576.546875}, {'score': 165464.78125}, {'score': 205806.59375}, {'score': 183997.40625}, {'score': 200303.046875}, {'score': 174357.421875}, {'score': 207200.3125}, {'score': 252201.9375}, {'score': 225695.65625}, {'score': 160357.5}, {'score': 206987.21875}, {'score': 159980.6875}, {'score': 182704.15625}, {'score': 190730.71875}, {'score': 226543.3125}, {'score': 209722.78125}, {'score': 187644.171875}, {'score': 174932.6875}, {'score': 170470.5}, {'score': 250244.75}, {'score': 181596.5625}, {'score': 193463.8125}, {'score': 192402.875}, {'score': 243340.15625}, {'score': 211361.375}, {'score': 202172.0}, {'score': 205698.90625}, {'score': 222184.203125}, {'score': 112206.8671875}, {'score': 139664.453125}, {'score': 256963.71875}, {'score': 174170.59375}, {'score': 174278.09375}, {'score': 163077.75}, {'score': 190403.40625}, {'score': 150362.171875}, {'score': 173038.125}, {'score': 145632.859375}, {'score': 384493.59375}, {'score': 191524.4375}, {'score': 211461.96875}, {'score': 201581.625}, {'score': 217847.015625}, {'score': 250947.0625}, {'score': 115554.8671875}, {'score': 173496.515625}, {'score': 227538.5}, {'score': 164494.8125}, {'score': 177062.625}, {'score': 184335.53125}, {'score': 177076.84375}, {'score': 204029.734375}, {'score': 209029.0625}, {'score': 129481.0234375}, {'score': 92418.5234375}, {'score': 170484.4375}, {'score': 160835.90625}, {'score': 212237.75}, {'score': 280452.96875}, {'score': 232343.6875}, {'score': 156402.625}, {'score': 198998.875}, {'score': 196935.6875}, {'score': 115120.390625}, {'score': 231934.40625}, {'score': 211297.9375}, {'score': 210575.1875}, {'score': 193305.375}, {'score': 201844.25}, {'score': 155883.15625}, {'score': 183285.171875}, {'score': 203146.96875}, {'score': 196179.25}, {'score': 197137.8125}, {'score': 175768.25}, {'score': 225393.53125}, {'score': 172735.71875}, {'score': 177800.09375}, {'score': 151612.25}, {'score': 215218.1875}, {'score': 182639.375}, {'score': 247427.875}, {'score': 201229.5}, {'score': 143369.0625}, {'score': 259997.59375}, {'score': 218466.0625}, {'score': 182899.4375}, {'score': 197058.90625}, {'score': 208666.28125}, {'score': 200706.21875}, {'score': 320945.15625}, {'score': 189104.65625}, {'score': 45926.7421875}, {'score': 168257.203125}, {'score': 165356.46875}, {'score': 222037.59375}, {'score': 213709.09375}, {'score': 227153.234375}, {'score': 211498.25}, {'score': 166433.0625}, {'score': 214181.125}, {'score': 165649.75}, {'score': 163730.90625}, {'score': 260779.84375}, {'score': 257216.3125}, {'score': 169953.96875}, {'score': 182638.25}, {'score': 236750.0}, {'score': 408346.90625}, {'score': 226499.4375}, {'score': 178069.375}, {'score': 158491.21875}, {'score': 191547.4375}, {'score': 196968.09375}, {'score': 233429.75}, {'score': 179627.0}, {'score': 176090.09375}, {'score': 96282.2578125}, {'score': 231442.78125}, {'score': 220659.53125}, {'score': 238760.96875}, {'score': 229462.03125}, {'score': 184206.375}, {'score': 194078.15625}, {'score': 198493.96875}, {'score': 224882.4375}, {'score': 160094.84375}, {'score': 158721.09375}, {'score': 205822.578125}, {'score': 244366.125}, {'score': 168663.265625}, {'score': 324104.75}, {'score': 181011.09375}, {'score': 167267.65625}, {'score': 152986.34375}, {'score': 236671.46875}, {'score': 233361.140625}, {'score': 232034.828125}, {'score': 157161.03125}, {'score': 174909.65625}, {'score': 205592.5625}, {'score': 200535.03125}, {'score': 151067.59375}, {'score': 201535.21875}, {'score': 188916.0}, {'score': 181976.78125}, {'score': 173403.75}, {'score': 187176.09375}, {'score': 187420.25}, {'score': 177904.3125}, {'score': 168856.0}, {'score': 249513.71875}, {'score': 204295.6875}, {'score': 194257.171875}, {'score': 167228.96875}, {'score': 206942.75}, {'score': 316903.15625}, {'score': 281298.8125}, {'score': 140307.28125}, {'score': 169740.8125}, {'score': 151188.5}, {'score': 98578.3984375}, {'score': 208307.875}, {'score': 212133.5}, {'score': 146588.96875}, {'score': 212164.6875}, {'score': 194415.1875}, {'score': 142753.75}, {'score': 212731.28125}, {'score': 183632.71875}, {'score': 209281.8125}, {'score': 179680.5625}, {'score': 187559.125}, {'score': 196456.875}, {'score': 145520.578125}, {'score': 202465.90625}, {'score': 136201.1875}, {'score': 194038.84375}, {'score': 181352.25}, {'score': 218834.9375}, {'score': 121475.9921875}, {'score': 189221.375}, {'score': 207346.5}, {'score': 167037.84375}, {'score': 237019.875}, {'score': 192814.0}, {'score': 202614.78125}, {'score': 169242.09375}, {'score': 270137.4375}, {'score': 169529.40625}, {'score': 143588.3125}, {'score': 195036.15625}, {'score': 178438.40625}, {'score': 180425.609375}, {'score': 227056.0625}, {'score': 229885.40625}, {'score': 192860.59375}, {'score': 212112.03125}, {'score': 230989.0625}, {'score': 193397.9375}, {'score': 203682.609375}, {'score': 169308.90625}, {'score': 175035.859375}, {'score': 200159.28125}, {'score': 198749.15625}, {'score': 167781.125}, {'score': 223163.8125}, {'score': 174334.1875}, {'score': 152978.765625}, {'score': 185789.21875}, {'score': 214048.625}, {'score': 184227.09375}, {'score': 203463.203125}, {'score': 236855.9375}, {'score': 211401.109375}, {'score': 175384.3125}, {'score': 204500.65625}, {'score': 174655.3125}, {'score': 209454.796875}, {'score': 151088.34375}, {'score': 162747.109375}, {'score': 206130.4375}, {'score': 231909.34375}, {'score': 262680.53125}, {'score': 137020.4375}, {'score': 191558.21875}, {'score': 182361.3125}, {'score': 222859.96875}, {'score': 179596.71875}, {'score': 225818.90625}, {'score': 170553.015625}, {'score': 178028.78125}, {'score': 210303.65625}, {'score': 229001.53125}, {'score': 243477.40625}, {'score': 243359.71875}, {'score': 173062.09375}, {'score': 126015.0625}, {'score': 164105.96875}, {'score': 206314.96875}, {'score': 136125.0625}, {'score': 265784.0625}, {'score': 185371.03125}, {'score': 207722.1875}, {'score': 132349.03125}, {'score': 130701.078125}, {'score': 168647.0625}, {'score': 214104.21875}, {'score': 233164.40625}, {'score': 178282.125}, {'score': 199483.78125}, {'score': 225157.90625}, {'score': 154229.8125}, {'score': 200748.4375}, {'score': 143547.53125}, {'score': 188907.421875}, {'score': 175280.3125}, {'score': 164275.859375}, {'score': 162329.890625}, {'score': 193911.6875}, {'score': 143420.109375}, {'score': 188078.5}, {'score': 194839.28125}, {'score': 172610.28125}, {'score': -15069.1640625}, {'score': 199336.21875}, {'score': 250079.40625}, {'score': 189752.5}, {'score': 160545.9375}, {'score': 273488.5625}, {'score': 249934.8125}, {'score': 191818.671875}, {'score': 206587.625}, {'score': 185585.25}, {'score': 155431.078125}, {'score': 186563.34375}, {'score': 243393.65625}, {'score': 177909.5}, {'score': 197359.9375}, {'score': 221100.125}, {'score': 181293.375}, {'score': 173356.984375}, {'score': 67438.2421875}, {'score': 206127.84375}, {'score': 152294.96875}, {'score': 202828.28125}, {'score': 154307.875}, {'score': 215762.125}, {'score': 200509.03125}, {'score': 266116.6875}, {'score': 152933.4375}, {'score': 216814.09375}, {'score': 156533.5}, {'score': 203458.34375}, {'score': 185736.21875}, {'score': 157159.28125}, {'score': 146328.203125}, {'score': 42980.8359375}, {'score': -197416.75}, {'score': 229409.84375}, {'score': 128165.6953125}, {'score': 191433.8125}, {'score': 220166.984375}, {'score': 194481.25}, {'score': 259194.28125}, {'score': 170154.296875}, {'score': 127706.984375}, {'score': 163232.4375}, {'score': 112018.1640625}, {'score': 175011.71875}, {'score': 136170.515625}, {'score': 209719.625}, {'score': 197190.625}, {'score': 178732.125}, {'score': 126739.3125}, {'score': 163515.359375}, {'score': 191070.296875}, {'score': 222512.734375}, {'score': 197497.375}, {'score': 191253.53125}, {'score': 181043.46875}, {'score': 215033.9375}, {'score': 193502.375}, {'score': 211111.96875}, {'score': 174274.09375}, {'score': 272743.59375}, {'score': 209906.25}, {'score': 190050.5625}, {'score': 234110.875}, {'score': 215976.4375}, {'score': 169656.0}, {'score': 150165.796875}, {'score': 197293.234375}, {'score': 186026.84375}, {'score': 193286.03125}, {'score': 202015.8125}, {'score': 188423.875}, {'score': 187965.5625}, {'score': 170133.625}, {'score': 290239.84375}, {'score': 205439.375}, {'score': 143946.1875}, {'score': 219844.5625}, {'score': 227074.96875}, {'score': 171112.375}, {'score': 165004.34375}, {'score': 194334.8125}, {'score': 189969.71875}, {'score': 147041.46875}, {'score': 179148.625}, {'score': 171786.90625}, {'score': 228471.6875}, {'score': 198957.5625}, {'score': 191177.171875}, {'score': 217519.296875}, {'score': 156367.171875}, {'score': 180000.375}, {'score': 201762.5}, {'score': 210671.21875}, {'score': 185360.34375}, {'score': 188396.015625}, {'score': 233673.25}, {'score': 178840.46875}, {'score': 192437.46875}, {'score': 178941.453125}, {'score': 218556.453125}, {'score': 168424.6875}, {'score': 201093.71875}, {'score': 272651.625}, {'score': 150427.375}, {'score': 187691.125}, {'score': 200171.65625}, {'score': 148848.75}, {'score': 153421.15625}, {'score': 124493.5703125}, {'score': 162132.6875}, {'score': 235222.4375}, {'score': 226380.875}, {'score': 191536.875}, {'score': 179990.71875}, {'score': 160667.46875}, {'score': 172302.9375}, {'score': 267181.21875}, {'score': 206887.5625}, {'score': 171281.9375}, {'score': 172588.21875}, {'score': 217638.90625}, {'score': 188770.5}, {'score': 221326.53125}, {'score': 142102.3125}, {'score': 224190.25}, {'score': 197997.890625}, {'score': 227568.15625}, {'score': 190504.5625}, {'score': 240214.671875}, {'score': 200169.703125}, {'score': 141810.5625}, {'score': 216807.1875}, {'score': 228853.59375}, {'score': 183590.71875}, {'score': 132859.40625}, {'score': 216438.25}, {'score': 219528.375}, {'score': 219470.84375}, {'score': 181032.78125}, {'score': 193463.234375}, {'score': 183520.875}, {'score': 185832.90625}, {'score': 230960.4375}, {'score': 204223.40625}, {'score': 202200.125}, {'score': 196280.09375}, {'score': 170249.8125}, {'score': 161680.25}, {'score': 235877.25}, {'score': 193858.78125}, {'score': 196978.75}, {'score': 160975.5625}, {'score': 216974.5}, {'score': 207433.84375}, {'score': 101491.6171875}, {'score': 195140.75}, {'score': 202698.3125}, {'score': 183590.90625}, {'score': 201988.25}, {'score': 162373.328125}, {'score': 183777.34375}, {'score': 212943.65625}, {'score': 211928.9375}, {'score': 179736.59375}, {'score': 181378.90625}, {'score': 223083.78125}, {'score': 219091.125}, {'score': 206433.3125}, {'score': 166303.828125}, {'score': 200659.6875}, {'score': 193129.8125}, {'score': 200636.90625}, {'score': 205404.3125}, {'score': 185540.34375}, {'score': 177440.359375}, {'score': 173532.5}, {'score': 227000.1875}, {'score': 6224.6796875}, {'score': 171476.09375}, {'score': 277868.34375}, {'score': 177955.328125}, {'score': 188075.8125}, {'score': 197409.75}, {'score': 161170.8125}, {'score': 189901.09375}, {'score': 178851.21875}, {'score': 208506.53125}, {'score': 128415.6328125}, {'score': 209279.125}, {'score': 186361.84375}, {'score': 230398.84375}, {'score': 235328.8125}, {'score': 239191.90625}, {'score': 219100.125}, {'score': 215753.0625}, {'score': 213194.203125}, {'score': 246848.03125}, {'score': 118837.84375}, {'score': 163621.03125}, {'score': 173668.921875}, {'score': 152675.109375}, {'score': 189308.9375}, {'score': 170736.0}, {'score': 249570.5}, {'score': 154992.90625}, {'score': 177350.28125}, {'score': 189818.15625}, {'score': 212086.921875}, {'score': 208404.8125}, {'score': 174635.5625}, {'score': 158569.15625}, {'score': 204089.96875}, {'score': 197052.5}, {'score': 215705.1875}, {'score': 181618.65625}, {'score': 210023.40625}, {'score': 129536.6328125}, {'score': 170921.484375}, {'score': 231211.4375}, {'score': 148613.78125}, {'score': 171320.21875}, {'score': 159245.609375}, {'score': 189205.875}, {'score': 223314.84375}, {'score': 137122.3125}, {'score': 213951.59375}, {'score': 202736.28125}, {'score': 212822.625}, {'score': 243633.71875}, {'score': 182705.84375}, {'score': 210712.4375}, {'score': 219904.3125}, {'score': 177878.65625}, {'score': 171404.875}, {'score': 236471.515625}, {'score': 165119.328125}, {'score': 142562.1875}, {'score': 151182.59375}, {'score': 184635.09375}, {'score': 156247.84375}, {'score': 232856.59375}, {'score': 212453.25}, {'score': 160210.84375}, {'score': 218866.125}, {'score': 193086.96875}, {'score': 232863.5625}, {'score': 197074.21875}, {'score': 177414.0625}, {'score': 144975.5}, {'score': 222513.84375}, {'score': 165304.09375}, {'score': 207157.625}, {'score': 232711.46875}, {'score': 211869.125}, {'score': 209871.90625}, {'score': 213132.6875}, {'score': 206135.21875}, {'score': 188377.46875}, {'score': 144026.578125}, {'score': 243893.5625}, {'score': 159527.8125}, {'score': 163460.3125}, {'score': 162785.734375}, {'score': 198167.46875}, {'score': 223329.71875}, {'score': 230228.5}, {'score': 139313.53125}, {'score': 160702.609375}, {'score': 190942.953125}, {'score': 215071.78125}, {'score': 215621.96875}, {'score': 190797.875}, {'score': 159007.625}, {'score': 178599.34375}, {'score': 227825.84375}, {'score': 263193.125}, {'score': 177879.78125}, {'score': 192999.9375}, {'score': 215904.9375}, {'score': 199828.859375}, {'score': 185319.875}, {'score': 171982.09375}, {'score': 180511.34375}, {'score': 179664.953125}, {'score': 186613.328125}, {'score': 187620.3125}, {'score': 146403.75}, {'score': 242025.3125}, {'score': 203640.3125}, {'score': 168203.375}, {'score': 235844.53125}, {'score': 183761.15625}, {'score': 192050.140625}, {'score': 234627.140625}, {'score': 160358.484375}, {'score': 376287.4375}, {'score': 253566.46875}, {'score': 138006.75}, {'score': 157520.59375}, {'score': 195283.40625}, {'score': 229613.625}, {'score': 168568.59375}, {'score': 148664.65625}, {'score': 220206.34375}, {'score': 189448.59375}, {'score': 169420.8125}, {'score': 253823.25}, {'score': 164351.5625}, {'score': 140129.140625}, {'score': 177910.875}, {'score': 119810.90625}, {'score': 165082.375}, {'score': 190163.4375}, {'score': 201606.703125}, {'score': 192557.5}, {'score': 162146.90625}, {'score': 274558.59375}, {'score': 244211.015625}, {'score': 189356.25}, {'score': 260904.34375}, {'score': 266782.3125}, {'score': 161554.5}, {'score': 173199.71875}, {'score': 208595.1875}, {'score': 198408.71875}, {'score': 137940.96875}, {'score': 191864.8125}, {'score': 187907.171875}, {'score': 219508.3125}, {'score': 228753.265625}, {'score': 149740.421875}, {'score': 190975.5625}, {'score': 178139.5}, {'score': 237177.578125}, {'score': 276830.21875}, {'score': 174880.1875}, {'score': 197132.96875}, {'score': 160932.5625}, {'score': 240618.453125}, {'score': 219214.0}, {'score': 177688.15625}, {'score': 127470.2421875}, {'score': 237328.1875}, {'score': 224548.9375}, {'score': 216878.0}, {'score': 193911.9375}, {'score': 154757.21875}, {'score': 193842.0}, {'score': 209446.6875}, {'score': 201431.1875}, {'score': 158078.75}, {'score': 156007.5}, {'score': 173612.46875}, {'score': 182255.390625}, {'score': 191533.59375}, {'score': 213299.78125}, {'score': 241623.265625}, {'score': 188301.1875}, {'score': 156958.015625}, {'score': 180880.28125}, {'score': 177629.21875}, {'score': 108480.3984375}, {'score': 168217.5}, {'score': 112663.9296875}, {'score': 189052.59375}, {'score': 245546.25}, {'score': 163980.25}, {'score': 199451.96875}, {'score': 265513.625}, {'score': 179843.8125}, {'score': 185697.71875}, {'score': 244280.671875}, {'score': 159029.03125}, {'score': 218453.0}, {'score': 149781.859375}, {'score': 206026.03125}, {'score': 224416.96875}, {'score': 433730.9375}, {'score': 204754.4375}, {'score': 201353.828125}, {'score': 222220.3125}, {'score': 162742.375}, {'score': 200221.5625}, {'score': 168035.21875}, {'score': 192361.90625}, {'score': 213124.3125}, {'score': 69294.1875}, {'score': 178816.0625}, {'score': 200557.453125}, {'score': 175197.59375}, {'score': 263819.15625}, {'score': 216538.328125}, {'score': 115277.8828125}, {'score': 143812.375}, {'score': 168570.0}, {'score': 203105.625}, {'score': 176616.875}, {'score': 121273.3828125}, {'score': 182494.59375}, {'score': 204694.640625}, {'score': 195532.625}, {'score': 200696.71875}, {'score': 217456.0}, {'score': 212122.15625}, {'score': 145658.875}, {'score': 194943.78125}, {'score': 140900.625}, {'score': 164188.90625}, {'score': 169524.46875}, {'score': 196893.0625}, {'score': 170143.28125}, {'score': 206492.78125}, {'score': 172682.5625}, {'score': 189367.09375}, {'score': 145724.3125}, {'score': 176836.34375}, {'score': 173225.625}, {'score': 173717.84375}, {'score': 155423.28125}, {'score': 191684.71875}, {'score': 829301.625}, {'score': 156660.75}, {'score': 218301.09375}, {'score': 229286.375}, {'score': 227596.75}, {'score': 333778.0}, {'score': 148390.671875}, {'score': 200481.203125}, {'score': 154729.125}, {'score': 89447.6328125}, {'score': 202355.03125}, {'score': 160718.15625}, {'score': 201556.46875}, {'score': 185614.75}, {'score': 237686.96875}, {'score': 207023.34375}, {'score': 173051.3125}, {'score': 172967.875}, {'score': 239995.8125}, {'score': 148975.421875}, {'score': 208622.4375}, {'score': 189889.171875}, {'score': 207427.8125}, {'score': 192996.78125}, {'score': 201892.5625}, {'score': 153713.1875}, {'score': 173117.34375}, {'score': 160272.6875}, {'score': 221552.125}, {'score': 208792.875}, {'score': 167258.90625}, {'score': 199193.90625}, {'score': 165249.3125}, {'score': 173273.703125}, {'score': 230866.125}, {'score': 199293.859375}, {'score': 191159.96875}, {'score': 209806.453125}, {'score': 199322.5}, {'score': 182173.78125}, {'score': 269469.0}, {'score': 237735.25}, {'score': 212793.71875}, {'score': 185551.1875}, {'score': 218456.0}, {'score': 194281.125}, {'score': 161120.53125}, {'score': 211982.53125}, {'score': 186445.625}, {'score': 142387.875}, {'score': 167721.875}, {'score': 247527.9375}, {'score': 167337.90625}, {'score': 162075.1875}, {'score': 204484.375}, {'score': 179944.328125}, {'score': 218048.5}, {'score': 178801.171875}, {'score': 197724.6875}, {'score': 181241.1875}, {'score': 194367.8125}, {'score': 222735.75}, {'score': 205654.96875}, {'score': 180571.0625}, {'score': 185492.46875}, {'score': 267380.46875}, {'score': 181410.625}, {'score': 208771.375}, {'score': 221211.28125}, {'score': 312150.4375}, {'score': 136850.0}, {'score': 154451.015625}, {'score': 252347.9375}, {'score': 189436.546875}, {'score': 189341.40625}, {'score': 151113.5625}, {'score': 239385.03125}, {'score': 172307.125}, {'score': 106063.3984375}, {'score': 167212.25}, {'score': 202944.71875}, {'score': 156220.21875}, {'score': 163287.09375}, {'score': -12500.3515625}, {'score': 199814.9375}, {'score': 161039.59375}, {'score': 165698.375}, {'score': 204107.09375}, {'score': 214688.25}, {'score': 226023.84375}, {'score': 189966.6875}, {'score': 189354.859375}, {'score': 201592.28125}, {'score': 194132.15625}, {'score': 209964.28125}, {'score': 181312.46875}, {'score': 206305.78125}, {'score': 140878.375}, {'score': 164855.59375}, {'score': 155231.59375}, {'score': 202611.375}, {'score': 145155.8125}, {'score': 157571.9375}, {'score': 227737.15625}, {'score': 169843.9375}, {'score': 185928.75}, {'score': 221303.0625}, {'score': 186167.390625}, {'score': 214695.96875}, {'score': 197219.96875}, {'score': 181212.15625}, {'score': 182490.9375}, {'score': 185912.625}, {'score': 231414.703125}, {'score': 123095.6171875}, {'score': 156161.9375}, {'score': 196240.65625}, {'score': 233708.6875}, {'score': 225185.296875}, {'score': 213121.25}, {'score': 206519.21875}, {'score': 158398.140625}, {'score': 181145.125}, {'score': 184998.4375}, {'score': 204879.828125}, {'score': 190688.5}, {'score': 220759.65625}, {'score': 166649.21875}, {'score': 184347.40625}, {'score': 193347.75}, {'score': 186042.03125}, {'score': 195564.9375}, {'score': 176363.875}, {'score': 208190.53125}, {'score': 198973.65625}, {'score': 180419.546875}, {'score': 185264.34375}, {'score': 183228.53125}, {'score': 209226.96875}, {'score': 152567.359375}, {'score': 152138.375}, {'score': 177088.65625}, {'score': 205007.140625}, {'score': 234202.6875}, {'score': 199725.25}, {'score': 157111.125}, {'score': 189078.90625}, {'score': 197344.25}, {'score': 136734.0625}, {'score': 174824.921875}, {'score': 208950.0}, {'score': 221982.34375}, {'score': 148703.84375}, {'score': 155789.375}, {'score': 153393.53125}, {'score': 194495.5625}, {'score': 206604.40625}, {'score': 221401.28125}, {'score': 183838.65625}, {'score': 173263.5625}, {'score': 179781.453125}, {'score': 222519.28125}, {'score': 164640.671875}, {'score': 204117.328125}, {'score': 227115.5625}, {'score': 191491.03125}, {'score': 185412.34375}, {'score': 192182.0625}, {'score': 201713.125}, {'score': 231697.3125}, {'score': 164456.1875}, {'score': 243863.96875}, {'score': 242853.125}, {'score': 183245.28125}, {'score': 210066.0625}, {'score': 156311.5625}, {'score': 177477.109375}, {'score': 181100.75}, {'score': 258771.21875}, {'score': 169120.625}, {'score': 192578.875}, {'score': 200355.28125}, {'score': 242728.0}, {'score': 207603.9375}, {'score': 178909.375}, {'score': 209183.765625}, {'score': 222722.96875}, {'score': 173689.203125}, {'score': 202753.15625}, {'score': 191194.0}, {'score': 199434.875}, {'score': 162877.421875}, {'score': 198510.40625}, {'score': 243019.0}, {'score': 219603.75}, {'score': 145246.0}, {'score': 224466.40625}, {'score': 147927.9375}, {'score': 193913.0625}, {'score': 212890.6875}, {'score': 172305.046875}, {'score': 157169.140625}, {'score': 210742.09375}, {'score': 310366.5625}, {'score': 182479.46875}, {'score': 206744.0}, {'score': 202049.59375}, {'score': 139690.796875}, {'score': 166096.234375}, {'score': 234017.03125}, {'score': 256944.53125}, {'score': 202813.40625}, {'score': 244588.28125}, {'score': 137671.53125}, {'score': 165295.46875}, {'score': 189511.15625}, {'score': 173983.953125}, {'score': 165161.21875}, {'score': 136333.90625}, {'score': 218926.625}, {'score': 274075.1875}, {'score': 167173.125}, {'score': 165815.375}, {'score': 176511.0625}, {'score': 257235.15625}, {'score': 198931.5}, {'score': 182668.625}, {'score': 168393.3125}, {'score': 184115.1875}, {'score': 178208.15625}, {'score': 194745.21875}, {'score': 197221.90625}, {'score': 204848.96875}, {'score': 203660.4375}, {'score': 197370.59375}, {'score': 196402.5}, {'score': 229846.453125}, {'score': 240446.375}, {'score': 208403.65625}, {'score': 153625.875}, {'score': 247453.125}, {'score': 153443.0625}, {'score': 176478.828125}, {'score': 243032.96875}, {'score': 224576.6875}, {'score': 178102.921875}, {'score': 197807.375}, {'score': 190340.65625}, {'score': 155572.6875}, {'score': 199394.4375}, {'score': 136865.4375}, {'score': 244025.1875}, {'score': 262999.71875}, {'score': 154667.640625}, {'score': 191229.359375}, {'score': 228368.59375}, {'score': 180430.0625}, {'score': 243572.15625}, {'score': 203640.84375}, {'score': 373367.4375}, {'score': 169766.125}, {'score': 198819.9375}, {'score': 207267.875}, {'score': 223188.21875}, {'score': 202946.75}, {'score': 221751.15625}, {'score': 169066.0}, {'score': 205287.3125}, {'score': 148402.9375}, {'score': 219021.4375}, {'score': 166752.3125}, {'score': 218843.71875}, {'score': 158246.921875}, {'score': 158474.84375}, {'score': 155102.21875}, {'score': 162460.6875}, {'score': 186305.296875}, {'score': 297335.28125}, {'score': 198601.125}, {'score': 167777.1875}, {'score': 210075.9375}, {'score': 205492.90625}, {'score': 144737.25}, {'score': 187910.34375}, {'score': 199720.703125}, {'score': 139838.578125}, {'score': 222926.65625}, {'score': 196104.71875}, {'score': 157501.1875}, {'score': 184412.625}, {'score': 166754.640625}, {'score': 143802.625}, {'score': 130123.484375}, {'score': 211886.4375}, {'score': 197947.6875}, {'score': 245363.171875}, {'score': 172311.8125}, {'score': 160338.046875}, {'score': 148700.9375}, {'score': 158233.4375}, {'score': 193014.0625}, {'score': 168288.34375}, {'score': 134291.09375}, {'score': 144238.125}, {'score': 294696.25}, {'score': 279416.34375}, {'score': 167639.09375}, {'score': 172776.53125}, {'score': 137820.140625}, {'score': 156908.796875}, {'score': 201629.5}, {'score': 286177.375}, {'score': 183451.609375}, {'score': 192847.59375}, {'score': 143344.46875}, {'score': 237876.34375}, {'score': 205355.8125}, {'score': 212373.03125}, {'score': 191854.5}, {'score': 195332.34375}, {'score': 175510.5625}, {'score': 196651.9375}, {'score': 189862.34375}, {'score': 129056.625}, {'score': 222893.8125}, {'score': 271329.15625}, {'score': 162917.6875}, {'score': 177627.53125}, {'score': 194984.015625}, {'score': 171664.375}, {'score': 204219.8125}, {'score': 172179.5625}, {'score': 167375.171875}, {'score': 205465.5625}, {'score': 215218.21875}, {'score': 165492.109375}, {'score': 225749.53125}, {'score': 195996.6875}, {'score': 183916.015625}, {'score': 157729.34375}, {'score': 178302.125}, {'score': 225565.1875}, {'score': 147294.390625}, {'score': 199320.640625}, {'score': 197823.96875}, {'score': 215309.21875}, {'score': 233774.40625}, {'score': 195022.734375}, {'score': 193888.34375}, {'score': 176802.84375}, {'score': 174988.109375}, {'score': 171383.21875}, {'score': 210342.9375}, {'score': 184891.390625}, {'score': 221251.75}, {'score': 243364.59375}, {'score': 216397.0625}, {'score': 198543.3125}, {'score': 173406.125}, {'score': 411008.15625}, {'score': 194843.4375}, {'score': 204353.34375}, {'score': 179074.4375}, {'score': 181272.90625}, {'score': 194342.671875}, {'score': 191920.65625}, {'score': 226886.328125}, {'score': 197383.59375}, {'score': 168091.03125}, {'score': 157483.46875}, {'score': 182951.6875}, {'score': 247664.0625}, {'score': 178870.25}, {'score': 211999.4375}, {'score': 184108.640625}, {'score': 184735.15625}, {'score': 199991.34375}, {'score': 98823.7890625}, {'score': 162635.703125}, {'score': 203073.65625}, {'score': 297392.96875}, {'score': 177178.71875}, {'score': 167788.6875}, {'score': 179144.125}, {'score': 104898.34375}, {'score': 205937.75}, {'score': 162343.859375}, {'score': 194382.09375}, {'score': 197835.84375}, {'score': 188479.421875}, {'score': 204436.59375}, {'score': 164940.96875}, {'score': 179470.21875}, {'score': 202967.15625}, {'score': 241478.65625}, {'score': 199730.25}, {'score': 170722.0625}, {'score': 218271.46875}, {'score': 218211.4375}, {'score': 152968.5625}, {'score': 249092.78125}, {'score': 108603.7109375}, {'score': 162709.625}, {'score': 212943.75}, {'score': 145204.03125}, {'score': 153208.203125}, {'score': 164853.84375}, {'score': 218519.03125}, {'score': 164026.890625}, {'score': 147424.6875}, {'score': 64658.3359375}, {'score': 215644.1875}, {'score': 160100.125}, {'score': 206139.5}, {'score': 182656.40625}, {'score': 205200.78125}, {'score': 120466.8671875}, {'score': 213809.6875}, {'score': 209188.09375}, {'score': 196180.21875}, {'score': 255532.75}, {'score': 144008.96875}, {'score': 192525.0625}, {'score': 220830.71875}, {'score': 208299.3125}, {'score': 180100.59375}, {'score': 166865.4375}, {'score': 147560.65625}, {'score': 186347.53125}, {'score': 156086.15625}, {'score': 211151.484375}, {'score': 204804.5625}, {'score': 65668.3984375}, {'score': 125094.6640625}, {'score': 178790.875}, {'score': 171914.65625}, {'score': 171570.296875}, {'score': 202779.5}, {'score': 187008.625}, {'score': 206637.15625}, {'score': 233818.75}, {'score': 212006.5}, {'score': 241366.375}, {'score': 463853.0}, {'score': 184788.5}, {'score': 160517.59375}, {'score': 206091.09375}, {'score': 209098.90625}, {'score': 163997.78125}, {'score': 240712.46875}, {'score': 179581.28125}, {'score': 107624.7265625}, {'score': 142207.484375}, {'score': 203404.34375}, {'score': 163546.265625}, {'score': 138737.15625}, {'score': 198507.0}, {'score': 145051.46875}, {'score': 193899.375}, {'score': 273106.53125}, {'score': 154801.75}, {'score': 205271.375}, {'score': 192508.8125}, {'score': 190016.25}, {'score': 207535.53125}, {'score': 188745.09375}, {'score': 227682.796875}, {'score': 206068.875}, {'score': 158875.0625}, {'score': 252903.875}, {'score': 148607.59375}, {'score': 187326.0}, {'score': 238537.8125}, {'score': 208917.125}, {'score': 305553.03125}, {'score': 257191.75}, {'score': 206662.59375}, {'score': 238310.03125}, {'score': 172982.46875}, {'score': 236734.8125}, {'score': 195224.34375}, {'score': 185755.5625}, {'score': -301.234375}, {'score': 138814.3125}, {'score': 193835.4375}, {'score': 281299.65625}, {'score': 169055.09375}, {'score': 161314.09375}, {'score': 178987.515625}, {'score': 163065.4375}, {'score': 174414.125}, {'score': 191579.90625}, {'score': 220480.25}, {'score': 179147.375}, {'score': 185552.03125}, {'score': 191628.375}, {'score': 209150.6875}, {'score': 242046.4375}, {'score': 224603.21875}, {'score': 221708.9375}, {'score': 165048.375}, {'score': 192692.640625}, {'score': 146396.125}, {'score': 259568.0}, {'score': 150401.578125}, {'score': 151951.625}, {'score': 191739.59375}, {'score': 188455.828125}, {'score': 214026.875}, {'score': 195106.125}, {'score': 107499.2265625}, {'score': 170999.6875}, {'score': 207564.5}, {'score': 179980.0}, {'score': 181610.875}, {'score': 145128.5625}, {'score': 206141.484375}, {'score': 221010.71875}, {'score': 225552.5625}, {'score': 191185.125}, {'score': 150637.1875}, {'score': 211669.75}, {'score': 178626.25}, {'score': 165497.25}, {'score': 182636.6875}, {'score': 224301.46875}, {'score': 210176.0625}, {'score': 202009.90625}, {'score': 191621.0}, {'score': 75613.6796875}, {'score': 165748.125}, {'score': 206171.03125}, {'score': 228500.875}, {'score': 109817.6953125}, {'score': 252745.9375}, {'score': 239013.84375}, {'score': 218980.1875}, {'score': 195682.625}, {'score': 182075.375}, {'score': 168760.0}, {'score': 207171.09375}, {'score': 138921.078125}, {'score': 189720.9375}, {'score': 178356.90625}, {'score': 186696.53125}, {'score': 175692.609375}, {'score': 218551.265625}, {'score': 210855.34375}, {'score': 240150.40625}, {'score': 210726.53125}, {'score': 233289.3125}, {'score': 119436.5859375}, {'score': 177632.8125}, {'score': 220804.375}, {'score': 149254.09375}, {'score': 247424.3125}, {'score': 130335.4453125}, {'score': 200749.5625}, {'score': 208167.234375}, {'score': 175201.1875}, {'score': 210354.53125}, {'score': 202184.9375}, {'score': 148914.8125}, {'score': 155414.125}, {'score': 211407.46875}, {'score': 231133.328125}, {'score': 221577.5}, {'score': 272860.125}, {'score': 154303.09375}, {'score': 135066.75}, {'score': 151929.28125}, {'score': 143612.15625}, {'score': 163616.34375}, {'score': 218060.125}, {'score': 273851.84375}, {'score': 217517.8125}, {'score': 229025.3125}, {'score': 218450.53125}, {'score': 205725.03125}, {'score': 174987.375}, {'score': 227149.15625}, {'score': 194900.4375}, {'score': 355810.0625}, {'score': 178372.46875}, {'score': 193860.28125}, {'score': 209543.96875}, {'score': 162606.28125}, {'score': 229984.4375}, {'score': 161003.46875}, {'score': 67089.6796875}, {'score': 218470.46875}, {'score': 193066.65625}, {'score': 185346.875}, {'score': 173738.609375}, {'score': 202096.0625}, {'score': 191828.71875}, {'score': 175750.84375}, {'score': 214683.4375}, {'score': 177533.3125}, {'score': 205497.3125}, {'score': 160731.359375}, {'score': 205980.5}, {'score': 187141.53125}, {'score': 208240.21875}, {'score': 195264.1875}, {'score': 211066.4375}, {'score': 181399.1875}, {'score': 172037.71875}, {'score': 248198.84375}, {'score': 157918.09375}, {'score': 169506.6875}, {'score': 157979.890625}, {'score': 200959.65625}, {'score': 202217.03125}, {'score': 243458.25}, {'score': 152006.59375}, {'score': 159943.75}, {'score': 188382.75}, {'score': 126845.4296875}, {'score': 159552.1875}, {'score': 160113.65625}, {'score': 223281.5}, {'score': 200879.65625}, {'score': 170042.375}, {'score': 198151.8125}, {'score': 212025.6875}, {'score': 203981.75}, {'score': 187306.625}, {'score': 172016.078125}, {'score': 220650.09375}, {'score': 167069.6875}, {'score': 168638.515625}, {'score': 154275.90625}, {'score': 174776.78125}, {'score': 200102.90625}, {'score': 163031.875}, {'score': 177337.1875}, {'score': 194393.625}, {'score': 182359.53125}, {'score': 156962.0625}, {'score': 169478.75}, {'score': 127024.3203125}, {'score': 183981.46875}, {'score': 182846.96875}, {'score': 196573.25}, {'score': 202719.671875}, {'score': 188999.84375}, {'score': 199423.6875}, {'score': 196383.953125}, {'score': 136039.9375}, {'score': 294090.75}, {'score': 207447.21875}, {'score': 265044.3125}, {'score': 208231.828125}, {'score': 167688.125}, {'score': 175472.15625}, {'score': 194376.6875}, {'score': 247113.78125}, {'score': 314444.5625}, {'score': 190607.96875}, {'score': 159888.890625}, {'score': 155204.71875}, {'score': 190686.21875}, {'score': 179054.90625}, {'score': 186748.78125}, {'score': 186542.65625}, {'score': 223670.359375}, {'score': 230574.21875}, {'score': 219357.640625}, {'score': 183930.59375}, {'score': 200074.03125}, {'score': 216503.15625}, {'score': 207764.59375}, {'score': 149915.84375}, {'score': 168975.4375}, {'score': 210520.15625}, {'score': 139822.875}, {'score': 197302.96875}, {'score': 217384.203125}, {'score': 239602.0}, {'score': 224098.6875}, {'score': 187473.84375}, {'score': 187852.875}, {'score': 189687.59375}, {'score': 181015.0}, {'score': 203568.28125}, {'score': 191175.84375}, {'score': 119812.1484375}, {'score': 218033.53125}, {'score': 187604.0625}, {'score': 197012.828125}, {'score': 205968.0625}, {'score': 178913.3125}, {'score': 197320.96875}, {'score': 86442.8515625}, {'score': 242307.703125}, {'score': 234473.71875}, {'score': 192140.625}, {'score': 170919.375}, {'score': 212593.84375}, {'score': 187830.71875}, {'score': 210648.96875}, {'score': 173419.125}, {'score': 164789.5625}, {'score': 261989.4375}, {'score': 134528.46875}, {'score': 198700.671875}, {'score': 189879.28125}, {'score': 218129.6875}, {'score': 187784.5}, {'score': 130447.7578125}, {'score': 190052.875}, {'score': 200006.59375}, {'score': 201024.53125}, {'score': 225343.53125}, {'score': 170290.65625}, {'score': 205958.5}, {'score': 159396.1875}, {'score': 239198.5625}, {'score': 132877.71875}, {'score': 194708.625}, {'score': 164910.65625}, {'score': 178295.84375}, {'score': 161528.5}, {'score': 209400.375}, {'score': 238895.6875}, {'score': 133341.75}, {'score': 197126.96875}, {'score': 207027.375}, {'score': 37430.7578125}, {'score': 224589.21875}, {'score': 228310.75}, {'score': 208347.328125}, {'score': 151035.171875}, {'score': 216275.3125}, {'score': 69820.8046875}, {'score': 208348.53125}, {'score': 177677.9375}, {'score': 226743.03125}, {'score': 199336.5}, {'score': 168078.828125}, {'score': 180240.40625}, {'score': 203363.34375}, {'score': 194305.484375}, {'score': 131508.953125}, {'score': 193766.40625}, {'score': 208456.65625}, {'score': 166541.546875}, {'score': 226719.4375}, {'score': 212951.90625}, {'score': 235488.390625}, {'score': 211147.1875}, {'score': 164107.5}, {'score': 143931.5}, {'score': 154907.875}, {'score': 236707.859375}, {'score': 168159.84375}, {'score': 166052.734375}, {'score': 205996.5625}, {'score': 153085.03125}, {'score': 260799.09375}, {'score': 219378.84375}, {'score': 109699.3828125}, {'score': 177578.3125}, {'score': 168606.28125}, {'score': 160412.28125}, {'score': 192542.609375}, {'score': 197203.75}, {'score': 215522.09375}, {'score': 322372.3125}, {'score': 189616.65625}, {'score': 227148.890625}, {'score': 232066.375}, {'score': 185172.1875}, {'score': 190285.9375}, {'score': 207058.875}, {'score': 171622.90625}, {'score': 265101.4375}, {'score': 249714.15625}, {'score': 243163.125}, {'score': 226044.40625}, {'score': 75559.3046875}, {'score': 170179.8125}, {'score': 180319.09375}, {'score': 200126.796875}, {'score': 226812.59375}, {'score': 169434.6875}, {'score': 211273.5625}, {'score': 180057.3125}, {'score': 214982.0}, {'score': 147677.734375}, {'score': 123039.828125}, {'score': 171313.46875}, {'score': 256451.125}, {'score': 207521.0}, {'score': 191746.65625}, {'score': 150365.078125}, {'score': 227608.5625}, {'score': 168237.53125}, {'score': 177261.3125}, {'score': 194870.53125}, {'score': 184945.875}, {'score': 232810.28125}, {'score': 194212.96875}, {'score': 262288.96875}, {'score': 222251.15625}, {'score': 300421.71875}, {'score': 204735.625}, {'score': 180067.6875}, {'score': 229892.71875}, {'score': 232080.203125}, {'score': 143898.828125}, {'score': 237944.6875}, {'score': 194688.3125}, {'score': 200836.71875}, {'score': 127647.7109375}, {'score': 209047.9375}, {'score': 162796.625}, {'score': 178242.46875}, {'score': 257470.53125}, {'score': 125372.1796875}, {'score': 179392.59375}, {'score': 167272.625}, {'score': 199308.625}, {'score': 206789.71875}, {'score': 178225.96875}, {'score': 255486.40625}, {'score': 176425.90625}, {'score': 214935.84375}, {'score': 162681.59375}, {'score': 160502.625}, {'score': 184613.59375}, {'score': 164233.0}, {'score': 192383.1875}, {'score': 182276.375}, {'score': 219549.75}, {'score': 186688.5}, {'score': 139284.125}, {'score': 204046.890625}, {'score': 184153.71875}, {'score': 167786.3125}, {'score': 207289.0}, {'score': 175979.546875}, {'score': 233287.46875}, {'score': 233514.3125}, {'score': 126142.0078125}, {'score': 160436.125}, {'score': 133950.921875}, {'score': 189819.0625}, {'score': 220776.1875}, {'score': 193226.3125}, {'score': 184645.375}, {'score': 143003.21875}, {'score': 223829.8125}, {'score': 178946.859375}, {'score': 160457.59375}, {'score': 173502.6875}, {'score': 302702.40625}, {'score': 196422.40625}, {'score': 202789.8125}, {'score': 166825.34375}, {'score': 171255.578125}, {'score': 224587.78125}, {'score': 186263.71875}, {'score': 148452.65625}, {'score': 175150.671875}, {'score': 197635.84375}, {'score': 202109.46875}, {'score': 215715.8125}, {'score': 188009.4375}, {'score': 96870.9765625}, {'score': 172463.671875}, {'score': 179597.1875}, {'score': 254484.34375}, {'score': 163368.375}, {'score': 171707.9375}, {'score': 185584.34375}, {'score': 179361.9375}, {'score': 160918.71875}, {'score': 154040.9375}, {'score': 144850.84375}, {'score': 164148.96875}, {'score': 185051.5625}, {'score': 182362.59375}, {'score': 185664.3125}, {'score': 234592.125}, {'score': 139443.8125}, {'score': 244569.875}, {'score': 245721.71875}, {'score': 207642.59375}, {'score': 212819.375}, {'score': 198760.46875}, {'score': 141146.6875}, {'score': 77464.4921875}, {'score': 202974.96875}, {'score': 182169.53125}, {'score': 135137.75}, {'score': 208834.71875}, {'score': 130795.375}, {'score': 232323.71875}, {'score': 121040.2265625}, {'score': 174798.375}, {'score': 225153.65625}, {'score': 163220.46875}, {'score': 215617.3125}, {'score': 169127.0625}, {'score': 178060.625}, {'score': 108281.1171875}, {'score': 196608.734375}, {'score': 226019.65625}, {'score': 217506.4375}, {'score': 144343.21875}, {'score': 192981.71875}, {'score': 139037.625}, {'score': 173523.921875}, {'score': 159437.40625}, {'score': 195148.1875}, {'score': 251878.875}, {'score': 201675.46875}, {'score': 126075.4765625}, {'score': 148188.34375}, {'score': 200769.65625}, {'score': 191047.90625}, {'score': 197830.5625}, {'score': 181934.6875}, {'score': 151609.4375}, {'score': 283062.5625}, {'score': 160029.375}, {'score': 220124.125}, {'score': 170854.046875}, {'score': 183234.5}, {'score': 218166.21875}, {'score': 183550.09375}, {'score': 162713.84375}, {'score': 143137.90625}, {'score': 187084.46875}, {'score': 185627.0}, {'score': 272811.78125}, {'score': 209026.34375}, {'score': 181633.65625}, {'score': 123230.5234375}, {'score': 172024.125}, {'score': 201215.90625}, {'score': 214088.6875}, {'score': 205736.71875}, {'score': 177641.59375}, {'score': 190867.59375}, {'score': 89979.1953125}, {'score': 159016.75}, {'score': 190289.1875}, {'score': 191470.125}, {'score': 218408.6875}, {'score': 159147.59375}, {'score': 204767.21875}, {'score': 210765.65625}, {'score': 182319.9375}, {'score': 173458.421875}, {'score': 154796.09375}, {'score': 174390.953125}, {'score': 129081.6171875}, {'score': 222684.28125}, {'score': 140856.625}, {'score': 192148.875}, {'score': 165950.234375}, {'score': 127425.0078125}, {'score': 246828.5625}, {'score': 193533.75}, {'score': 159473.75}, {'score': 138189.75}, {'score': 207234.9375}, {'score': 146715.390625}, {'score': 197150.34375}, {'score': 214383.84375}, {'score': 210601.0}, {'score': 218346.890625}, {'score': 189815.4375}, {'score': 185033.46875}, {'score': 131774.09375}, {'score': 175328.625}, {'score': 234217.34375}, {'score': 188123.65625}, {'score': 182769.3125}, {'score': 173158.78125}, {'score': 156212.875}, {'score': 166792.4375}, {'score': 202673.28125}, {'score': 226720.09375}, {'score': 200205.5}, {'score': 162666.59375}, {'score': 172893.53125}, {'score': 184570.5}, {'score': 174453.75}, {'score': 176940.15625}, {'score': 183050.6875}, {'score': 163251.3125}, {'score': 201215.59375}, {'score': 223179.171875}, {'score': 163605.0625}, {'score': 171920.78125}, {'score': 197104.6875}, {'score': 188354.34375}, {'score': 176436.53125}, {'score': 179368.15625}, {'score': 177748.421875}, {'score': 191531.625}, {'score': 219025.3125}, {'score': 174608.921875}, {'score': 180840.8125}, {'score': 195578.96875}, {'score': 219525.609375}, {'score': 148731.71875}, {'score': 186377.09375}, {'score': 183893.109375}, {'score': 157109.59375}, {'score': 145030.09375}, {'score': 158130.46875}, {'score': 191175.6875}, {'score': 197641.375}, {'score': 98288.7890625}, {'score': 246711.5625}, {'score': 247714.625}, {'score': 217718.546875}, {'score': 344026.84375}, {'score': 217470.34375}, {'score': 229504.125}, {'score': 179907.09375}, {'score': 235738.46875}, {'score': 160948.28125}, {'score': 222491.9375}, {'score': 254904.34375}, {'score': 184410.90625}, {'score': 157613.53125}, {'score': 173358.78125}, {'score': 228040.25}, {'score': 205128.1875}, {'score': 200655.5625}, {'score': 226100.90625}, {'score': 214125.109375}, {'score': 206452.75}, {'score': 173038.1875}, {'score': 118327.640625}, {'score': 323593.125}, {'score': 124775.1171875}, {'score': 186232.53125}, {'score': 171255.84375}, {'score': 197257.40625}, {'score': 176028.125}, {'score': 199811.5}, {'score': 136714.8125}, {'score': 211015.46875}, {'score': 181638.96875}, {'score': 220676.65625}, {'score': 218909.9375}, {'score': 170480.5625}, {'score': 193102.03125}, {'score': 155266.875}, {'score': 218789.3125}, {'score': 166684.921875}, {'score': 183573.9375}, {'score': 180998.21875}, {'score': 176251.9375}, {'score': 233208.0}, {'score': 165820.859375}, {'score': 172570.328125}, {'score': 185678.53125}, {'score': 192606.21875}, {'score': 180580.46875}, {'score': 193188.09375}, {'score': 107445.6484375}, {'score': 223112.90625}, {'score': 181644.65625}, {'score': 170533.765625}, {'score': 226993.09375}, {'score': 191003.34375}, {'score': 174822.1875}, {'score': 205585.015625}, {'score': 138599.375}, {'score': 173916.21875}, {'score': 172626.46875}, {'score': 237298.0625}, {'score': 215449.8125}, {'score': 145446.46875}, {'score': 179153.0}, {'score': 203843.46875}, {'score': 213920.96875}, {'score': 166368.75}, {'score': 211897.21875}, {'score': 134437.328125}, {'score': 178163.625}, {'score': 183837.46875}, {'score': 172585.75}, {'score': 194128.28125}, {'score': 222689.90625}, {'score': 148715.125}, {'score': 188717.125}, {'score': 176390.625}, {'score': 189058.5}, {'score': 243842.4375}, {'score': 214690.890625}, {'score': 181900.5}, {'score': 184048.46875}, {'score': 178505.5}, {'score': 180908.671875}, {'score': 174708.9375}, {'score': 170388.5625}, {'score': 153490.21875}, {'score': 219818.03125}, {'score': 190628.921875}, {'score': 149511.890625}, {'score': 206965.4375}, {'score': 168705.84375}, {'score': 184326.28125}, {'score': 137419.1875}, {'score': 124952.0859375}, {'score': 107579.5078125}, {'score': 202482.5}, {'score': 258788.5}, {'score': 161982.5}, {'score': 210084.0}, {'score': 178832.5625}, {'score': 190122.4375}, {'score': 99112.2265625}, {'score': 166812.90625}, {'score': 149728.46875}, {'score': 201495.9375}, {'score': 156749.46875}, {'score': 199259.9375}, {'score': 187235.59375}, {'score': 148568.125}, {'score': 253573.15625}, {'score': 172645.34375}, {'score': 167355.9375}, {'score': 347726.8125}, {'score': 131010.9453125}, {'score': 199600.0625}, {'score': 202813.46875}, {'score': 222552.5625}, {'score': 177173.9375}, {'score': 417084.96875}, {'score': 168327.65625}, {'score': 210205.5}, {'score': 204142.5}, {'score': 193021.46875}, {'score': 278374.96875}, {'score': 204280.21875}, {'score': 178909.1875}, {'score': 193600.25}, {'score': 200392.15625}, {'score': 202748.6875}, {'score': 170153.34375}, {'score': 225312.9375}, {'score': 156843.125}, {'score': 345342.875}, {'score': 179977.390625}, {'score': 247337.0}, {'score': 157461.09375}, {'score': 234530.28125}, {'score': 179580.375}, {'score': 46438.0859375}, {'score': 208808.5625}, {'score': 73646.9296875}, {'score': 162560.96875}, {'score': 241387.9375}, {'score': 196495.875}, {'score': 163585.5625}, {'score': 232034.296875}, {'score': 124208.1328125}, {'score': 211120.03125}, {'score': 130052.8515625}, {'score': 211715.875}, {'score': 179087.53125}, {'score': 180780.03125}, {'score': 202341.375}, {'score': 206904.6875}, {'score': 166909.0625}, {'score': 195032.125}, {'score': 162890.65625}, {'score': 200827.6875}, {'score': 183252.59375}, {'score': 156420.15625}, {'score': 211258.8125}, {'score': 179094.265625}, {'score': 139853.875}, {'score': 134995.84375}, {'score': 233396.0625}, {'score': 225157.1875}, {'score': 184379.96875}, {'score': 214217.78125}, {'score': 199311.1875}, {'score': 214167.796875}, {'score': 68152.8359375}, {'score': 197270.25}, {'score': 158883.21875}, {'score': 238117.09375}, {'score': 221530.3125}, {'score': 207943.046875}, {'score': 202618.15625}, {'score': 149944.96875}, {'score': 250371.40625}, {'score': 188565.34375}, {'score': 218552.953125}, {'score': 139599.65625}, {'score': 193318.03125}, {'score': 193032.9375}, {'score': 188255.28125}, {'score': 183870.09375}, {'score': 233118.90625}, {'score': 168791.578125}, {'score': 182035.71875}, {'score': 203320.375}, {'score': 169203.25}, {'score': 181519.1875}, {'score': 221585.4375}, {'score': 145382.1875}, {'score': 223797.21875}, {'score': 197985.203125}, {'score': 229145.984375}, {'score': 222770.71875}, {'score': 179545.71875}, {'score': 183911.65625}, {'score': 132065.6875}, {'score': 148608.359375}, {'score': 177878.25}, {'score': 183085.3125}, {'score': 232478.3125}, {'score': 174466.34375}, {'score': 187334.265625}, {'score': 127738.8515625}, {'score': 234830.375}, {'score': 167348.5625}, {'score': 229154.34375}, {'score': 165217.75}, {'score': 180452.0}, {'score': 221447.90625}, {'score': 290793.21875}, {'score': 178539.40625}, {'score': 193740.734375}, {'score': 183479.75}, {'score': 228296.359375}, {'score': 209831.09375}, {'score': 186514.875}, {'score': 176816.234375}, {'score': 186051.84375}, {'score': 76322.4921875}, {'score': 218521.9375}, {'score': 161489.515625}, {'score': 224263.78125}, {'score': 155579.4375}, {'score': 229668.03125}, {'score': 171960.390625}, {'score': 242079.65625}, {'score': 227363.375}, {'score': 158964.34375}, {'score': 150082.875}, {'score': 167598.0}, {'score': 182804.78125}, {'score': 123380.6171875}, {'score': 172224.109375}, {'score': 222256.578125}, {'score': 225381.0}, {'score': 157072.28125}, {'score': 177076.28125}, {'score': 182490.96875}, {'score': 142174.8125}, {'score': 182017.875}, {'score': 209176.625}, {'score': 227085.3125}, {'score': 194333.25}, {'score': 160941.890625}, {'score': 219983.9375}, {'score': 392931.34375}, {'score': 233297.3125}, {'score': 203809.765625}, {'score': 227080.71875}, {'score': 190356.53125}, {'score': 187225.5}, {'score': 209345.53125}, {'score': 230935.875}, {'score': 157922.0}, {'score': 201185.4375}, {'score': 212031.15625}, {'score': 194063.5}, {'score': 183308.4375}, {'score': 154600.625}, {'score': 114985.8359375}, {'score': 174495.71875}, {'score': 120974.2421875}, {'score': 222100.3125}, {'score': 146720.96875}, {'score': 146271.140625}, {'score': 165522.21875}, {'score': 189885.8125}, {'score': 194468.625}, {'score': 166136.40625}, {'score': 203284.3125}, {'score': 199406.6875}, {'score': 177900.78125}, {'score': 259179.03125}, {'score': 257939.15625}, {'score': 214244.234375}, {'score': 167426.15625}, {'score': 210442.4375}, {'score': 226717.03125}, {'score': 160801.90625}, {'score': 193503.15625}, {'score': 191233.125}, {'score': 200143.1875}, {'score': 437950.3125}, {'score': 176437.921875}, {'score': 169191.265625}, {'score': 182030.5625}, {'score': 199273.25}, {'score': 161751.4375}, {'score': 158696.59375}, {'score': 270526.15625}, {'score': 225542.03125}, {'score': 138610.75}, {'score': 159546.1875}, {'score': 155104.890625}, {'score': 148614.21875}, {'score': 168205.53125}, {'score': 221522.625}, {'score': 213346.9375}, {'score': 205932.53125}, {'score': 197013.25}, {'score': 258351.53125}, {'score': 125133.6953125}, {'score': 164585.59375}, {'score': 234384.0}, {'score': 145277.53125}, {'score': 151033.296875}, {'score': 160603.453125}, {'score': 201747.9375}, {'score': 210060.21875}, {'score': 182324.1875}, {'score': 177849.46875}, {'score': 197508.6875}, {'score': 250187.5625}, {'score': 177795.234375}, {'score': 184608.890625}, {'score': 192333.0625}, {'score': 227766.0}, {'score': 148370.46875}, {'score': 233097.359375}, {'score': 162998.09375}, {'score': 251830.28125}, {'score': 166964.8125}, {'score': 199867.4375}, {'score': 253822.125}, {'score': 185689.5}, {'score': 174901.03125}, {'score': 193922.125}, {'score': 114962.15625}, {'score': 221419.453125}, {'score': 108218.2890625}, {'score': 159096.09375}, {'score': 142774.28125}, {'score': 181044.9375}, {'score': 186621.28125}, {'score': 103212.8828125}, {'score': 161579.0}, {'score': 253400.90625}, {'score': 192428.96875}, {'score': 205817.09375}, {'score': 212605.34375}, {'score': 196783.46875}, {'score': 199530.6875}, {'score': 142376.34375}, {'score': 179928.34375}, {'score': 140379.125}, {'score': 191489.46875}, {'score': 169248.859375}, {'score': 197490.125}, {'score': 193227.171875}, {'score': 157637.59375}, {'score': 214023.125}, {'score': 185329.34375}, {'score': 413887.625}, {'score': 216934.1875}, {'score': 251018.78125}, {'score': 140064.25}, {'score': 163769.171875}, {'score': 200401.8125}, {'score': 186458.71875}, {'score': 179386.375}, {'score': 141375.1875}, {'score': 137496.640625}, {'score': 179781.4375}, {'score': 212454.796875}, {'score': 198287.359375}, {'score': 207807.859375}, {'score': 225057.5625}, {'score': 203707.96875}, {'score': 177600.375}, {'score': 188579.4375}, {'score': 192330.59375}, {'score': 174582.453125}, {'score': 201514.640625}, {'score': 93664.1171875}, {'score': 174781.015625}, {'score': 242076.4375}, {'score': 174927.28125}, {'score': 195926.09375}, {'score': 195412.6875}, {'score': 183894.90625}, {'score': 197305.5}, {'score': 158293.28125}, {'score': 197481.1875}, {'score': 161944.28125}, {'score': 170706.9375}, {'score': 171814.875}, {'score': 204297.59375}, {'score': 193587.3125}, {'score': 190632.375}, {'score': 161753.484375}, {'score': 220449.84375}, {'score': 243118.84375}, {'score': 236815.96875}, {'score': 164292.3125}, {'score': 207102.484375}, {'score': 198401.71875}, {'score': 179079.03125}, {'score': 182369.53125}, {'score': 186366.6875}, {'score': 195407.3125}, {'score': 206758.59375}, {'score': 177098.1875}, {'score': 143656.1875}, {'score': 193174.734375}, {'score': 264045.96875}, {'score': 184804.96875}, {'score': 209241.171875}, {'score': 165911.5625}, {'score': 298873.28125}, {'score': 192210.03125}, {'score': 136105.84375}, {'score': 181997.25}, {'score': 238776.34375}, {'score': 171151.125}, {'score': 216410.59375}, {'score': 188402.96875}, {'score': 191649.0625}, {'score': 187600.46875}, {'score': 197231.875}, {'score': 160860.5625}, {'score': 199106.34375}, {'score': 195665.0625}, {'score': 297930.03125}, {'score': 190150.875}, {'score': 231434.28125}, {'score': 151477.03125}, {'score': 193007.9375}, {'score': 178677.296875}, {'score': 106677.265625}, {'score': 173989.21875}, {'score': 204181.65625}, {'score': 180835.875}, {'score': 174718.96875}, {'score': 446863.0625}, {'score': 236669.0}, {'score': 189926.15625}, {'score': 238901.625}, {'score': 210123.3125}, {'score': 196079.5625}, {'score': 137011.546875}, {'score': 278546.6875}, {'score': 148265.71875}, {'score': 133513.28125}, {'score': 156883.03125}, {'score': 204785.65625}, {'score': 199610.875}, {'score': 222544.84375}, {'score': 174171.765625}, {'score': 209942.0625}, {'score': 245063.5625}, {'score': 212832.5}, {'score': 187517.96875}, {'score': 160506.78125}, {'score': 196851.109375}, {'score': 201113.84375}, {'score': 189954.0625}, {'score': 198136.75}, {'score': 184333.046875}, {'score': 182177.09375}, {'score': 206644.25}, {'score': 133658.59375}, {'score': 141506.03125}, {'score': 216941.09375}, {'score': 143519.375}, {'score': 179020.828125}, {'score': 184830.03125}, {'score': 183587.78125}, {'score': 205155.96875}, {'score': 199012.9375}, {'score': 248904.9375}, {'score': 251624.15625}, {'score': 192979.625}, {'score': 159626.390625}, {'score': 130915.9921875}, {'score': 158960.03125}, {'score': 206701.78125}, {'score': 107397.4140625}, {'score': 186515.3125}, {'score': 191923.15625}, {'score': 184255.625}, {'score': 151683.609375}, {'score': 126874.5703125}, {'score': 242579.75}, {'score': 216894.96875}, {'score': 86367.90625}, {'score': 180843.90625}, {'score': 206579.9375}, {'score': 228761.625}, {'score': 195652.25}, {'score': 154271.78125}, {'score': 218519.78125}, {'score': 169876.3125}, {'score': 113710.5859375}, {'score': 185801.84375}, {'score': 207678.640625}, {'score': 186408.4375}, {'score': 184072.625}, {'score': 234637.28125}, {'score': 129272.3828125}, {'score': 350072.5}, {'score': 200559.4375}, {'score': 198348.828125}, {'score': 338667.125}, {'score': 150223.375}, {'score': 217780.4375}, {'score': 220728.9375}, {'score': 192377.40625}, {'score': 158467.4375}, {'score': 181555.625}, {'score': 159207.171875}, {'score': 198850.4375}, {'score': 161244.96875}, {'score': 190570.03125}, {'score': 160231.90625}, {'score': 329961.625}, {'score': 180429.40625}, {'score': 126784.96875}, {'score': 231600.65625}, {'score': 207045.40625}, {'score': 160467.15625}, {'score': 167439.5}, {'score': 195123.3125}, {'score': 173496.859375}, {'score': 185606.09375}, {'score': 178771.203125}, {'score': 189728.8125}, {'score': 163065.125}, {'score': 169201.3125}, {'score': 182216.984375}, {'score': 187292.0625}, {'score': 169676.359375}, {'score': 188038.8125}, {'score': 176958.25}, {'score': 201557.25}, {'score': 204944.8125}, {'score': 132475.3125}, {'score': 205008.40625}, {'score': 259211.8125}, {'score': 210071.53125}, {'score': 182450.421875}, {'score': 286968.5625}, {'score': 219950.28125}, {'score': 200178.6875}, {'score': 158226.09375}, {'score': 181326.265625}, {'score': 155906.65625}, {'score': 119212.328125}, {'score': 159342.28125}, {'score': 197170.53125}, {'score': 186654.875}, {'score': 362246.0625}, {'score': 127974.4921875}, {'score': 132229.71875}, {'score': 222772.9375}, {'score': 172863.34375}, {'score': 172470.625}, {'score': 74466.7109375}, {'score': 219352.78125}, {'score': 172923.5}, {'score': 202805.34375}, {'score': 167209.234375}, {'score': 184586.03125}, {'score': 198244.25}, {'score': 182265.375}, {'score': 184653.859375}, {'score': 199761.953125}, {'score': 205336.75}, {'score': 193357.796875}, {'score': 183484.28125}, {'score': 205575.140625}, {'score': 185440.53125}, {'score': 198482.3125}, {'score': 185710.09375}, {'score': 232751.09375}, {'score': 187343.453125}, {'score': 146351.359375}, {'score': 204542.59375}, {'score': 349168.40625}, {'score': 178959.71875}, {'score': 114575.0390625}, {'score': 167627.671875}, {'score': 158423.109375}, {'score': 122696.3203125}, {'score': 155886.921875}, {'score': 164693.8125}, {'score': 189947.9375}, {'score': 193147.53125}, {'score': 222270.28125}, {'score': 182927.90625}, {'score': 179842.78125}, {'score': 171114.6875}, {'score': 237494.421875}, {'score': 245880.59375}, {'score': 186408.828125}, {'score': 124983.5390625}, {'score': 233707.84375}, {'score': 186158.59375}, {'score': 204231.1875}, {'score': 250194.0}, {'score': 227028.84375}, {'score': 195744.0}, {'score': 214009.8125}, {'score': 183852.28125}, {'score': 138134.171875}, {'score': 313327.71875}, {'score': 274223.8125}, {'score': 184790.375}, {'score': 146710.34375}, {'score': 157633.4375}, {'score': 249935.40625}, {'score': 238401.90625}, {'score': 157644.78125}, {'score': 164803.40625}, {'score': 215738.84375}, {'score': 215893.796875}, {'score': 207706.59375}, {'score': 209149.375}, {'score': 213150.15625}, {'score': 190785.9375}, {'score': 76582.7109375}, {'score': 198025.5625}, {'score': 192666.53125}, {'score': 183941.71875}, {'score': 165533.125}, {'score': 148093.3125}, {'score': 200332.0625}, {'score': 198879.9375}, {'score': 154335.5}, {'score': 123653.8359375}, {'score': 217470.09375}, {'score': 190445.8125}, {'score': 149040.8125}, {'score': 167123.125}, {'score': 235710.09375}, {'score': 216272.5625}, {'score': 169019.703125}, {'score': 184554.9375}, {'score': 201411.21875}, {'score': 213295.3125}, {'score': 158044.96875}, {'score': 204241.6875}, {'score': 188001.15625}, {'score': 180799.46875}, {'score': 158580.6875}, {'score': 174526.46875}, {'score': 189254.4375}, {'score': 201818.15625}, {'score': 188808.21875}, {'score': 174684.875}, {'score': 185358.5625}, {'score': 128734.734375}, {'score': 91600.0234375}, {'score': 211812.34375}, {'score': 205458.3125}, {'score': 233842.375}, {'score': 206177.125}, {'score': 197544.4375}, {'score': 195570.859375}, {'score': 168525.09375}, {'score': 177048.375}, {'score': 155708.75}, {'score': 208483.25}, {'score': 265741.0625}, {'score': 232151.6875}, {'score': 191399.859375}, {'score': 221360.09375}, {'score': 188896.25}, {'score': 205557.0}, {'score': 219684.21875}, {'score': 135339.625}, {'score': 206301.65625}, {'score': 148222.21875}, {'score': 139295.34375}, {'score': 256354.84375}, {'score': 165508.96875}, {'score': 166773.390625}, {'score': 252002.25}, {'score': 189065.765625}, {'score': 173414.15625}, {'score': 183531.28125}, {'score': 148540.78125}, {'score': 184663.1875}, {'score': 174542.78125}, {'score': 223244.6875}, {'score': 224851.6875}, {'score': 211826.984375}, {'score': 255432.03125}, {'score': 189327.828125}, {'score': 167545.484375}, {'score': 317338.9375}, {'score': 201386.4375}, {'score': 203403.8125}, {'score': 223489.59375}, {'score': 197754.5}, {'score': 172415.75}, {'score': 193562.59375}, {'score': 165582.96875}, {'score': 223553.375}, {'score': 188693.4375}, {'score': 172095.03125}, {'score': 88051.3046875}, {'score': 157075.75}, {'score': 146059.03125}, {'score': 204577.75}, {'score': 196288.15625}, {'score': 194813.8125}, {'score': 123864.3046875}, {'score': 160254.71875}, {'score': 170108.53125}, {'score': 378175.5625}, {'score': 157152.390625}, {'score': 188489.21875}, {'score': 227232.75}, {'score': 188386.40625}, {'score': 192004.375}, {'score': 183761.90625}, {'score': 176236.78125}, {'score': 200088.21875}, {'score': 189615.5}, {'score': 234497.8125}, {'score': 144466.75}, {'score': 171530.03125}, {'score': 198413.1875}, {'score': 195107.21875}, {'score': 129465.6328125}, {'score': 172121.875}, {'score': 161521.25}, {'score': 226742.84375}, {'score': 238786.03125}, {'score': 166452.4375}, {'score': 160621.15625}, {'score': 168516.703125}, {'score': 192003.40625}, {'score': 221733.71875}, {'score': 183893.5}, {'score': 204651.28125}, {'score': 170401.28125}, {'score': 221125.59375}, {'score': 181256.5625}, {'score': 186240.96875}, {'score': 231151.375}, {'score': 186577.953125}, {'score': 199870.875}, {'score': 232118.0625}, {'score': 162294.125}, {'score': 203368.515625}, {'score': 217251.53125}, {'score': 146462.03125}, {'score': 138256.96875}, {'score': 147651.8125}, {'score': 193711.15625}, {'score': 181855.3125}, {'score': 151664.609375}, {'score': 191587.34375}, {'score': 176207.78125}, {'score': 182806.890625}, {'score': 187637.34375}, {'score': 165031.171875}, {'score': 159708.96875}, {'score': 230593.5625}, {'score': 250974.9375}, {'score': 120519.1484375}, {'score': 142413.703125}, {'score': 160150.265625}, {'score': 241598.1875}, {'score': 179501.796875}, {'score': 143199.640625}, {'score': 171763.203125}, {'score': 201109.15625}, {'score': 219926.84375}, {'score': 250860.21875}, {'score': 215301.578125}, {'score': 170709.46875}, {'score': 203553.96875}, {'score': 261041.6875}, {'score': 173782.4375}, {'score': 235160.71875}, {'score': 203539.4375}, {'score': 314720.8125}, {'score': 217076.0625}, {'score': 141401.609375}, {'score': 160719.296875}, {'score': 185198.6875}, {'score': 227331.03125}, {'score': 381819.875}, {'score': 184560.421875}, {'score': 195796.65625}, {'score': 172985.0}, {'score': 209219.6875}, {'score': 170549.53125}, {'score': 189225.53125}, {'score': 312028.84375}, {'score': 214025.9375}, {'score': 190070.28125}, {'score': 204201.03125}, {'score': 192354.6875}, {'score': 184177.875}, {'score': 156679.609375}, {'score': 185933.53125}, {'score': 196908.71875}, {'score': 57276.15234375}, {'score': 232891.96875}, {'score': 242397.96875}, {'score': 209684.59375}, {'score': 210498.9375}, {'score': 189697.4375}, {'score': 269409.875}, {'score': 180529.78125}, {'score': 183278.796875}, {'score': 191337.4375}, {'score': 172499.5}, {'score': 228628.1875}, {'score': 128348.9453125}, {'score': 230562.46875}, {'score': 186713.0625}, {'score': 239244.96875}, {'score': 186991.4375}, {'score': 136688.96875}, {'score': 227056.21875}, {'score': 180024.40625}, {'score': 182767.0625}, {'score': 149051.640625}, {'score': 227284.84375}, {'score': 165657.640625}, {'score': 284211.96875}, {'score': 217225.0}, {'score': 155974.4375}, {'score': 172175.9375}, {'score': 222670.0625}, {'score': 192906.21875}, {'score': 159548.71875}, {'score': 244467.1875}, {'score': 235160.875}, {'score': -93035.7890625}, {'score': 156997.75}, {'score': 267885.84375}, {'score': 195105.71875}, {'score': 215651.6875}, {'score': 157721.78125}, {'score': 223336.484375}, {'score': 240496.5625}, {'score': 201226.90625}, {'score': 253620.75}, {'score': 143580.65625}, {'score': 211081.34375}, {'score': 224065.71875}, {'score': 165878.8125}, {'score': 160435.59375}, {'score': 153822.0}, {'score': 212572.296875}, {'score': 255724.59375}, {'score': 216661.25}, {'score': 220148.46875}, {'score': 229817.59375}, {'score': 199827.21875}, {'score': 271751.0625}, {'score': 224009.71875}, {'score': 170557.765625}, {'score': 488975.3125}, {'score': 247388.71875}, {'score': 183534.4375}, {'score': 153803.25}, {'score': 200189.90625}, {'score': 30398.4375}, {'score': 151887.71875}, {'score': 183454.875}, {'score': 157939.359375}, {'score': 174371.078125}, {'score': 232997.5}, {'score': 253342.8125}, {'score': 182379.125}, {'score': 198226.59375}, {'score': 174183.375}, {'score': 164695.96875}, {'score': 194205.84375}, {'score': 212480.484375}, {'score': 164555.6875}, {'score': 185813.671875}, {'score': 258047.84375}, {'score': 235269.890625}, {'score': 273732.96875}, {'score': 166666.390625}, {'score': 153891.625}, {'score': 158086.46875}, {'score': 150012.5}, {'score': 181057.03125}, {'score': 190201.59375}, {'score': 202037.703125}, {'score': 158396.8125}, {'score': 167353.625}, {'score': 152219.3125}, {'score': 171083.328125}, {'score': 222450.453125}, {'score': 195999.828125}, {'score': 140912.5625}, {'score': 192668.125}, {'score': 222267.71875}, {'score': 179959.03125}, {'score': 192914.0625}, {'score': 115331.4921875}, {'score': 196178.4375}, {'score': 272405.75}, {'score': 188153.625}, {'score': 201290.375}, {'score': 186729.65625}, {'score': 189393.5}, {'score': 195402.0625}, {'score': 262762.03125}, {'score': 535596.6875}, {'score': 111106.8046875}, {'score': 200205.9375}, {'score': 197461.578125}, {'score': 194531.46875}, {'score': 195642.84375}, {'score': 220358.59375}, {'score': 194135.125}, {'score': 564609.0625}, {'score': 153259.765625}, {'score': 275549.34375}, {'score': 232819.765625}, {'score': 192426.0}, {'score': 160287.15625}, {'score': 191893.96875}, {'score': 178709.609375}, {'score': 236917.96875}, {'score': 202488.625}, {'score': 229852.125}, {'score': 214494.3125}, {'score': 197165.03125}, {'score': 191657.21875}, {'score': 197515.1875}, {'score': 187627.96875}, {'score': 153924.8125}, {'score': 225608.390625}, {'score': 198911.59375}, {'score': 162492.5}, {'score': 186229.875}, {'score': 207599.21875}, {'score': 202102.625}, {'score': 206985.8125}, {'score': 209348.5625}, {'score': 184708.25}, {'score': 189265.6875}, {'score': 165333.375}, {'score': 245803.1875}, {'score': 180341.484375}, {'score': 195782.515625}, {'score': 155588.25}, {'score': 168886.53125}, {'score': 159360.875}, {'score': 182020.3125}, {'score': 214629.46875}, {'score': 154034.03125}, {'score': 205984.21875}, {'score': 216869.0}, {'score': 183097.96875}, {'score': 160670.0625}, {'score': 188993.96875}, {'score': 215093.28125}, {'score': 129997.296875}, {'score': 185395.859375}, {'score': 203787.28125}, {'score': 219856.5625}, {'score': 178439.34375}, {'score': 211196.65625}, {'score': 212200.0}, {'score': 194398.59375}, {'score': 171678.09375}, {'score': 138029.0625}, {'score': 261384.78125}, {'score': 129012.4609375}, {'score': 173178.21875}, {'score': 137128.53125}, {'score': 219697.28125}, {'score': 203134.484375}, {'score': 143176.046875}, {'score': 165680.375}, {'score': 236053.34375}, {'score': 206422.4375}, {'score': 188022.0}, {'score': 192674.46875}, {'score': 149696.9375}, {'score': 195108.375}, {'score': 225415.9375}, {'score': 108030.203125}, {'score': 128773.6796875}, {'score': 195186.90625}, {'score': 161269.140625}, {'score': 175568.4375}, {'score': 188173.46875}, {'score': 184601.703125}, {'score': 193518.46875}, {'score': 249716.6875}, {'score': 212209.28125}, {'score': 179996.265625}, {'score': 185128.5625}, {'score': 157720.34375}, {'score': 121655.921875}, {'score': 187405.5}, {'score': 156506.734375}, {'score': 201301.40625}, {'score': 275520.90625}, {'score': 163866.65625}, {'score': 185608.03125}, {'score': 175054.15625}, {'score': 188343.28125}, {'score': 167888.78125}, {'score': 210607.875}, {'score': 121248.6171875}, {'score': 219293.109375}, {'score': 201496.03125}, {'score': 248752.90625}, {'score': 196725.109375}, {'score': 173711.125}, {'score': 136277.984375}, {'score': 171977.84375}, {'score': 173457.0625}]}

In \[110\]:

    #Extracting the predicted scores
    predicted_results = np.array([r['score'] for r in predicted_result['predictions']])
    print(predictions)

    [199673.375    151846.03125  116469.078125 ... 136277.984375 171977.84375
     173457.0625  ]

In \[113\]:

    #Creating a Pandas dataframe to view the actual and predicted values side by side
    house_value_compare_df = pd.DataFrame({'Actual_Median_House_Value': y_test.flatten(), 'Predicted_Median_House_Value': predicted_results.flatten()})
    print(house_value_compare_df)

          Actual_Median_House_Value  Predicted_Median_House_Value
    0                       98900.0                 199673.375000
    1                      153000.0                 151846.031250
    2                       91300.0                 116469.078125
    3                      345200.0                 229660.750000
    4                      344100.0                 219234.468750
    ...                         ...                           ...
    6738                    83800.0                 196725.109375
    6739                    68800.0                 173711.125000
    6740                   130700.0                 136277.984375
    6741                   253900.0                 171977.843750
    6742                   209800.0                 173457.062500

    [6743 rows x 2 columns]

In \[112\]:

    #Deleting the endpoint
    linear_regressor.delete_endpoint()